<a href="https://colab.research.google.com/github/hawa1983/Capstone/blob/main/ibx_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feasibility of Implementing the GTFS Stops-to-GeoDataFrame Workflow Entirely in Python

## Executive summary

From my perspective as the implementer, **every task implied by our conversation and the attached notebook-style pipeline can be implemented using Python code**, including: reading GTFS `stops.txt` from GitHub, handling parsing issues, filtering by `location_type`, constructing a `GeoDataFrame` from `stop_lon/stop_lat`, reprojecting, buffering, spatial joins, mapping, exporting, and even scaling to large GTFS tables with an embedded analytical engine (DuckDB). The key nuance is what “entirely in Python” means in practice:

Python can orchestrate all steps, but several geospatial capabilities rely on **native (non-Python) libraries** under the hood—most notably **GEOS, PROJ, and GDAL** that power Shapely/GeoPandas’ geometry, CRS transforms, and file I/O. GeoPandas explicitly notes this dependency stack and that installation can be challenging without managed environments. citeturn2search1turn2search6

Similarly, any pipeline that downloads MTA GTFS feeds, Census ACS data, NYC Open Data (Socrata), or GBFS feeds is still “Python-implemented,” but requires **network access** and sometimes **API keys / rate-limit hygiene** (especially for Census and Socrata). citeturn10search1turn11search0turn12view0

## Inventory of tasks and code patterns implied by the conversation and attached pipeline

Below is a consolidated inventory of the operations I need to support, grouped by the work they perform. (This includes the explicit items you listed and the additional geospatial/data-engineering patterns present in the attached pipeline.)

Data acquisition and ingestion includes reading GTFS text files (especially `stops.txt`) from a GitHub-hosted location and learning to avoid the “blob page HTML” trap; pandas can read URLs directly, but only if the URL resolves to the raw file. The GitHub UI explicitly provides a “Raw” view for unstylized file contents—which is what scripts should target. citeturn6search15turn5view0

Robust parsing includes handling `ParserError` conditions, malformed rows, delimiter/quoting mismatches, encoding issues, and choosing appropriate parsing engines. Pandas supports an `engine` parameter and `on_bad_lines` behavior to skip/warn/process malformed lines. citeturn5view0turn1view2

GTFS semantics includes treating `stop_lat`/`stop_lon` as the stop coordinates and using `location_type` to distinguish platforms/stops vs stations and other location types. The GTFS reference defines the allowed `location_type` values (0/empty stop/platform; 1 station; 2 entrance/exit; 3 generic node; 4 boarding area) and specifies when `stop_lat/stop_lon` are required. citeturn7view0turn1view1

Geospatial modeling includes creating point geometries from longitude/latitude (`points_from_xy`), setting CRS (usually EPSG:4326 for GTFS), reprojecting to a projected CRS for accurate distance/area work (`to_crs`), building buffers, and forming study-area bounding boxes/unions. GeoPandas explicitly documents that `points_from_xy` assumes x=longitude and y=latitude for geographic coordinates. citeturn1view3turn1view5

Spatial analytics includes spatial joins (`sjoin`) with predicates (`within`, `intersects`, sometimes `dwithin`), nearest-neighbor queries using spatial indexes (`sindex`), clipping layers, and computing derived metrics (counts/densities, proximity distances). GeoPandas documents `sjoin` and notes that operations are planar (not geodesic). citeturn1view4turn3search3

Scaling and performance includes using DuckDB to query large GTFS tables (e.g., `stop_times.txt`, `trips.txt`, `routes.txt`) without loading everything into pandas; DuckDB documents CSV “auto detection” and provides `read_csv_auto`/`read_csv` paths to handle varied CSV dialects. citeturn3search1turn3search1

Outputs include exporting tabular results (CSV) and geospatial layers (GeoPackage, GeoJSON, etc.). GeoPandas’ `to_file` writes via the Pyogrio or Fiona I/O engines (both GDAL/OGR-backed), and supports drivers such as `"GPKG"`. citeturn2search0turn9search2

Visualization includes static plotting (GeoPandas/Matplotlib) and interactive HTML maps (Folium). Folium’s documentation shows standard patterns for MarkerCluster and saving maps to HTML. citeturn3search8turn3search0  
If I choose Kepler.gl, I must account for Jupyter extension requirements (Node + JupyterLab extensions) beyond Python-only pip installs. citeturn4search2

## Python implementation patterns for each major task

I can implement every item you enumerated with mainstream Python libraries; the sections below focus on the most “load-bearing” patterns and parameters I need to get right.

For reading GTFS from GitHub correctly, the core rule is: **don’t use a `.../blob/...` URL with pandas** because that serves an HTML page. Instead, I use a raw-content URL (either via GitHub’s “Raw” view or the `raw.githubusercontent.com` format), which is explicitly meant to show raw file content without styling. citeturn6search15

For parsing and handling malformed rows, pandas provides the knobs I need. The `engine` parameter governs the parsing backend, and `on_bad_lines` defines how to handle lines with too many fields; it supports `'error'`, `'warn'`, `'skip'`, or a callable (noting callable signatures differ across engines). citeturn5view0turn1view2  
Separators and quoting are equally important: `sep` can be a single-character delimiter or regex, but regex delimiters force the Python engine and can ignore quoted data. citeturn5view1

For filtering stations using `location_type`, I must treat GTFS semantics carefully: a station is `location_type=1` per the GTFS reference; platforms/stops are `location_type=0` or empty. citeturn7view0turn1view1  
In real feeds, `location_type` might be missing or blank; in those cases, robust code defaults missing values to 0 (stop/platform) and filters accordingly.

For creating a GeoDataFrame from stop longitude and latitude, GeoPandas gives me the canonical pattern: `geopandas.points_from_xy(x, y, crs=...)` and then `GeoDataFrame(..., geometry=...)`. GeoPandas explicitly documents that for geographic coordinates it assumes `x = longitude` and `y = latitude`. citeturn1view3

For reprojection, `GeoDataFrame.to_crs()` is the standard. It requires that the current CRS is set, and it transforms coordinates assuming **planar segments** (it does not perform geodesic transformations of entire geometries). citeturn1view5  
For city-scale distance/buffer analysis, I typically reproject WGS84 points (EPSG:4326) into an **appropriate projected CRS** (e.g., a local State Plane or UTM zone).

For buffering, I use `GeoSeries.buffer(distance)` with distances expressed in the units of the CRS. GeoPandas provides the buffer method at the GeoSeries level. citeturn4search13  
The key operational point is that GeoPandas geometry ops are planar. The `sjoin` docs explicitly state every operation is planar, and GeoPandas warns when buffering in a geographic CRS because degrees are not linear units; the practical mitigation is reprojecting first. citeturn1view4turn1view5

For spatial joins, I use `geopandas.sjoin(left, right, predicate=...)`, selecting `predicate="within"` for point-in-polygon counts, or `predicate="intersects"` for polygon overlap. GeoPandas documents `sjoin` and supported predicates, and its spatial index documentation explains that `sindex` uses Shapely’s STRtree. citeturn1view4turn3search3

For mapping, Folium produces interactive HTML entirely from Python code, but requires a browser (or notebook renderer) to view. Marker clustering is a documented plugin pattern. citeturn3search0turn3search8  
If I use Kepler.gl, I still write Python, but the Jupyter integration specifically requires **Node** and JupyterLab extensions—so the runtime environment is not “Python-only.” citeturn4search2

For scaling, DuckDB is a strong option for GTFS tables like `stop_times.txt` that can become very large. DuckDB’s CSV auto-detection documentation describes how it infers delimiter/quoting/types because CSV is not self-describing. citeturn3search1

## Tasks that require non-Python system dependencies or external services

The most important boundary is that many “Python geospatial” libraries are Python interfaces over mature C/C++ geospatial stacks.

GeoPandas explicitly states it depends on **GEOS, GDAL, and PROJ**, and warns these can be a challenge to install in some environments. citeturn2search1  
Shapely likewise describes itself as manipulating planar geometries and is built on the GEOS library. citeturn2search6  
GeoPandas file output uses Pyogrio or Fiona, both of which bind to GDAL/OGR drivers. GeoPandas `to_file` documents that it writes any OGR data source supported by Pyogrio or Fiona. citeturn2search0turn9search6turn2search5  
This matters if I interpret “entirely in Python” as “no compiled dependencies”: that stricter goal is generally **not realistic** for serious vector GIS operations because GEOS/PROJ/GDAL are foundational.

If the workflow includes raster operations (even “if needed”), Rasterio is Python-controlled but depends on GDAL and its dependencies. citeturn9search0

Interactive visualization can introduce non-Python requirements. Folium is pure Python on the generation side, but renders in a web browser (Leaflet JS). citeturn3search8  
Kepler.gl’s Jupyter mode requires Node and JupyterLab extensions per official docs. citeturn4search2

External services and APIs are also not “purely local,” even though I call them from Python:

- ACS via Census API is explicitly an API-based access method. citeturn10search1turn10search5  
- NYC Open Data is served via Socrata’s SODA API; Socrata documents the `within_box` geospatial filter I would use for bounding-box queries. citeturn11search0turn11search15  
- Socrata geospatial ordering can be subtle: point geometries use GeoJSON/WKT “lon,lat” order, while `within_box` uses “latitude, longitude corners” in conventional order; Socrata documents this distinction. citeturn11search0turn11search17  
- GBFS is a published standard; `station_information.json` is a specified feed and is part of the official “Current Version” reference. citeturn12view0turn12view1  
- MTA data is published under its developer resources portal. citeturn10search0turn10search8

## Environment constraints treated as open-ended

Several practical constraints are not specified and materially affect implementation choices:

Dataset scale is unknown: a small single-feed GTFS can be handled with pandas; multi-feed (bus-by-borough) plus `stop_times.txt` often motivates DuckDB/Dask/Polars patterns. DuckDB provides a pragmatic “SQL over CSV” approach without requiring a separate database. citeturn3search1

Execution environment is unknown (local laptop vs Colab vs server). This primarily impacts geospatial dependency installation: pip wheels may “just work” in some setups, but conda-forge is often the smoother path for GDAL/PROJ/GEOS stacks, which GeoPandas highlights as an installation complexity. citeturn2search1turn2search4

Operating system is not stated; Windows environments in particular often benefit from conda distributions for GDAL-family libraries; Docker can provide reproducibility across OSes at the cost of container overhead.

Network access and credentials are unspecified: ACS requires a Census API key for production usage; Socrata APIs can require app tokens for better rate limits; some environments block outbound HTTP.

## Compatibility table for pip-only, conda-forge, and Docker approaches

The table below compares three realistic ways I can run the full pipeline.

| Approach | What I mean by it | Typical install commands | Pros | Cons / gotchas |
|---|---|---|---|---|
| pip-only (wheels) | Use `pip` in venv; rely on prebuilt wheels bundling native libs where available | `python -m venv .venv` → `pip install -U pip` → `pip install pandas geopandas shapely pyproj pyogrio fiona duckdb folium requests networkx` | Fast to start; fits many notebook environments | May hit binary incompatibilities around GDAL/OGR stacks; Pyogrio docs explicitly warn pip installs can encounter GDAL version mismatches depending on what gets compiled/installed. citeturn2search4turn2search1 |
| conda-forge | Use conda/mamba to install GeoPandas stack with consistent GDAL/PROJ/GEOS | `mamba create -n geo python=3.12 -c conda-forge geopandas pyogrio fiona gdal pandas shapely pyproj duckdb folium requests networkx` | Most reliable for GDAL/PROJ/GEOS; aligns with GeoPandas guidance that base C libs can be challenging and installation recommendations matter. citeturn2search1turn2search4 | Heavier environment tooling; slower cold-start than pip |
| Docker | Containerize OS + dependencies + code | Use a GDAL-capable base image or micromamba image; then install conda-forge stack | Best reproducibility; good for CI and deployment | Requires Docker runtime; file permissions/volumes; for Kepler.gl you still may need Node tooling inside container if running JupyterLab extensions. citeturn4search2 |

A useful implementation detail for GeoPandas I/O performance: GeoPandas notes that as of version 1.0 the default I/O engine changed from Fiona to Pyogrio for performance reasons, and the engine can be configured. citeturn9search2

## Reference code snippets for robust Python implementations

The snippets below are intentionally defensive and cover the workflow elements you listed: raw GitHub reads, parsing failures, filtering station rows, creating GeoDataFrames, reprojecting for distance, buffering, spatial joins, mapping, and exporting.

### Reading GTFS `stops.txt` from GitHub robustly

# Section A — Spatial Framework

## Purpose of the Code

The purpose of this code is to combine two separate transit stop datasets into one clean, unified file. Specifically, it retrieves an existing GTFS stops file from your GitHub repository and merges it with a second file containing IBX stop coordinates. The end goal is to produce an updated stops dataset that includes both the original transit system stops and the IBX stops in a single, usable file.

First, the script ensures that GitHub links are usable for data processing. Standard GitHub “blob” URLs display files as web pages rather than raw text, which makes them unsuitable for automated downloads. The code converts those links into raw file URLs so the actual data content can be accessed directly. It then downloads the file and checks that the response is not an HTML page. This validation step prevents errors caused by accidentally loading a webpage instead of the intended dataset.

Next, the script loads both the original stops file and the IBX stops file into structured tables using pandas. Once loaded, it standardizes key fields such as latitude, longitude, and location type. This ensures that geographic coordinates are stored as numeric values and that required columns follow a consistent format. Standardizing data types is critical because mapping systems, routing algorithms, and GTFS validation tools rely on properly formatted numerical coordinates and consistent schema structures.

After cleaning and standardizing the data, the two datasets are merged into one combined table. The IBX stops are appended to the original stops list, and any duplicate stop identifiers are resolved by keeping the most recent entry. This ensures that IBX stop information overrides older entries if the same stop ID appears in both datasets. The result is a unified dataset without duplicate stop IDs and with consistent formatting across all records.

Finally, the combined dataset is saved as a new file. This output serves as an updated GTFS-compatible stops file that includes IBX stops alongside the existing system stops. Overall, the script functions as a streamlined data pipeline: it downloads, validates, cleans, merges, and exports transit stop data into a structured and reusable format suitable for further analysis, mapping, or integration into a GTFS feed.


In [1]:
# Enable postponed evaluation of type hints (improves forward reference handling)
from __future__ import annotations

# Standard library imports
import re                     # Used for pattern matching GitHub URLs
from io import StringIO       # Allows treating text as a file-like object

# Third-party libraries
import requests               # Handles HTTP requests to download files
import pandas as pd           # Data manipulation and CSV processing


def github_blob_to_raw(url: str) -> str:
    """
    Convert a standard GitHub 'blob' URL into a raw content URL.

    Example:
    https://github.com/org/repo/blob/main/file.txt
    ->
    https://raw.githubusercontent.com/org/repo/main/file.txt

    If the URL is already raw (or not a GitHub blob link),
    return it unchanged.
    """
    # Match GitHub blob URL pattern
    m = re.match(r"^https?://github\.com/([^/]+)/([^/]+)/blob/([^/]+)/(.*)$", url)
    if not m:
        # If it does not match the blob format, return original URL
        return url

    # Extract organization, repository, branch, and file path
    org, repo, branch, path = m.groups()

    # Construct equivalent raw GitHub URL
    return f"https://raw.githubusercontent.com/{org}/{repo}/{branch}/{path}"


def read_text_csv(url: str, *, timeout: int = 60) -> pd.DataFrame:
    """
    Download a CSV/text file from a URL and load it into a pandas DataFrame.

    - Automatically converts GitHub blob URLs to raw URLs.
    - Verifies that the downloaded content is not HTML.
    - Parses the file as a comma-separated CSV.
    """

    # Ensure URL points to raw file content (not a GitHub HTML page)
    raw_url = github_blob_to_raw(url)

    # Send HTTP GET request to retrieve file contents
    r = requests.get(raw_url, timeout=timeout)

    # Raise an error if the request failed (e.g., 404 or 500)
    r.raise_for_status()

    # Inspect the beginning of the response to ensure it is not HTML
    head = r.text[:2000].lstrip().lower()
    if head.startswith("<!doctype html") or "<html" in head[:200]:
        raise ValueError(
            "URL did not resolve to raw text content (looks like HTML). "
            "Use a raw.githubusercontent.com URL."
        )

    # Read the downloaded text into a pandas DataFrame
    # - sep="," specifies comma-delimited file
    # - on_bad_lines="warn" skips malformed rows with a warning
    # - low_memory=False prevents dtype guessing issues
    return pd.read_csv(
        StringIO(r.text),
        sep=",",
        encoding="utf-8",
        on_bad_lines="warn",
        engine="c",
        low_memory=False,
    )


# ----------------------------
# Load GTFS and IBX stop files
# ----------------------------

# Load main GTFS stops file from GitHub
stops_df = read_text_csv(
    "https://github.com/hawa1983/Capstone/blob/main/stops.txt"
)

# Load IBX stops file from raw GitHub URL
ibx_df = read_text_csv(
    "https://raw.githubusercontent.com/hawa1983/Capstone/refs/heads/main/ibx%20stops%20coordinates.txt"
)


# ----------------------------
# Normalize Data Types
# ----------------------------

# Ensure latitude, longitude, and location_type
# are properly formatted in both datasets
for df in (stops_df, ibx_df):

    # Convert latitude and longitude to numeric values
    # Invalid values are converted to NaN
    df["stop_lat"] = pd.to_numeric(df["stop_lat"], errors="coerce")
    df["stop_lon"] = pd.to_numeric(df["stop_lon"], errors="coerce")

    # Ensure location_type exists and is an integer
    if "location_type" in df.columns:
        df["location_type"] = (
            pd.to_numeric(df["location_type"], errors="coerce")
            .fillna(0)          # Replace missing values with 0
            .astype(int)        # Convert to integer
        )
    else:
        # If column is missing, create it with default value 0
        df["location_type"] = 0


# ----------------------------
# Merge Datasets
# ----------------------------

# Combine original stops and IBX stops into one DataFrame
combined_stops = pd.concat([stops_df, ibx_df], ignore_index=True)

# Remove duplicate stop_id entries
# If duplicates exist, keep the last occurrence (IBX overrides original)
combined_stops_df = (
    combined_stops
    .drop_duplicates(subset=["stop_id"], keep="last")
    .reset_index(drop=True)
)


# ----------------------------
# Save Combined File
# ----------------------------

# Define output filename
output_file = "stops_with_ibx.csv"

# Write merged dataset to CSV without row index
combined_stops_df.to_csv(output_file, index=False)

# Confirm successful save
print(f"File saved as: {output_file}")

display(combined_stops_df.tail(19))

File saved as: stops_with_ibx.csv


,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
1497,IBX_1,Roosevelt Avenue,40.7465,-73.8910,1,NaN
1498,IBX_2,Grand Avenue,40.7289,-73.8897,1,NaN
1499,IBX_3,Eliot Avenue,40.7255,-73.8792,1,NaN
1500,IBX_4,Metropolitan Avenue,40.7119,-73.8896,1,NaN
1501,IBX_5,Myrtle Avenue,40.6997,-73.9109,1,NaN
1502,IBX_6,Wilson Avenue,40.6871,-73.9030,1,NaN
1503,IBX_7,Atlantic Avenue,40.6835,-73.9083,1,NaN
1504,IBX_8,Sutter Avenue,40.6696,-73.9022,1,NaN
1505,IBX_9,Livonia Avenue,40.6644,-73.9002,1,NaN
1506,IBX_10,Linden Blvd,40.6555,-73.9060,1,NaN


In [2]:
print("stops_df:", len(stops_df))
print("ibx_df:", len(ibx_df))
print("combined_stops_df:", len(combined_stops_df))


stops_df: 1497
ibx_df: 19
combined_stops_df: 1516


In [3]:
combined_stops_df[combined_stops_df["stop_name"].astype(str).str.contains("Woodside|Bay Ridge|Jackson Heights|Atlantic", case=False, na=False)].tail(30)


,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
210,235,Atlantic Av-Barclays Ctr,40.684359,-73.977666,1,NaN
211,235N,Atlantic Av-Barclays Ctr,40.684359,-73.977666,0,235
212,235S,Atlantic Av-Barclays Ctr,40.684359,-73.977666,0,235
492,712,61 St-Woodside,40.745630,-73.902984,1,NaN
493,712N,61 St-Woodside,40.745630,-73.902984,0,712
494,712S,61 St-Woodside,40.745630,-73.902984,0,712
804,D24,Atlantic Av-Barclays Ctr,40.684460,-73.976890,1,NaN
805,D24N,Atlantic Av-Barclays Ctr,40.684460,-73.976890,0,D24
806,D24S,Atlantic Av-Barclays Ctr,40.684460,-73.976890,0,D24
1194,L24,Atlantic Av,40.675345,-73.903097,1,NaN


In [4]:
ibx_bad = ibx_df[ibx_df["stop_lat"].isna() | ibx_df["stop_lon"].isna()]
print("IBX rows with missing coords:", len(ibx_bad))
ibx_bad.head()


IBX rows with missing coords: 0


,stop_id,stop_name,stop_lat,stop_lon,location_type


## Filtering GTFS Stations from the Combined Stops Dataset

This code extracts only **station-level stops** from the merged GTFS dataset. In GTFS, the `location_type` field is used to classify different types of stops. A value of `1` represents a **station**, while other values (such as `0`) typically represent individual boarding stops or platforms.

First, the code retrieves the `location_type` column from the `combined_stops` DataFrame. If the column does not exist, it defaults to `0`, which prevents the code from failing due to a missing field. It then converts the values in this column to numeric form, coercing any invalid entries into missing values. Those missing values are replaced with `0`, and the column is cast to integers. This ensures that the filtering step works reliably and that all values follow GTFS enumeration rules.

Next, the code creates a new DataFrame called `stations_df` by selecting only the rows where `location_type` equals `1`. This isolates records that represent stations rather than platforms or other stop types. The `.copy()` method is used to create a separate DataFrame, avoiding potential warnings or unintended modifications to the original dataset.

Finally, the `display()` function shows the filtered station-level records. In summary, this code segment cleans and standardizes the `location_type` field, then filters the dataset to include only GTFS-defined stations.


In [5]:
# GTFS uses the "location_type" field to classify stop types.
# According to GTFS enumerations:
#   0 = Stop/Platform (default)
#   1 = Station
#   2 = Entrance/Exit
#   3 = Generic Node
#   4 = Boarding Area
# Here, we want to isolate only stations (location_type == 1).

# Safely retrieve the "location_type" column from the combined dataset.
# If the column does not exist, default to 0 (treat all rows as regular stops).
loc = combined_stops_df.get("location_type", 0)

# Convert the location_type values to numeric:
# - errors="coerce" converts invalid or non-numeric values to NaN
# - fillna(0) replaces missing values with 0 (default stop/platform)
# - astype(int) ensures values are stored as integers for reliable comparison
loc = pd.to_numeric(loc, errors="coerce").fillna(0).astype(int)

# Filter the dataset to include only rows where location_type == 1 (stations).
# loc.eq(1) creates a Boolean mask selecting only station rows.
# .copy() creates a separate DataFrame to avoid modifying the original data.
stations_df = combined_stops_df.loc[loc.eq(1)].copy()

# Display the filtered station-level stops.
# This allows you to inspect only GTFS-defined stations.
display(stations_df)


,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,101,Van Cortlandt Park-242 St,40.889248,-73.898583,1,NaN
3,103,238 St,40.884667,-73.900870,1,NaN
6,104,231 St,40.878856,-73.904834,1,NaN
9,106,Marble Hill-225 St,40.874561,-73.909831,1,NaN
12,107,215 St,40.869444,-73.915279,1,NaN
...,...,...,...,...,...,...
1511,IBX_15,McDonald Avenue,40.625500,-73.977700,1,NaN
1512,IBX_16,New Utrecht Avenue,40.620900,-73.990300,1,NaN
1513,IBX_17,8 Avenue,40.637200,-74.001700,1,NaN
1514,IBX_18,4 Avenue,40.645200,-74.011300,1,NaN


This matches the GTFS reference semantics: station rows are `location_type = 1`, while platforms/stops are `0` or empty. citeturn7view0turn1view1

### Validating latitude/longitude columns before geometry construction

## Validating and Cleaning GTFS Stop Coordinates

This code validates that the stop dataset contains proper geographic coordinate fields and removes any records with invalid latitude or longitude values.

First, it defines a set of required columns: `stop_lat` and `stop_lon`. These fields are mandatory in GTFS because they store the geographic coordinates of each stop. The code then checks whether these required columns are actually present in the `stops_df` DataFrame. If either column is missing, it raises a `ValueError` and stops execution. This prevents downstream errors and ensures the dataset meets minimum GTFS structural requirements.

Next, the code converts both the latitude and longitude columns to numeric values. If any values are non-numeric (for example, text or malformed entries), they are coerced into missing values (`NaN`). This step ensures the coordinate fields are stored in a numeric format suitable for geographic validation and mapping.

After conversion, the code applies geographic boundary checks. Valid latitude values must fall between -90 and 90 degrees, and valid longitude values must fall between -180 and 180 degrees. These are the universal bounds for real-world geographic coordinates. The `between()` function creates a logical mask that identifies rows where both latitude and longitude fall within valid ranges.

Finally, the dataset is filtered to keep only rows with valid coordinates. Any stops with missing, malformed, or out-of-range coordinates are removed. The `.copy()` method ensures that the filtered result is stored as a clean, independent DataFrame.

In summary, this code performs structural validation and geographic quality control on the GTFS stop data, ensuring that all remaining stops contain valid numeric latitude and longitude values within real-world bounds.


In [6]:
# --------------------------------------------
# Validate Required Coordinate Columns
# --------------------------------------------

# Define the set of required coordinate columns.
# GTFS requires stop_lat (latitude) and stop_lon (longitude)
# to properly define the geographic location of each stop.
required = {"stop_lat", "stop_lon"}

# Determine whether any required columns are missing
# by subtracting the existing columns from the required set.
missing = required - set(combined_stops_df.columns)

# If any required columns are missing, raise an error
# to stop execution and prevent downstream failures.
if missing:
    raise ValueError(f"Missing required columns: {missing}")


# --------------------------------------------
# Convert Coordinates to Numeric
# --------------------------------------------

# GTFS defines stop_lat and stop_lon as numeric latitude
# and longitude values (in decimal degrees).
# Convert both columns to numeric types:
# - errors="coerce" converts invalid values to NaN
combined_stops_df["stop_lat"] = pd.to_numeric(combined_stops_df["stop_lat"], errors="coerce")
combined_stops_df["stop_lon"] = pd.to_numeric(combined_stops_df["stop_lon"], errors="coerce")


# --------------------------------------------
# Validate Geographic Bounds
# --------------------------------------------

# Latitude must be between -90 and 90 degrees.
# Longitude must be between -180 and 180 degrees.
# The .between() method returns True for valid values
# and False for values outside these bounds (or NaN).

valid = (
    combined_stops_df["stop_lat"].between(-90, 90)
    & combined_stops_df["stop_lon"].between(-180, 180)
)

# --------------------------------------------
# Filter Out Invalid Rows
# --------------------------------------------

# Keep only rows where both latitude and longitude are valid.
# Rows with:
#   - Missing coordinates (NaN)
#   - Non-numeric values (converted to NaN)
#   - Out-of-range geographic values
# are removed.
#
# .copy() ensures we create a clean DataFrame
# and avoid modifying a view of the original.
combined_stops_df = combined_stops_df.loc[valid].copy()


# --------------------------------------------
# Display Cleaned Dataset
# --------------------------------------------

# Show the validated stops dataset.
# At this point, all rows have:
#   - Required coordinate columns
#   - Numeric latitude and longitude values
#   - Coordinates within real-world geographic bounds
display(combined_stops_df)


,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,101,Van Cortlandt Park-242 St,40.889248,-73.898583,1,NaN
1,101N,Van Cortlandt Park-242 St,40.889248,-73.898583,0,101
2,101S,Van Cortlandt Park-242 St,40.889248,-73.898583,0,101
3,103,238 St,40.884667,-73.900870,1,NaN
4,103N,238 St,40.884667,-73.900870,0,103
...,...,...,...,...,...,...
1511,IBX_15,McDonald Avenue,40.625500,-73.977700,1,NaN
1512,IBX_16,New Utrecht Avenue,40.620900,-73.990300,1,NaN
1513,IBX_17,8 Avenue,40.637200,-74.001700,1,NaN
1514,IBX_18,4 Avenue,40.645200,-74.011300,1,NaN


GTFS also allows `stop_lat/stop_lon` to be optional for some `location_type` values; robust code should allow nulls for those rows if I keep them. citeturn7view0

### Creating a GeoDataFrame from `stop_lon/stop_lat`


In [7]:
import geopandas as gpd

# --------------------------------------------
# Create Station Subset FROM the cleaned dataset
# --------------------------------------------

# GTFS defines location_type = 1 as a station.
# Use the already cleaned combined_stops_df to ensure
# coordinates are valid before creating geometry.
loc = combined_stops_df.get("location_type", 0)
loc = pd.to_numeric(loc, errors="coerce").fillna(0).astype(int)

# Filter only station-level records (location_type == 1)
stations_df = combined_stops_df.loc[loc.eq(1)].copy()


# --------------------------------------------
# Convert to GeoDataFrame
# --------------------------------------------

# GeoPandas points_from_xy expects:
#   x = longitude
#   y = latitude
# CRS "EPSG:4326" corresponds to standard WGS84 lat/lon.
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(
        stations_df["stop_lon"],
        stations_df["stop_lat"],
        crs="EPSG:4326"
    ),
)

# Display the spatial station dataset
display(stations_gdf)


,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,geometry
0,101,Van Cortlandt Park-242 St,40.889248,-73.898583,1,NaN,POINT (-73.89858 40.88925)
3,103,238 St,40.884667,-73.900870,1,NaN,POINT (-73.90087 40.88467)
6,104,231 St,40.878856,-73.904834,1,NaN,POINT (-73.90483 40.87886)
9,106,Marble Hill-225 St,40.874561,-73.909831,1,NaN,POINT (-73.90983 40.87456)
12,107,215 St,40.869444,-73.915279,1,NaN,POINT (-73.91528 40.86944)
...,...,...,...,...,...,...,...
1511,IBX_15,McDonald Avenue,40.625500,-73.977700,1,NaN,POINT (-73.9777 40.6255)
1512,IBX_16,New Utrecht Avenue,40.620900,-73.990300,1,NaN,POINT (-73.9903 40.6209)
1513,IBX_17,8 Avenue,40.637200,-74.001700,1,NaN,POINT (-74.0017 40.6372)
1514,IBX_18,4 Avenue,40.645200,-74.011300,1,NaN,POINT (-74.0113 40.6452)


## Reprojecting Station Data to a Local Coordinate System

This code is transforming the station GeoDataFrame from one coordinate reference system (CRS) to another.

Your `stations_gdf` was originally created using **EPSG:4326**, which is the standard WGS84 geographic coordinate system. In this system, locations are stored as latitude and longitude in decimal degrees. While this format is ideal for web maps and GPS, it is not well suited for measuring distances or areas because degrees are not consistent linear units.

The `.to_crs("EPSG:2263")` step converts the dataset into a projected coordinate system. EPSG:2263 is the New York State Plane coordinate system (Long Island zone), and its units are in feet. Projected systems use linear units like feet or meters, which makes them appropriate for spatial analysis tasks such as calculating distances between stations, buffering, clustering, or performing spatial joins.

Importantly, `.to_crs()` does not just relabel the data—it mathematically transforms the coordinate values into the new projection. That transformation requires that the original GeoDataFrame already has a defined CRS (which yours does: EPSG:4326).

The resulting `stations_proj` GeoDataFrame contains the same station records, but the geometry coordinates are now expressed in feet rather than degrees. This makes the data more suitable for accurate distance calculations and local spatial analysis in the New York City area.


In [8]:
# ---------------------------------------------------------
# Reproject GeoDataFrame to a Different Coordinate System
# ---------------------------------------------------------

# The original stations_gdf is in EPSG:4326 (WGS84),
# which stores coordinates as latitude/longitude in degrees.
# While this CRS is ideal for web maps and GPS,
# it is NOT ideal for measuring distances or areas because
# degrees are not linear units.

# The .to_crs() method transforms the geometry coordinates
# from their current CRS into a new coordinate reference system.
# IMPORTANT: The input GeoDataFrame must already have a defined CRS.

# Here we convert to EPSG:2263.
# EPSG:2263 = NAD83 / New York Long Island (State Plane)
# Units are in feet, making it suitable for:
#   - Distance calculations
#   - Buffer analysis
#   - Spatial clustering
#   - Local NYC mapping and analysis

stations_proj = stations_gdf.to_crs("EPSG:2263")  # NYC State Plane (feet)

# After transformation:
# - Geometry coordinates are no longer latitude/longitude
# - Coordinates are now projected X/Y values in feet
# - Attribute data (columns) remain unchanged
# - Only the geometry column is mathematically transformed

# Display the projected GeoDataFrame
# This allows inspection of the new projected coordinate values.
display(stations_proj)


,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,geometry
0,101,Van Cortlandt Park-242 St,40.889248,-73.898583,1,NaN,POINT (1012291.156 263271.208)
3,103,238 St,40.884667,-73.900870,1,NaN,POINT (1011660.704 261601.442)
6,104,231 St,40.878856,-73.904834,1,NaN,POINT (1010566.908 259483.047)
9,106,Marble Hill-225 St,40.874561,-73.909831,1,NaN,POINT (1009186.665 257916.747)
12,107,215 St,40.869444,-73.915279,1,NaN,POINT (1007681.798 256050.919)
...,...,...,...,...,...,...,...
1511,IBX_15,McDonald Avenue,40.625500,-73.977700,1,NaN,POINT (990440.272 167163.683)
1512,IBX_16,New Utrecht Avenue,40.620900,-73.990300,1,NaN,POINT (986942.815 165487.143)
1513,IBX_17,8 Avenue,40.637200,-74.001700,1,NaN,POINT (983778.179 171425.523)
1514,IBX_18,4 Avenue,40.645200,-74.011300,1,NaN,POINT (981114.151 174340.339)


In [9]:
import geopandas as gpd
import pandas as pd

# ---------------------------------------------------------
# 1) Load 2020 TIGER/Line Block Groups for New York State
# ---------------------------------------------------------
# NY State FIPS = 36. TIGER/Line 2020 block groups for NY are available as a zipped shapefile.
# GeoPandas can read zipped shapefiles directly from a URL.
tiger_bg_ny_url = "https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_36_bg.zip"  # NY block groups (2020) :contentReference[oaicite:0]{index=0}

bg_ny = gpd.read_file(tiger_bg_ny_url)

# Quick validation: inspect columns and CRS
print("Loaded rows:", len(bg_ny))
print("CRS:", bg_ny.crs)
print("Columns:", list(bg_ny.columns))

# ---------------------------------------------------------
# 2) Filter to NYC (Five Borough Counties) using COUNTYFP
# ---------------------------------------------------------
# NYC counties (boroughs) by county FIPS:
# 005 Bronx, 047 Kings (Brooklyn), 061 New York (Manhattan), 081 Queens, 085 Richmond (Staten Island)
nyc_countyfps = {"005", "047", "061", "081", "085"}

# TIGER 2020 block group fields commonly include:
# STATEFP, COUNTYFP, TRACTCE, BLKGRPCE, and GEOID (often GEOID/ GEOID20 depending on vintage)
# For TIGER2020 BG, fields are typically STATEFP and COUNTYFP (no '20' suffix), but we guard anyway.
state_col = "STATEFP" if "STATEFP" in bg_ny.columns else "STATEFP20"
county_col = "COUNTYFP" if "COUNTYFP" in bg_ny.columns else "COUNTYFP20"

# Keep only New York State (36) and NYC counties
bg_nyc = bg_ny.loc[
    (bg_ny[state_col].astype(str) == "36") &
    (bg_ny[county_col].astype(str).isin(nyc_countyfps))
].copy()

print("NYC block groups:", len(bg_nyc))

# ---------------------------------------------------------
# 3) (Optional but recommended) Reproject to match stations
# ---------------------------------------------------------
# If you plan to buffer stations and do distance-based calculations,
# you should work in a projected CRS (your stations_proj is EPSG:2263 in feet).
# Reproject block groups to the SAME CRS for spatial joins/overlays.
#
# NOTE: stations_proj.crs should be EPSG:2263 if you followed your earlier step.
bg_nyc_proj = bg_nyc.to_crs(stations_proj.crs)

# Preview the NYC block group GeoDataFrames
display(bg_nyc.head())
display(bg_nyc_proj.head())


Loaded rows: 16070
CRS: EPSG:4269
Columns: ['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry']
NYC block groups: 6807


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
24,36,061,023900,1,360610239001,Block Group 1,G5030,S,27517,0,+40.8322236,-073.9404112,"POLYGON ((-73.94112 40.83166, -73.94088 40.832..."
25,36,061,013900,1,360610139001,Block Group 1,G5030,S,23621,0,+40.7688543,-073.9868884,"POLYGON ((-73.98806 40.76979, -73.98666 40.769..."
26,36,061,007800,2,360610078002,Block Group 2,G5030,S,33890,0,+40.7471571,-073.9756186,"POLYGON ((-73.97673 40.74763, -73.97635 40.748..."
27,36,061,008900,1,360610089001,Block Group 1,G5030,S,20377,0,+40.7443158,-074.0010568,"POLYGON ((-74.00226 40.74521, -73.99942 40.744..."
28,36,061,008900,4,360610089004,Block Group 4,G5030,S,42006,0,+40.7458221,-074.0036736,"POLYGON ((-74.00511 40.7464, -74.00465 40.7470..."


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
24,36,061,023900,1,360610239001,Block Group 1,G5030,S,27517,0,+40.8322236,-073.9404112,"POLYGON ((1000544.606 242277.459, 1000609.46 2..."
25,36,061,013900,1,360610139001,Block Group 1,G5030,S,23621,0,+40.7688543,-073.9868884,"POLYGON ((987556.436 219732.621, 987946.195 21..."
26,36,061,007800,2,360610078002,Block Group 2,G5030,S,33890,0,+40.7471571,-073.9756186,"POLYGON ((990696.33 211658.547, 990801.847 211..."
27,36,061,008900,1,360610089001,Block Group 1,G5030,S,20377,0,+40.7443158,-074.0010568,"POLYGON ((983622.385 210775.651, 984410.163 21..."
28,36,061,008900,4,360610089004,Block Group 4,G5030,S,42006,0,+40.7458221,-074.0036736,"POLYGON ((982834.912 211211.061, 982960.722 21..."


## Estimate Population Within 0.5 Miles of Each Station (Area-Weighted Method)

Below is a complete workflow that:

1. Creates 0.5-mile station buffers (in feet, using EPSG:2263),
2. Loads 2020 ACS 5-year total population for block groups (B01003),
3. Joins ACS population to NYC block-group geometries,
4. Intersects buffers with block groups,
5. Applies area-weighted population allocation,
6. Produces population within 0.5 miles of each station.





In [10]:
# =========================================================
# Station 0.5-mile buffers + NYC Block Group intersections
# + Area-weighted population within 0.5 miles
# (FIXED: pop_density_0p5mi uses pandas Series, not numpy array)
# =========================================================

import geopandas as gpd
import pandas as pd
import numpy as np
import requests

# -----------------------------
# User config
# -----------------------------
ACS_YEAR = 2020  # ACS 2020 5-year
STATEFP = "36"
NYC_COUNTYFPS = ["005", "047", "061", "081", "085"]  # Bronx, Kings, New York, Queens, Richmond

FT_PER_MILE = 5280.0
BUFFER_DIST_FT = 0.5 * FT_PER_MILE  # 2,640 feet
SQFT_PER_SQMI = FT_PER_MILE ** 2

TIGER_BG_NY_URL = "https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_36_bg.zip"
ACS_BASE = f"https://api.census.gov/data/{ACS_YEAR}/acs/acs5"

# -----------------------------
# 0) Preconditions
# -----------------------------
if "stations_proj" not in globals():
    raise NameError(
        "stations_proj is not defined. You need a GeoDataFrame named stations_proj "
        "with columns ['stop_id','stop_name','geometry'] and a projected CRS (e.g., EPSG:2263)."
    )

required_cols = {"stop_id", "stop_name", "geometry"}
missing = required_cols - set(stations_proj.columns)
if missing:
    raise ValueError(f"stations_proj is missing required columns: {sorted(missing)}")

if stations_proj.crs is None:
    raise ValueError("stations_proj.crs is None. Set a CRS (ideally EPSG:2263) before buffering.")

# -----------------------------
# 1) Load TIGER block groups for NY (2020)
# -----------------------------
bg_ny = gpd.read_file(TIGER_BG_NY_URL)
print("Loaded TIGER BG rows:", len(bg_ny))
print("TIGER CRS:", bg_ny.crs)

state_col = "STATEFP" if "STATEFP" in bg_ny.columns else "STATEFP20"
county_col = "COUNTYFP" if "COUNTYFP" in bg_ny.columns else "COUNTYFP20"
geoid_col = "GEOID" if "GEOID" in bg_ny.columns else ("GEOID20" if "GEOID20" in bg_ny.columns else None)
if geoid_col is None:
    raise ValueError("Could not find GEOID column in TIGER BG file (expected GEOID or GEOID20).")

# -----------------------------
# 2) Filter to NYC block groups
# -----------------------------
bg_nyc = bg_ny.loc[
    (bg_ny[state_col].astype(str) == STATEFP) &
    (bg_ny[county_col].astype(str).isin(NYC_COUNTYFPS))
].copy()

print("NYC TIGER BG rows:", len(bg_nyc))

# -----------------------------
# 3) Reproject BGs to match stations CRS
# -----------------------------
bg_nyc_proj = bg_nyc.to_crs(stations_proj.crs)
print("Reprojected NYC BG CRS:", bg_nyc_proj.crs)

bg_nyc_proj["GEOID"] = bg_nyc_proj[geoid_col].astype(str)

# -----------------------------
# 4) Fetch ACS population for NYC BGs (B01003_001E)
# -----------------------------
def fetch_acs_bg_population_for_county(county_fips: str) -> pd.DataFrame:
    url = (
        f"{ACS_BASE}?get=B01003_001E&for=block%20group:*"
        f"&in=state:{STATEFP}%20county:{county_fips}%20tract:*"
    )
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    data = r.json()
    if not isinstance(data, list) or len(data) < 2:
        raise RuntimeError(f"Bad ACS payload for county {county_fips}: {str(data)[:200]}")

    df = pd.DataFrame(data[1:], columns=data[0])
    df["GEOID"] = (
        df["state"].astype(str).str.zfill(2) +
        df["county"].astype(str).str.zfill(3) +
        df["tract"].astype(str).str.zfill(6) +
        df["block group"].astype(str).str.zfill(1)
    )
    df["population"] = pd.to_numeric(df["B01003_001E"], errors="coerce").fillna(0)
    return df[["GEOID", "population"]]

acs_pop = pd.concat(
    [fetch_acs_bg_population_for_county(c) for c in NYC_COUNTYFPS],
    ignore_index=True
).drop_duplicates(subset=["GEOID"])

print("ACS BG pop rows:", len(acs_pop))

# -----------------------------
# 5) Build bg_pop (BG geometry + population)
# -----------------------------
bg_pop = bg_nyc_proj.merge(acs_pop, on="GEOID", how="left")
bg_pop["population"] = bg_pop["population"].fillna(0)

bg_pop["bg_area"] = bg_pop.geometry.area
bg_pop = bg_pop.loc[bg_pop["bg_area"] > 0].copy()

print("bg_pop rows (NYC BGs with geometry):", len(bg_pop))
print("bg_pop population > 0 share:", (bg_pop["population"] > 0).mean())

# -----------------------------
# 6) Build 0.5-mile station buffers (feet)
# -----------------------------
station_buffers = stations_proj[["stop_id", "stop_name", "geometry"]].copy()
station_buffers["geometry"] = station_buffers.geometry.buffer(BUFFER_DIST_FT)

# -----------------------------
# 7) Intersect buffers with BGs
# -----------------------------
intersections = gpd.overlay(
    station_buffers,
    bg_pop[["GEOID", "population", "bg_area", "geometry"]],
    how="intersection"
)

print("Intersections rows:", len(intersections))

# -----------------------------
# 8) Area-weighted population within buffer
# -----------------------------
intersections["intersect_area"] = intersections.geometry.area
intersections["area_ratio"] = np.where(
    intersections["bg_area"] > 0,
    intersections["intersect_area"] / intersections["bg_area"],
    0.0
).astype(float).clip(0, 1)

intersections["pop_within"] = intersections["population"] * intersections["area_ratio"]

pop_by_station = (
    intersections.groupby(["stop_id", "stop_name"], as_index=False)["pop_within"]
    .sum()
    .round()
    .astype({"pop_within": int})
    .rename(columns={"pop_within": "population_0p5mi"})
)

station_buffers = station_buffers.merge(pop_by_station, on=["stop_id", "stop_name"], how="left")
station_buffers["population_0p5mi"] = station_buffers["population_0p5mi"].fillna(0).astype(int)

# -----------------------------
# 9) Derived metrics + ranking  (FIXED)
# -----------------------------
station_buffers["buffer_area_sqmi"] = station_buffers.geometry.area / SQFT_PER_SQMI

# ✅ compute density in pandas so we can safely fillna/astype
station_buffers["pop_density_0p5mi"] = (
    station_buffers["population_0p5mi"]
    .div(station_buffers["buffer_area_sqmi"].replace({0: np.nan}))
    .round(0)
    .fillna(0)
    .astype(int)
)

station_buffers["pop_rank"] = (
    station_buffers["population_0p5mi"]
    .rank(ascending=False, method="min")
    .astype(int)
)

station_buffers_sorted = station_buffers.sort_values("population_0p5mi", ascending=False)

print("\nTop 10 stations by population within 0.5 miles:")
display(
    station_buffers_sorted[
        ["stop_id", "stop_name", "population_0p5mi", "pop_density_0p5mi", "pop_rank"]
    ].tail(10)
)

# -----------------------------
# 10) Preserve BG detail table for later features
# -----------------------------
station_bg_detail = intersections.copy()

detail_cols = [c for c in [
    "stop_id", "stop_name",
    "GEOID",
    "population", "bg_area", "intersect_area", "area_ratio", "pop_within"
] if c in station_bg_detail.columns]

display(station_bg_detail[detail_cols].tail())

# -----------------------------
# 11) Save outputs
# -----------------------------
station_buffers.to_csv("station_population_analysis_0p5mi.csv", index=False)
station_bg_detail.to_csv("station_blockgroup_detail_0p5mi.csv", index=False)

print("\nSaved: station_population_analysis_0p5mi.csv")
print("Saved: station_blockgroup_detail_0p5mi.csv")


Loaded TIGER BG rows: 16070
TIGER CRS: EPSG:4269
NYC TIGER BG rows: 6807
Reprojected NYC BG CRS: EPSG:2263
ACS BG pop rows: 6807
bg_pop rows (NYC BGs with geometry): 6807
bg_pop population > 0 share: 0.9390333480240929
Intersections rows: 23421

Top 10 stations by population within 0.5 miles:


,stop_id,stop_name,population_0p5mi,pop_density_0p5mi,pop_rank
498,S31,St George,5746,7328,509
482,S15,Prince's Bay,4538,5787,510
479,S11,Arthur Kill,4521,5766,511
478,S09,Tottenville,4095,5222,512
156,702,Mets-Willets Point,3443,4391,513
481,S14,Pleasant Plains,3319,4233,514
480,S13,Richmond Valley,2429,3098,515
517,IBX_19,Brooklyn Army Terminal,1640,2091,516
351,H04,Broad Channel,1200,1530,517
362,H19,Broad Channel,1187,1514,518


,stop_id,stop_name,GEOID,population,bg_area,intersect_area,area_ratio,pop_within
23416,IBX_19,Brooklyn Army Terminal,360470022003,940,1.031276e+06,8.673770e+04,0.084107,79.060772
23417,IBX_19,Brooklyn Army Terminal,360470030002,620,4.768618e+05,1.429062e+04,0.029968,18.580190
23418,IBX_19,Brooklyn Army Terminal,360470030001,925,7.091931e+05,5.502170e+05,0.775835,717.647586
23419,IBX_19,Brooklyn Army Terminal,360470022001,1038,1.293441e+06,1.308070e+05,0.101131,104.974005
23420,IBX_19,Brooklyn Army Terminal,360470034001,874,3.747025e+06,2.976980e+06,0.794491,694.385557



Saved: station_population_analysis_0p5mi.csv
Saved: station_blockgroup_detail_0p5mi.csv


# Section B — Variables

In [11]:
# =========================================================
# Urban Variable 1: Population within 0.5 mile (already built)
# Source: ACS 5-year B01003 Total Population (block group)
# Method: Area-weighted allocation using station_bg_detail area_ratio
# =========================================================

# Assumes you already computed:
# - bg_pop: NYC block groups with "GEOID", "population", "bg_area", geometry
# - station_buffers: buffer polygons with stop_id, stop_name, geometry
# - station_bg_detail (aka intersections): overlay(station_buffers, bg_pop), with area_ratio
#
# If your "population_0p5mi" is already in station_buffers, this is just a quick validation.

# 1) Validate required columns exist
required_cols = {"stop_id", "stop_name", "population", "area_ratio"}
missing = required_cols - set(station_bg_detail.columns)
if missing:
    raise ValueError(f"station_bg_detail missing columns: {missing}")

# 2) Recompute pop_within from stored block-group population and area weights
station_bg_detail["pop_within"] = station_bg_detail["population"] * station_bg_detail["area_ratio"]

# 3) Aggregate to station level (sum area-weighted population pieces)
pop_by_station = (
    station_bg_detail.groupby(["stop_id", "stop_name"])["pop_within"]
    .sum()
    .round()
    .astype(int)
    .reset_index()
    .rename(columns={"pop_within": "population_0p5mi"})
)

# 4) Attach back to station_buffers
station_buffers = station_buffers.drop(columns=["population_0p5mi"], errors="ignore").merge(
    pop_by_station, on=["stop_id", "stop_name"], how="left"
)
station_buffers["population_0p5mi"] = station_buffers["population_0p5mi"].fillna(0).astype(int)

display(station_buffers[["stop_id", "stop_name", "population_0p5mi"]].head())


,stop_id,stop_name,population_0p5mi
0,101,Van Cortlandt Park-242 St,16751
1,103,238 St,30765
2,104,231 St,43760
3,106,Marble Hill-225 St,38704
4,107,215 St,29362


In [12]:
# =========================================================
# Urban Variable 2: Employment within 0.5 mile (LODES jobs)
# FIX: Auto-detect a working LODES WAC URL instead of using a placeholder.
# =========================================================

import pandas as pd
import geopandas as gpd
import requests
from io import BytesIO

# --- SETTINGS ---
LODES_STATE = "ny"
LODES_YEAR = 2019          # try 2020 first; if not found, try 2019/2021/etc.
LODES_VERSIONS = ["LODES8", "LODES7"]  # try newest first, then fallback
SEGMENT = "S000"           # all segments (all workers)
JOBTYPE = "JT00"           # all jobs
TIMEOUT = 120

def find_lodes_wac_url(state: str, year: int) -> str:
    """
    Try common LODES WAC file name patterns across LODES versions and return the first working URL.
    This avoids relying on directory listings (which can be hard to fetch in some environments).
    """
    # Known base path format for LODES downloads
    bases = [f"https://lehd.ces.census.gov/data/lodes/{v}/{state}/wac/" for v in LODES_VERSIONS]

    # Common filename patterns seen in LODES releases
    # Pattern A (very common): <st>_wac_<seg>_<jt>_<year>.csv.gz
    # Pattern B (sometimes):   <st>_wac_<jt>_<seg>_<year>.csv.gz
    # Pattern C (rare):        <st>_wac_<seg>_<year>.csv.gz  (less detailed)
    candidates = [
        f"{state}_wac_{SEGMENT}_{JOBTYPE}_{year}.csv.gz",
        f"{state}_wac_{JOBTYPE}_{SEGMENT}_{year}.csv.gz",
        f"{state}_wac_{SEGMENT}_{year}.csv.gz",
    ]

    # Try each candidate; accept first HTTP 200
    for base in bases:
        for fname in candidates:
            url = base + fname
            try:
                r = requests.get(url, timeout=TIMEOUT, stream=True)
                if r.status_code == 200:
                    return url
            except requests.RequestException:
                # Try next candidate
                pass

    raise FileNotFoundError(
        f"Could not find a working WAC URL for state={state}, year={year}. "
        "Try a different year (e.g., 2019, 2021) or verify the LODES version available."
    )

# 1) Find a working WAC URL automatically
wac_url = find_lodes_wac_url(LODES_STATE, LODES_YEAR)
print("Using WAC URL:", wac_url)

# 2) Download and load the gzipped CSV
r = requests.get(wac_url, timeout=TIMEOUT)
r.raise_for_status()
wac = pd.read_csv(BytesIO(r.content), compression="gzip", dtype={"w_geocode": str})

# 3) Validate expected columns and keep total jobs
# In WAC, total jobs is typically "C000"
if "w_geocode" not in wac.columns or "C000" not in wac.columns:
    raise ValueError(f"Expected columns 'w_geocode' and 'C000'. Found: {list(wac.columns)[:40]}")

wac = wac[["w_geocode", "C000"]].rename(columns={"w_geocode": "block_geoid", "C000": "jobs"})
wac["jobs"] = pd.to_numeric(wac["jobs"], errors="coerce").fillna(0)

# ---------------------------------------------------------
# 4) Load NYC Census blocks (2020 TIGER), create centroids,
#    and attach LODES jobs to each workplace block
# ---------------------------------------------------------

# TIGER 2020 blocks for NY State
blocks_url = "https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/tl_2020_36_tabblock20.zip"
blocks = gpd.read_file(blocks_url)

# Filter to NYC counties using the same nyc_countyfps you used for block groups
blocks = blocks.loc[blocks["COUNTYFP20"].astype(str).isin(nyc_countyfps)].copy()

# Normalize GEOID field name to 'block_geoid' (15-digit block id)
blocks = blocks.rename(columns={"GEOID20": "block_geoid"})

# Keep only what we need and create centroid points for spatial join
blocks = blocks[["block_geoid", "geometry"]].copy()
blocks["geometry"] = blocks.geometry.centroid

# Reproject to match station buffers CRS (EPSG:2263)
blocks = blocks.to_crs(station_buffers.crs)

# Join LODES jobs onto blocks (inner keeps blocks that exist in WAC)
blocks_jobs = blocks.merge(wac, on="block_geoid", how="inner")
blocks_jobs["jobs"] = pd.to_numeric(blocks_jobs["jobs"], errors="coerce").fillna(0)

# ---------------------------------------------------------
# 5) Spatial join: workplace blocks within station buffers
# ---------------------------------------------------------
join_jobs = gpd.sjoin(
    blocks_jobs[["block_geoid", "jobs", "geometry"]],
    station_buffers[["stop_id", "stop_name", "geometry"]],
    how="inner",
    predicate="within",
)

# ---------------------------------------------------------
# 6) Aggregate jobs per station buffer and attach to stations
# ---------------------------------------------------------
jobs_by_station = (
    join_jobs.groupby(["stop_id", "stop_name"])["jobs"]
    .sum()
    .round()
    .astype(int)
    .reset_index()
    .rename(columns={"jobs": "jobs_0p5mi"})
)

station_buffers = station_buffers.merge(jobs_by_station, on=["stop_id", "stop_name"], how="left")
station_buffers["jobs_0p5mi"] = station_buffers["jobs_0p5mi"].fillna(0).astype(int)

# ---------------------------------------------------------
# 7) Display output
# ---------------------------------------------------------
display(
    station_buffers.sort_values("jobs_0p5mi", ascending=False)[
        ["stop_id", "stop_name", "jobs_0p5mi"]
    ].head(10)
)

display(station_buffers["jobs_0p5mi"].describe())



Using WAC URL: https://lehd.ces.census.gov/data/lodes/LODES8/ny/wac/ny_wac_S000_JT00_2019.csv.gz


/tmp/ipython-input-2556053195.py:88: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks["geometry"] = blocks.geometry.centroid


,stop_id,stop_name,jobs_0p5mi
261,D16,42 St-Bryant Pk,642321
174,724,5 Av,615870
447,R16,Times Sq-42 St,589011
178,902,Times Sq-42 St,570084
260,D15,47-50 Sts-Rockefeller Ctr,567866
177,901,Grand Central-42 St,551165
175,725,Times Sq-42 St,549469
24,127,Times Sq-42 St,547419
262,D17,34 St-Herald Sq,524997
448,R17,34 St-Herald Sq,508278


,jobs_0p5mi
count,518.000000
mean,58844.708494
std,112725.034329
min,474.000000
25%,6393.500000
50%,11738.000000
75%,35362.750000
max,642321.000000


In [13]:
!pip install osmnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 5.2 MB/s eta 0:00:00


In [14]:
# =========================================================
# Urban Variable 4: Street Intersection Density (TIGER Roads)
# Corrected: TIGER roads are county-level, not statewide
# =========================================================

import geopandas as gpd
import pandas as pd
import numpy as np

# NYC county FIPS codes
nyc_countyfps = {
    "005",  # Bronx
    "047",  # Kings (Brooklyn)
    "061",  # New York (Manhattan)
    "081",  # Queens
    "085",  # Richmond (Staten Island)
}

# ---------------------------------------------------------
# 1) Download TIGER 2020 roads by county and combine
# ---------------------------------------------------------

roads_list = []

for county in nyc_countyfps:
    roads_url = f"https://www2.census.gov/geo/tiger/TIGER2020/ROADS/tl_2020_36{county}_roads.zip"
    print("Downloading:", roads_url)
    roads_c = gpd.read_file(roads_url)
    roads_list.append(roads_c)

roads = pd.concat(roads_list, ignore_index=True)

print("Total NYC road segments:", len(roads))

# Reproject to match station buffers (EPSG:2263 feet)
roads = roads.to_crs(station_buffers.crs)

# ---------------------------------------------------------
# 2) Compute intersection points
# ---------------------------------------------------------

# Spatial self-join to find intersecting road segments
road_pairs = gpd.sjoin(
    roads[["geometry"]],
    roads[["geometry"]],
    how="inner",
    predicate="intersects"
)

# --- After sjoin ---
road_pairs = gpd.sjoin(
    roads[["geometry"]],
    roads[["geometry"]],
    how="inner",
    predicate="intersects"
)

# GeoPandas keeps the LEFT index as the dataframe index
road_pairs = road_pairs.reset_index().rename(columns={"index": "index_left"})

# RIGHT index is stored in "index_right"
if "index_right" not in road_pairs.columns:
    raise ValueError(f"sjoin did not create 'index_right'. Columns: {list(road_pairs.columns)}")

# Remove self matches (same feature intersecting itself)
road_pairs = road_pairs[road_pairs["index_left"] != road_pairs["index_right"]].copy()

# Compute intersection geometry pairwise
left_geom = roads.geometry.iloc[road_pairs["index_left"]].values
right_geom = roads.geometry.iloc[road_pairs["index_right"]].values
road_pairs["intersect_geom"] = left_geom.intersection(right_geom)

# Keep only point intersections
road_pairs = road_pairs[road_pairs["intersect_geom"].geom_type == "Point"].copy()

intersection_gdf = gpd.GeoDataFrame(
    geometry=road_pairs["intersect_geom"],
    crs=station_buffers.crs
).drop_duplicates(subset="geometry")


print("Total unique intersections:", len(intersection_gdf))

# ---------------------------------------------------------
# 3) Count intersections within each station buffer
# ---------------------------------------------------------

nodes_join = gpd.sjoin(
    intersection_gdf,
    station_buffers[["stop_id", "stop_name", "geometry"]],
    how="inner",
    predicate="within"
)

node_counts = (
    nodes_join.groupby(["stop_id", "stop_name"])
    .size()
    .reset_index(name="intersection_count_0p5mi")
)

# ---------------------------------------------------------
# 4) Compute buffer area (sq miles)
# ---------------------------------------------------------

SQFT_PER_SQMI = 5280.0 ** 2

if "buffer_area_sqmi" not in station_buffers.columns:
    station_buffers["buffer_area_sqmi"] = (
        station_buffers.geometry.area / SQFT_PER_SQMI
    )

# ---------------------------------------------------------
# 5) Attach counts and compute density
# ---------------------------------------------------------

station_buffers = station_buffers.merge(
    node_counts,
    on=["stop_id", "stop_name"],
    how="left"
)

station_buffers["intersection_count_0p5mi"] = (
    station_buffers["intersection_count_0p5mi"]
    .fillna(0)
    .astype(int)
)

station_buffers["intersection_density_0p5mi"] = (
    station_buffers["intersection_count_0p5mi"]
    / station_buffers["buffer_area_sqmi"]
)

station_buffers["intersection_density_0p5mi"] = (
    station_buffers["intersection_density_0p5mi"]
    .replace([np.inf, -np.inf], 0)
    .fillna(0)
)

display(
    station_buffers[
        ["stop_id", "stop_name",
         "intersection_count_0p5mi",
         "intersection_density_0p5mi"]
    ].head()
)

print("Intersection density summary:")
display(station_buffers["intersection_density_0p5mi"].describe())


Downloading: https://www2.census.gov/geo/tiger/TIGER2020/ROADS/tl_2020_36081_roads.zip
Downloading: https://www2.census.gov/geo/tiger/TIGER2020/ROADS/tl_2020_36047_roads.zip
Downloading: https://www2.census.gov/geo/tiger/TIGER2020/ROADS/tl_2020_36085_roads.zip
Downloading: https://www2.census.gov/geo/tiger/TIGER2020/ROADS/tl_2020_36005_roads.zip
Downloading: https://www2.census.gov/geo/tiger/TIGER2020/ROADS/tl_2020_36061_roads.zip
Total NYC road segments: 22097
Total unique intersections: 51574


,stop_id,stop_name,intersection_count_0p5mi,intersection_density_0p5mi
0,101,Van Cortlandt Park-242 St,104,132.629864
1,103,238 St,142,181.090776
2,104,231 St,140,178.540202
3,106,Marble Hill-225 St,125,159.410894
4,107,215 St,79,100.747685


Intersection density summary:


,intersection_density_0p5mi
count,518.000000
mean,213.000036
std,68.250348
min,36.983327
25%,169.613192
50%,202.770658
75%,241.029272
max,419.569474


In [15]:
# =========================================================
# Urban Variable 4: Street Intersection Density (LION Nodes)
# Fix: avoid MergeError by overwriting existing columns safely
# =========================================================

from google.colab import drive
drive.mount("/content/drive")

import geopandas as gpd
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1) Load LION nodes (local .gdb on Drive)
# ---------------------------------------------------------
lion_gdb_path = "/content/drive/MyDrive/capstone/lion.gdb"  # your path
lion_nodes = gpd.read_file(lion_gdb_path, layer="node")

print("Loaded nodes:", len(lion_nodes))
print("CRS:", lion_nodes.crs)

# Remove virtual intersections (recommended)
if "VIntersect" in lion_nodes.columns:
    lion_nodes = lion_nodes[lion_nodes["VIntersect"] != "VirtualIntersection"].copy()

print("Physical intersections:", len(lion_nodes))

# ---------------------------------------------------------
# 2) Align CRS to station buffers
# ---------------------------------------------------------
lion_nodes = lion_nodes.to_crs(station_buffers.crs)

# ---------------------------------------------------------
# 3) Spatial join: nodes within each station buffer
# ---------------------------------------------------------
nodes_join = gpd.sjoin(
    lion_nodes[["geometry"]],
    station_buffers[["stop_id", "stop_name", "geometry"]],
    how="inner",
    predicate="within",
)

# Count nodes per station
node_counts = (
    nodes_join.groupby("stop_id")
    .size()
    .rename("intersection_count_0p5mi")
)

# ---------------------------------------------------------
# 4) Compute buffer area (sq mi) if needed
# ---------------------------------------------------------
SQFT_PER_SQMI = 5280.0 ** 2

if "buffer_area_sqmi" not in station_buffers.columns:
    station_buffers["buffer_area_sqmi"] = station_buffers.geometry.area / SQFT_PER_SQMI

# ---------------------------------------------------------
# 5) Overwrite (or create) intersection columns WITHOUT merge
# ---------------------------------------------------------
# Drop old columns if they exist to avoid confusion
for col in ["intersection_count_0p5mi", "intersection_density_0p5mi"]:
    if col in station_buffers.columns:
        station_buffers = station_buffers.drop(columns=[col])

# Attach counts by stop_id (keeps all original station columns)
station_buffers = station_buffers.join(node_counts, on="stop_id")

# Fill stations with no intersections counted
station_buffers["intersection_count_0p5mi"] = (
    station_buffers["intersection_count_0p5mi"].fillna(0).astype(int)
)

# Density per sq mi
station_buffers["intersection_density_0p5mi"] = (
    station_buffers["intersection_count_0p5mi"] / station_buffers["buffer_area_sqmi"]
).replace([np.inf, -np.inf], 0).fillna(0)

# ---------------------------------------------------------
# 6) Preview + quick summary
# ---------------------------------------------------------
display(
    station_buffers[
        ["stop_id", "stop_name", "intersection_count_0p5mi", "intersection_density_0p5mi"]
    ].head()
)

display(station_buffers["intersection_density_0p5mi"].describe())


Mounted at /content/drive
Loaded nodes: 139493
CRS: EPSG:2263
Physical intersections: 135669


,stop_id,stop_name,intersection_count_0p5mi,intersection_density_0p5mi
0,101,Van Cortlandt Park-242 St,371,473.131534
1,103,238 St,445,567.502784
2,104,231 St,483,615.963696
3,106,Marble Hill-225 St,437,557.300487
4,107,215 St,269,343.052245


,intersection_density_0p5mi
count,518.000000
mean,515.046136
std,229.917372
min,124.978141
25%,341.776957
50%,469.305673
75%,641.469439
max,1309.719908


In [16]:
# =========================================================
# Urban Variable 3: Land Use Mix Diversity (Entropy) within 0.5 mile
# Source: NYC Planning MapPLUTO (ArcGIS REST Feature Service)
# Output: station_buffers["landuse_entropy_0p5mi"]
#
# End-to-end steps:
#   A) Download MapPLUTO lots intersecting the station buffer extent (ArcGIS REST, paged)
#   B) Repair invalid lot geometries safely
#   C) Compute land-use entropy per station buffer using:
#        sjoin (candidate pairs) + exact intersection area (no accuracy loss)
#
# Prereq:
#   station_buffers GeoDataFrame exists with columns:
#     ["stop_id", "stop_name", "geometry"] in EPSG:2263 (feet)
# =========================================================

import geopandas as gpd
import pandas as pd
import numpy as np
import requests
from shapely.errors import GEOSException

# -----------------------------
# 0) Shapely geometry repair
# -----------------------------
try:
    from shapely import make_valid  # shapely>=2.0
except Exception:
    try:
        from shapely.validation import make_valid
    except Exception:
        make_valid = None

def safe_make_valid(geom):
    """
    Repair invalid geometries robustly:
      1) make_valid() if available
      2) buffer(0) fallback
      3) drop if still invalid/empty
    """
    if geom is None:
        return None
    if getattr(geom, "is_empty", True):
        return None

    if make_valid is not None:
        try:
            g2 = make_valid(geom)
            if g2 is None or g2.is_empty:
                return None
            return g2
        except Exception:
            pass

    try:
        g2 = geom.buffer(0)
        if g2 is None or g2.is_empty:
            return None
        return g2
    except GEOSException:
        return None

# ---------------------------------------------------------
# 1) MapPLUTO Feature Service endpoint
# ---------------------------------------------------------
MAPPLUTO_LAYER_URL = (
    "https://a841-dotweb01.nyc.gov/arcgis/rest/services/GAZETTEER/MapPLUTO/MapServer/0/query"
)

# ---------------------------------------------------------
# 2) Robust ArcGIS fetch with pagination (GeoJSON)
# ---------------------------------------------------------
def fetch_arcgis_geojson(
    url: str,
    *,
    where: str = "1=1",
    out_fields: str = "*",
    geom_envelope: tuple[float, float, float, float] | None = None,
    in_sr: int | None = None,
    out_sr: int | None = None,
    page_size: int = 2000,
    timeout: int = 180
) -> gpd.GeoDataFrame:
    """
    Fetch features from ArcGIS REST as GeoJSON with stable pagination.
    """
    all_features = []
    offset = 0

    while True:
        params = {
            "where": where,
            "outFields": out_fields,
            "f": "geojson",
            "returnGeometry": "true",
            "resultOffset": offset,
            "resultRecordCount": page_size,
            "orderByFields": "OBJECTID",
        }

        if geom_envelope is not None:
            minx, miny, maxx, maxy = geom_envelope
            params.update({
                "geometry": f"{minx},{miny},{maxx},{maxy}",
                "geometryType": "esriGeometryEnvelope",
                "spatialRel": "esriSpatialRelIntersects",
            })

        if in_sr is not None:
            params["inSR"] = in_sr
        if out_sr is not None:
            params["outSR"] = out_sr

        resp = requests.get(url, params=params, timeout=timeout)
        resp.raise_for_status()
        gj = resp.json()

        features = gj.get("features", [])
        all_features.extend(features)

        exceeded = bool(gj.get("exceededTransferLimit", False))

        if (len(features) < page_size and not exceeded) or len(features) == 0:
            break

        offset += page_size

    if not all_features:
        return gpd.GeoDataFrame({"geometry": []}, geometry="geometry")

    return gpd.GeoDataFrame.from_features(all_features)

# ---------------------------------------------------------
# A) Download MapPLUTO lots for buffer extent
# ---------------------------------------------------------
if station_buffers.crs is None:
    raise ValueError("station_buffers must have a CRS set (expected EPSG:2263).")

minx, miny, maxx, maxy = station_buffers.total_bounds

pluto_raw = fetch_arcgis_geojson(
    MAPPLUTO_LAYER_URL,
    out_fields="LandUse",            # keep payload small
    geom_envelope=(minx, miny, maxx, maxy),
    in_sr=2263,                      # bbox is in EPSG:2263 feet
    out_sr=2263,                     # request EPSG:2263 output
    page_size=2000,
    timeout=180
)

print("PLUTO rows downloaded:", len(pluto_raw))

# CRS handling: GeoJSON sometimes omits CRS metadata
if pluto_raw.crs is None:
    pluto_raw = pluto_raw.set_crs(station_buffers.crs, allow_override=True)
pluto_raw = pluto_raw.to_crs(station_buffers.crs)

# Detect land use field
landuse_field = next((c for c in ["LandUse", "LANDUSE", "landuse"] if c in pluto_raw.columns), None)
if landuse_field is None:
    raise ValueError(
        f"Could not find a land-use field. Columns (sample): {list(pluto_raw.columns)[:40]}"
    )

# Keep only needed columns
pluto = pluto_raw[[landuse_field, "geometry"]].copy()

# ---------------------------------------------------------
# B) Repair lot geometries safely
# ---------------------------------------------------------
pluto = pluto.loc[pluto.geometry.notna() & ~pluto.geometry.is_empty].copy()
pluto["geometry"] = pluto["geometry"].apply(safe_make_valid)
pluto = pluto.loc[pluto.geometry.notna() & ~pluto.geometry.is_empty].copy()

print("PLUTO rows after cleaning:", len(pluto))

# ---------------------------------------------------------
# C) FAST entropy computation (accurate)
#    sjoin candidate pairs + exact intersection areas
# ---------------------------------------------------------

# Ensure CRS match
if pluto.crs != station_buffers.crs:
    pluto = pluto.to_crs(station_buffers.crs)

# 1) Candidate lot-buffer pairs (fast via spatial index)
pairs = gpd.sjoin(
    pluto[[landuse_field, "geometry"]],
    station_buffers[["stop_id", "stop_name", "geometry"]],
    how="inner",
    predicate="intersects"
).reset_index().rename(columns={"index": "lot_idx", "index_right": "buf_idx"})

print("Candidate lot-buffer pairs:", len(pairs))

# 2) Attach buffer geometry to each pair (vectorized lookup)
buf_geom = station_buffers.geometry.reset_index(drop=True)
pairs = pairs.join(buf_geom.rename("buf_geom"), on="buf_idx")

# 3) Exact intersection area for each candidate pair
pairs["intersect_geom"] = pairs.geometry.intersection(pairs["buf_geom"])
pairs["piece_area"] = pairs["intersect_geom"].area

pairs = pairs.loc[pairs["piece_area"] > 0].copy()

print("Positive-area intersections:", len(pairs))

# 4) Area by station + land use
area_by_use = (
    pairs.groupby(["stop_id", "stop_name", landuse_field])["piece_area"]
    .sum()
    .reset_index()
)

# 5) Shares
total_area = (
    area_by_use.groupby(["stop_id", "stop_name"])["piece_area"]
    .sum()
    .reset_index()
    .rename(columns={"piece_area": "total_lot_area"})
)

area_by_use = area_by_use.merge(total_area, on=["stop_id", "stop_name"], how="left")
area_by_use["share"] = area_by_use["piece_area"] / area_by_use["total_lot_area"]

# 6) Vectorized entropy: -sum(p * ln(p))
area_by_use["p_log_p"] = area_by_use["share"] * np.log(area_by_use["share"])
entropy_by_station = (
    area_by_use.groupby(["stop_id", "stop_name"])["p_log_p"]
    .sum()
    .mul(-1)
    .reset_index()
    .rename(columns={"p_log_p": "landuse_entropy_0p5mi"})
)

# 7) Attach to stations (retain all)
station_buffers = station_buffers.merge(entropy_by_station, on=["stop_id", "stop_name"], how="left")
station_buffers["landuse_entropy_0p5mi"] = station_buffers["landuse_entropy_0p5mi"].fillna(0.0)

# Preview
display(station_buffers[["stop_id", "stop_name", "landuse_entropy_0p5mi"]].head())
display(
    station_buffers.sort_values("landuse_entropy_0p5mi", ascending=False)[
        ["stop_id", "stop_name", "landuse_entropy_0p5mi"]
    ].head(10)
)


KeyboardInterrupt: 

In [17]:
# =========================================================
# Urban Variable 5: Bike Share Stations Nearby (Citi Bike docks)
# Source: Citi Bike GBFS station_information.json
# Method: Count stations within buffer
# =========================================================

import geopandas as gpd
import pandas as pd
import requests

# Citi Bike GBFS endpoint (station locations)
gbfs_url = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"

r = requests.get(gbfs_url, timeout=60)
r.raise_for_status()
data = r.json()

stations = pd.DataFrame(data["data"]["stations"])

# Build GeoDataFrame in WGS84 then project to EPSG:2263
cb = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations["lon"], stations["lat"]),
    crs="EPSG:4326",
).to_crs(station_buffers.crs)

# Spatial join: Citi Bike stations within subway buffer
cb_join = gpd.sjoin(
    cb[["station_id", "name", "geometry"]],
    station_buffers[["stop_id", "stop_name", "geometry"]],
    how="inner",
    predicate="within",
)

# Count bike stations per subway station buffer
cb_counts = (
    cb_join.groupby(["stop_id", "stop_name"])
    .size()
    .reset_index(name="citibike_count_0p5mi")
)

station_buffers = station_buffers.merge(cb_counts, on=["stop_id", "stop_name"], how="left")
station_buffers["citibike_count_0p5mi"] = station_buffers["citibike_count_0p5mi"].fillna(0).astype(int)

display(station_buffers[["stop_id", "stop_name", "citibike_count_0p5mi"]].head())


,stop_id,stop_name,citibike_count_0p5mi
0,101,Van Cortlandt Park-242 St,9
1,103,238 St,18
2,104,231 St,21
3,106,Marble Hill-225 St,18
4,107,215 St,13


In [18]:
# =========================================================
# Urban Variable 6 (DETERMINISTIC + NYC-BG OFFSET-DETECTED): % Households with No Vehicle (0.5 mile)
# Data source: ACS 2020 5-year Summary File (SEQUENCE-BASED), NY Tracts+Block Groups only
#
# Output: station_buffers["pct_hh_no_vehicle_0p5mi"]
#
# Requires:
# - station_bg_detail: GEOID (12), area_ratio, stop_id, stop_name
# - station_buffers: stop_id, stop_name
# - nyc_countyfps: either ["005","047","061","081","085"] OR ["36005","36047","36061","36081","36085"]
# =========================================================

import os
import io
import re
import zipfile
import hashlib
import requests
import pandas as pd
import numpy as np

# -----------------------------
# Config
# -----------------------------
ACS_ENDYEAR = 2020
STATE_FIPS = "36"
STATE_ABBR = "ny"

TABLE_ID = "B08201"
LINE_TOTAL = 1
LINE_NO_VEH = 2  # top-level "No vehicle available"
OUT_COL = "pct_hh_no_vehicle_0p5mi"

WORKDIR = "./acs_sf_cache"
os.makedirs(WORKDIR, exist_ok=True)

NY_ZIP_URL = (
    "https://www2.census.gov/programs-surveys/acs/summary_file/2020/data/5_year_by_state/"
    "NewYork_Tracts_Block_Groups_Only.zip"
)
LOOKUP_URL = (
    "https://www2.census.gov/programs-surveys/acs/summary_file/2020/documentation/user_tools/"
    "ACS_5yr_Seq_Table_Number_Lookup.txt"
)

NY_ZIP_PATH = os.path.join(WORKDIR, "NewYork_Tracts_Block_Groups_Only_2020.zip")

DEBUG = True
CACHE_VERSION = "v6_no_vehicle_seq_2020_offsetdetect_on_nyc_bg_v2"

def _cache_path():
    key = f"{ACS_ENDYEAR}|{TABLE_ID}|{STATE_FIPS}|{CACHE_VERSION}"
    h = hashlib.md5(key.encode("utf-8")).hexdigest()[:10]
    return os.path.join(WORKDIR, f"acs{ACS_ENDYEAR}_{STATE_ABBR}_bg_{TABLE_ID}_{h}.parquet")

CACHE_PARQUET = _cache_path()

# -----------------------------
# Helpers
# -----------------------------
def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

def normalize_nyc_counties(nyc_countyfps, state_fips="36"):
    out = set()
    for c in nyc_countyfps:
        s = re.sub(r"\D", "", str(c).strip())
        if len(s) == 3:
            out.add(state_fips + s)
        elif len(s) == 5:
            out.add(s)
        else:
            out.add(state_fips + s[-3:].zfill(3))
    return out

NYC_COUNTIES = normalize_nyc_counties(nyc_countyfps, STATE_FIPS)
_log(f"NYC_COUNTIES (normalized) = {sorted(NYC_COUNTIES)}")

def clean_geoid12(x) -> str:
    s = re.sub(r"\D", "", str(x) if x is not None else "")
    if len(s) >= 12:
        s = s[-12:]
    else:
        s = s.zfill(12)
    return s

def parse_area_ratio(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip()
    s = s.str.replace("%", "", regex=False)
    s = s.str.replace(",", ".", regex=False)
    x = pd.to_numeric(s, errors="coerce").fillna(0)
    try:
        if float(x.quantile(0.99)) > 1.5:
            _log("⚠️ area_ratio appears to be 0-100. Converting to fraction by /100.")
            x = x / 100.0
    except Exception:
        pass
    return x.clip(lower=0, upper=1)

def download_if_missing(url: str, path: str, chunk=1024 * 1024):
    if os.path.exists(path) and os.path.getsize(path) > 0:
        _log(f"✅ Using cached file: {path}")
        return
    _log(f"⬇️ Downloading: {url}")
    with requests.get(url, stream=True, timeout=300) as r:
        r.raise_for_status()
        with open(path, "wb") as f:
            for part in r.iter_content(chunk_size=chunk):
                if part:
                    f.write(part)
    _log(f"✅ Saved: {path} ({os.path.getsize(path)/1e6:.1f} MB)")

def sniff_delimiter(sample: bytes) -> str:
    text = sample.decode("latin-1", errors="replace")
    candidates = [",", "\t", "|"]
    best = ","
    best_cols = 0
    first = text.splitlines()[0] if text.splitlines() else ""
    for d in candidates:
        cols = len(first.split(d)) if first else 0
        if cols > best_cols:
            best_cols = cols
            best = d
    return best

def read_lookup_from_web(url: str) -> pd.DataFrame:
    _log(f"🌐 Fetching lookup from web: {url}")
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    raw = r.content
    sep = sniff_delimiter(raw[:2000])
    _log(f"🔎 Lookup delimiter guessed as: {repr(sep)}")
    df = pd.read_csv(io.BytesIO(raw), sep=sep, header=None, dtype=str, encoding="latin-1", engine="python")
    df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
    _log(f"✅ Lookup loaded: {len(df):,} rows, {df.shape[1]} cols")
    return df

def get_seq_startpos_cells(lookup_df: pd.DataFrame, table_id: str):
    tt = lookup_df[lookup_df[1].astype(str).str.upper() == table_id.upper()].copy()
    if tt.empty:
        raise RuntimeError(f"Table {table_id} not found in lookup.")

    header = tt[tt.apply(lambda r: r.astype(str).str.contains("CELLS", case=False, na=False).any(), axis=1)]
    if header.empty:
        raise RuntimeError(f"Found rows for {table_id} but none contained 'CELLS'.")

    hr = header.iloc[0].tolist()
    seq = str(hr[2]).strip().zfill(4)

    cells_idx, cells_val = None, None
    for i, v in enumerate(hr):
        if isinstance(v, str) and "CELLS" in v.upper():
            cells_idx, cells_val = i, v
            break

    m = re.search(r"(\d+)\s*CELLS", str(cells_val), flags=re.I)
    if not m:
        raise RuntimeError(f"Could not parse cells count from '{cells_val}' for {table_id}.")
    cells = int(m.group(1))

    startpos = None
    for j in range(cells_idx - 1, -1, -1):
        v = hr[j]
        if isinstance(v, str) and v.strip().isdigit():
            startpos = int(v.strip())
            break
    if startpos is None:
        raise RuntimeError(f"Could not parse startpos for {table_id}.")

    _log(f"✅ Lookup parse: table={table_id} seq={seq} startpos={startpos} cells={cells}")
    return seq, startpos, cells

def get_table_line_map(lookup_df: pd.DataFrame, table_id: str) -> pd.DataFrame:
    t = lookup_df[lookup_df[1].astype(str).str.upper() == table_id.upper()].copy()
    t["line_num"] = pd.to_numeric(t[3], errors="coerce")

    text_cols = [c for c in [6, 7, 8] if c in t.columns]
    t["label"] = (
        t[text_cols]
        .fillna("")
        .astype(str)
        .agg(" ".join, axis=1)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

    out = t.loc[t["line_num"].notna(), ["line_num", "label"]].copy()
    out["line_num"] = out["line_num"].astype(int)
    out = out[out["label"].ne("")].drop_duplicates(subset=["line_num"]).sort_values("line_num")
    return out

def find_file_in_zip(z: zipfile.ZipFile, predicate, label="file") -> str:
    for name in z.namelist():
        if predicate(name):
            return name
    raise RuntimeError(f"No matching {label} found in zip.")

def read_csv_from_zip(z: zipfile.ZipFile, member: str, usecols=None, header=None, nrows=None) -> pd.DataFrame:
    raw = z.read(member)
    sep = sniff_delimiter(raw[:2000])
    return pd.read_csv(
        io.BytesIO(raw),
        sep=sep,
        header=header,
        dtype=str,
        encoding="latin-1",
        engine="python",
        usecols=usecols,
        nrows=nrows,
    )

def read_geo_from_zip_auto(z: zipfile.ZipFile, member: str) -> pd.DataFrame:
    raw = z.read(member)
    sep = sniff_delimiter(raw[:2000])

    df0 = pd.read_csv(io.BytesIO(raw), sep=sep, header=0, dtype=str, encoding="latin-1", engine="python")
    cols = [str(c).strip().upper() for c in df0.columns]
    useful = any(c in ("GEO_ID", "GEOID", "LOGRECNO") for c in cols) or any("LOGREC" in c for c in cols)
    if useful:
        df0.columns = [str(c).strip() for c in df0.columns]
        return df0

    df = pd.read_csv(io.BytesIO(raw), sep=sep, header=None, dtype=str, encoding="latin-1", engine="python")
    df.columns = list(range(df.shape[1]))
    return df

def build_bg_geoid_from_geofile(geo_df: pd.DataFrame) -> pd.DataFrame:
    g = geo_df.copy()

    if not isinstance(g.columns[0], (int, np.integer)):
        cols_upper = {str(c).strip().upper(): c for c in g.columns}

        def pick(*names):
            for n in names:
                if n in cols_upper:
                    return cols_upper[n]
            return None

        logrec = pick("LOGRECNO") or next((cols_upper[k] for k in cols_upper if "LOGREC" in k), None)
        geoid_src = pick("GEO_ID", "GEOID")
        if logrec is None or geoid_src is None:
            raise RuntimeError("Geo: could not find LOGRECNO and GEO_ID/GEOID.")

        s = g[geoid_src].astype(str).str.strip()
        geoid12 = s.str.extract(r"US(36\d{10})", expand=False)

        out = pd.DataFrame({"LOGRECNO": g[logrec].astype(str).str.strip(), "GEOID": geoid12}).dropna(subset=["GEOID"])
        out = out[out["GEOID"].str.fullmatch(r"36\d{10}", na=False)].drop_duplicates()
        if out.empty:
            raise RuntimeError("Geo: headered GEO_ID present, but no US36########## extracted.")
        return out

    g.columns = list(range(g.shape[1]))

    def logrec_score(col):
        s = g[col].astype(str).str.strip()
        return (s.str.fullmatch(r"\d+").mean(), s.nunique())

    scores = {c: logrec_score(c) for c in g.columns}
    logrec_col = sorted(scores, key=lambda c: (scores[c][0], scores[c][1]), reverse=True)[0]

    best_col, best_hits = None, 0
    for c in g.columns:
        s = g[c].astype(str).str.strip()
        hits = s.str.contains(r"US36\d{10}", regex=True, na=False).sum()
        if hits > best_hits:
            best_hits, best_col = hits, c
    if best_col is None or best_hits < 100:
        raise RuntimeError("Geo headerless: couldn't find US36########## column.")

    geoid12 = g[best_col].astype(str).str.strip().str.extract(r"US(36\d{10})", expand=False)
    out = pd.DataFrame({"LOGRECNO": g[logrec_col].astype(str).str.strip(), "GEOID": geoid12}).dropna(subset=["GEOID"])
    out = out[out["GEOID"].str.fullmatch(r"36\d{10}", na=False)].drop_duplicates()
    if out.empty:
        raise RuntimeError("Geo headerless: extracted no NY BG GEOIDs.")
    return out

def detect_offset_on_nyc_bg(est_sample_with_logrec: pd.DataFrame,
                           geo_key: pd.DataFrame,
                           base_total_col: int,
                           base_noveh_col: int,
                           window: int = 120):
    """
    Detect offset using only NYC BG rows.
    """
    s = est_sample_with_logrec.copy()
    s["LOGRECNO"] = s["LOGRECNO"].astype(str).str.strip()

    m = geo_key.merge(s, on="LOGRECNO", how="inner")
    m["county_prefix"] = m["GEOID"].astype(str).str[:5]
    m = m[m["county_prefix"].isin(NYC_COUNTIES)].copy()

    _log(f"🧪 Offset detect subset: merged NYC rows={len(m):,} (from sample)")
    if len(m) < 500:
        _log("⚠️ Very few NYC rows in the estimate sample. Increase sample_n.")

    cols = [c for c in m.columns if isinstance(c, (int, np.integer))]
    if not cols:
        raise RuntimeError("Offset detect: no numeric estimate columns found in merged sample.")

    X = m[cols].apply(pd.to_numeric, errors="coerce").fillna(0)

    candidates = []
    for off in range(-window, window + 1):
        tc = base_total_col + off
        nc = base_noveh_col + off
        if tc not in X.columns or nc not in X.columns:
            continue

        total = X[tc].to_numpy()
        noveh = X[nc].to_numpy()

        total_pos = (total > 0).mean()
        if total_pos < 0.80:
            continue

        bad = ((total > 0) & (noveh > total)).mean()

        # ✅ FIX: force float out array to avoid int casting error
        share = np.divide(
            noveh.astype(float),
            total.astype(float),
            out=np.zeros_like(total, dtype=float),
            where=total > 0
        )

        near_one = (share > 0.95).mean()
        near_zero = (share < 0.001).mean()

        score = (3.0 * total_pos) - (20.0 * bad) - (2.0 * near_one) - (0.25 * near_zero)
        candidates.append((score, bad, total_pos, near_one, near_zero, tc, nc, off))

    if not candidates:
        raise RuntimeError(
            "Offset detect (NYC-only): no viable candidates met total_pos>=0.80. "
            "Increase sample_n/window or inspect estimate columns."
        )

    candidates.sort(reverse=True, key=lambda x: x[0])
    top = candidates[:10]

    _log("✅ Offset detect (NYC-only): top candidates (score, bad, total_pos, near_one, near_zero, total_col, noveh_col, offset):")
    for i, row in enumerate(top, 1):
        score, bad, total_pos, near_one, near_zero, tc, nc, off = row
        _log(f"  {i}. score={score:.4f}, bad={bad:.4%}, total_pos={total_pos:.2%}, "
             f"near_one={near_one:.2%}, near_zero={near_zero:.2%}, Tcol={tc}, NVcol={nc}, off={off:+d}")

    best = top[0]
    _, _, _, _, _, tc, nc, off = best
    _log(f"🏁 Selected columns (NYC-only): total_col={tc} (offset {off:+d}), no_vehicle_col={nc}")
    return tc, nc, off

# =========================================================
# 0) Load from cache or build BG extract
# =========================================================
download_if_missing(NY_ZIP_URL, NY_ZIP_PATH)

# If you want a hard reset for THIS version:
# if os.path.exists(CACHE_PARQUET): os.remove(CACHE_PARQUET)

if os.path.exists(CACHE_PARQUET):
    _log(f"✅ Loading cached extract: {CACHE_PARQUET}")
    bg = pd.read_parquet(CACHE_PARQUET)

else:
    lookup = read_lookup_from_web(LOOKUP_URL)
    seq4, startpos, cells = get_seq_startpos_cells(lookup, TABLE_ID)

    line_map = get_table_line_map(lookup, TABLE_ID)
    _log("🧾 B08201 line labels (first 60):")
    _log(line_map.head(60).to_string(index=False))

    _log(f"✅ Using fixed lines: total line={LINE_TOTAL}, no-vehicle line={LINE_NO_VEH}")

    ID_COLS_ASSUMED = 6
    cell_total = startpos + (LINE_TOTAL - 1)
    cell_noveh = startpos + (LINE_NO_VEH - 1)

    base_total_col = ID_COLS_ASSUMED + (cell_total - 1)
    base_noveh_col = ID_COLS_ASSUMED + (cell_noveh - 1)

    _log(f"📌 Base columns (pre-offset): line{LINE_TOTAL}->cell{cell_total} base_col{base_total_col}, "
         f"line{LINE_NO_VEH}->cell{cell_noveh} base_col{base_noveh_col}")

    with zipfile.ZipFile(NY_ZIP_PATH, "r") as z:
        geo_member = find_file_in_zip(
            z,
            lambda n: re.search(r"(^|/)(g).*" + STATE_ABBR + r".*\.(csv|txt)$", n, flags=re.I) is not None,
            label="geography file",
        )
        _log(f"🗺️ Geography member: {geo_member}")

        est_member = find_file_in_zip(
            z,
            lambda n: (
                re.search(r"(^|/)(e).*" + STATE_ABBR + r".*\.(csv|txt)$", n, flags=re.I) is not None
                and seq4 in os.path.basename(n)
            ),
            label="estimate sequence file",
        )
        _log(f"📄 Estimate member: {est_member}")

        _log("📥 Reading geography file (can take a bit)...")
        geo_df = read_geo_from_zip_auto(z, geo_member)
        geo_key = build_bg_geoid_from_geofile(geo_df)
        _log(f"✅ Geography BG rows: {len(geo_key):,}")

        sample_n = 60000
        LOGREC_COL = 5

        # read nearly all cols (as you did) + LOGRECNO for robust detection
        usecols_sample = [LOGREC_COL] + list(range(0, 202))
        _log(f"🧪 Reading estimate sample (n={sample_n:,}) with cols [0..201] plus LOGRECNO for NYC-only offset detection...")
        est_sample = read_csv_from_zip(z, est_member, usecols=usecols_sample, header=None, nrows=sample_n)
        est_sample = est_sample.rename(columns={LOGREC_COL: "LOGRECNO"})

        total_col, noveh_col, offset = detect_offset_on_nyc_bg(
            est_sample_with_logrec=est_sample,
            geo_key=geo_key,
            base_total_col=base_total_col,
            base_noveh_col=base_noveh_col,
            window=120,
        )

        usecols_final = sorted(set([LOGREC_COL, total_col, noveh_col]))
        _log(f"📥 Reading FINAL estimate columns: {usecols_final}")
        est_df = read_csv_from_zip(z, est_member, usecols=usecols_final, header=None)

        est_df = est_df.rename(columns={
            LOGREC_COL: "LOGRECNO",
            total_col: "hh_total_raw",
            noveh_col: "hh_no_vehicle_raw",
        })
        est_df["LOGRECNO"] = est_df["LOGRECNO"].astype(str).str.strip()

        _log("🔬 Raw estimate sample (LOGRECNO, total, no_vehicle):")
        _log(est_df[["LOGRECNO", "hh_total_raw", "hh_no_vehicle_raw"]].head(10).to_string(index=False))

    merged = geo_key.merge(est_df, on="LOGRECNO", how="inner")
    _log(f"✅ Joined rows: {len(merged):,}")

    merged["county_prefix"] = merged["GEOID"].astype(str).str[:5]
    merged = merged[merged["county_prefix"].isin(NYC_COUNTIES)].copy()
    _log(f"✅ NYC BG rows retained: {len(merged):,}")

    merged["hh_total"] = pd.to_numeric(merged["hh_total_raw"], errors="coerce").fillna(0)
    merged["hh_no_vehicle"] = pd.to_numeric(merged["hh_no_vehicle_raw"], errors="coerce").fillna(0)

    _log(f"QA: BG total>0 share = {(merged['hh_total'] > 0).mean():.3%}")
    bad_share = ((merged["hh_total"] > 0) & (merged["hh_no_vehicle"] > merged["hh_total"])).mean()
    _log(f"QA: BG no_vehicle > total share = {bad_share:.3%}")

    if (merged["hh_total"] > 0).mean() < 0.80:
        raise RuntimeError("Totals not present for most NYC BGs — still wrong columns. Check top candidates output.")

    bg = merged[["GEOID", "hh_total", "hh_no_vehicle"]].copy()
    bg["GEOID"] = bg["GEOID"].map(clean_geoid12)

    bg.to_parquet(CACHE_PARQUET, index=False)
    _log(f"💾 Cached extracted table to: {CACHE_PARQUET}")

# =========================================================
# Area-weight into station buffers
# =========================================================
detail = station_bg_detail.copy()
detail["GEOID"] = detail["GEOID"].map(clean_geoid12)
detail["area_ratio"] = parse_area_ratio(detail["area_ratio"])

bg = bg.copy()
bg["GEOID"] = bg["GEOID"].map(clean_geoid12)

detail = detail.merge(bg, on="GEOID", how="left")
_log(f"✅ Merge match rate (rows with hh_total): {detail['hh_total'].notna().mean():.2%}")

detail[["hh_total", "hh_no_vehicle"]] = detail[["hh_total", "hh_no_vehicle"]].fillna(0)

detail["hh_total_within"] = detail["hh_total"] * detail["area_ratio"]
detail["hh_no_vehicle_within"] = detail["hh_no_vehicle"] * detail["area_ratio"]

agg = detail.groupby(["stop_id", "stop_name"], as_index=False)[["hh_total_within", "hh_no_vehicle_within"]].sum()

agg[OUT_COL] = np.where(
    agg["hh_total_within"] > 0,
    100.0 * agg["hh_no_vehicle_within"] / agg["hh_total_within"],
    np.nan,
)

_log(f"🧪 Station-level {OUT_COL} summary:")
print(agg[OUT_COL].describe(percentiles=[.01, .05, .1, .25, .5, .75, .9, .95, .99]))

if OUT_COL in station_buffers.columns:
    station_buffers = station_buffers.drop(columns=[OUT_COL])

station_buffers = station_buffers.merge(
    agg[["stop_id", "stop_name", OUT_COL]],
    on=["stop_id", "stop_name"],
    how="left",
)

_log("✅ Variable 6 complete.")
display(station_buffers[["stop_id", "stop_name", OUT_COL]].head())
print("✅ Variable 6 output column:", OUT_COL)


NYC_COUNTIES (normalized) = ['36005', '36047', '36061', '36081', '36085']
⬇️ Downloading: https://www2.census.gov/programs-surveys/acs/summary_file/2020/data/5_year_by_state/NewYork_Tracts_Block_Groups_Only.zip
✅ Saved: ./acs_sf_cache/NewYork_Tracts_Block_Groups_Only_2020.zip (278.7 MB)
🌐 Fetching lookup from web: https://www2.census.gov/programs-surveys/acs/summary_file/2020/documentation/user_tools/ACS_5yr_Seq_Table_Number_Lookup.txt
🔎 Lookup delimiter guessed as: ','
✅ Lookup loaded: 30,327 rows, 9 cols
✅ Lookup parse: table=B08201 seq=0027 startpos=75 cells=30
🧾 B08201 line labels (first 60):
 line_num                        label
        1                       Total:
        2         No vehicle available
        3          1 vehicle available
        4         2 vehicles available
        5         3 vehicles available
        6 4 or more vehicles available
        7          1-person household:
        8         No vehicle available
        9          1 vehicle available
      

,stop_id,stop_name,pct_hh_no_vehicle_0p5mi
0,101,Van Cortlandt Park-242 St,34.697480
1,103,238 St,30.753513
2,104,231 St,25.197863
3,106,Marble Hill-225 St,20.724292
4,107,215 St,14.853057


✅ Variable 6 output column: pct_hh_no_vehicle_0p5mi


In [19]:
# =========================================================
# Urban Variable 6 (DETERMINISTIC + NYC-BG OFFSET-DETECTED): % Households with No Vehicle (0.5 mile)
# Data source: ACS 2020 5-year Summary File (SEQUENCE-BASED), NY Tracts+Block Groups only
#
# Output: station_buffers["pct_hh_no_vehicle_0p5mi"]
#
# Requires (inputs already in your notebook):
# - station_bg_detail: GEOID (12), area_ratio, stop_id, stop_name
# - station_buffers: stop_id, stop_name
# - nyc_countyfps: either ["005","047","061","081","085"] OR ["36005","36047","36061","36081","36085"]
# =========================================================

import os
import io
import re
import zipfile
import hashlib
import requests
import pandas as pd
import numpy as np

# -----------------------------
# Config
# -----------------------------
ACS_ENDYEAR = 2020
STATE_FIPS = "36"
STATE_ABBR = "ny"

TABLE_ID = "B08201"
LINE_TOTAL = 1
LINE_NO_VEH = 2  # top-level "No vehicle available"
OUT_COL = "pct_hh_no_vehicle_0p5mi"

WORKDIR = "./acs_sf_cache"
os.makedirs(WORKDIR, exist_ok=True)

NY_ZIP_URL = (
    "https://www2.census.gov/programs-surveys/acs/summary_file/2020/data/5_year_by_state/"
    "NewYork_Tracts_Block_Groups_Only.zip"
)
LOOKUP_URL = (
    "https://www2.census.gov/programs-surveys/acs/summary_file/2020/documentation/user_tools/"
    "ACS_5yr_Seq_Table_Number_Lookup.txt"
)

NY_ZIP_PATH = os.path.join(WORKDIR, "NewYork_Tracts_Block_Groups_Only_2020.zip")

DEBUG = True
CACHE_VERSION = "v6_no_vehicle_seq_2020_offsetdetect_on_nyc_bg_clean_v1"

def _cache_path():
    key = f"{ACS_ENDYEAR}|{TABLE_ID}|{STATE_FIPS}|{CACHE_VERSION}"
    h = hashlib.md5(key.encode("utf-8")).hexdigest()[:10]
    return os.path.join(WORKDIR, f"acs{ACS_ENDYEAR}_{STATE_ABBR}_bg_{TABLE_ID}_{h}.parquet")

CACHE_PARQUET = _cache_path()

# -----------------------------
# Helpers
# -----------------------------
def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

def normalize_nyc_counties(nyc_countyfps, state_fips="36"):
    out = set()
    for c in nyc_countyfps:
        s = re.sub(r"\D", "", str(c).strip())
        if len(s) == 3:
            out.add(state_fips + s)
        elif len(s) == 5:
            out.add(s)
        else:
            out.add(state_fips + s[-3:].zfill(3))
    return out

NYC_COUNTIES = normalize_nyc_counties(nyc_countyfps, STATE_FIPS)
_log(f"NYC_COUNTIES (normalized) = {sorted(NYC_COUNTIES)}")

def clean_geoid12(x) -> str:
    s = re.sub(r"\D", "", str(x) if x is not None else "")
    if len(s) >= 12:
        s = s[-12:]
    else:
        s = s.zfill(12)
    return s

def parse_area_ratio(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip()
    s = s.str.replace("%", "", regex=False)
    s = s.str.replace(",", ".", regex=False)
    x = pd.to_numeric(s, errors="coerce").fillna(0)
    # if looks like 0..100, convert to 0..1
    try:
        if float(x.quantile(0.99)) > 1.5:
            _log("⚠️ area_ratio appears to be 0-100. Converting to fraction by /100.")
            x = x / 100.0
    except Exception:
        pass
    return x.clip(lower=0, upper=1)

def download_if_missing(url: str, path: str, chunk=1024 * 1024):
    if os.path.exists(path) and os.path.getsize(path) > 0:
        _log(f"✅ Using cached file: {path}")
        return
    _log(f"⬇️ Downloading: {url}")
    with requests.get(url, stream=True, timeout=300) as r:
        r.raise_for_status()
        with open(path, "wb") as f:
            for part in r.iter_content(chunk_size=chunk):
                if part:
                    f.write(part)
    _log(f"✅ Saved: {path} ({os.path.getsize(path)/1e6:.1f} MB)")

def sniff_delimiter(sample: bytes) -> str:
    text = sample.decode("latin-1", errors="replace")
    candidates = [",", "\t", "|"]
    best = ","
    best_cols = 0
    first = text.splitlines()[0] if text.splitlines() else ""
    for d in candidates:
        cols = len(first.split(d)) if first else 0
        if cols > best_cols:
            best_cols = cols
            best = d
    return best

def read_lookup_from_web(url: str) -> pd.DataFrame:
    _log(f"🌐 Fetching lookup from web: {url}")
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    raw = r.content
    sep = sniff_delimiter(raw[:2000])
    _log(f"🔎 Lookup delimiter guessed as: {repr(sep)}")
    df = pd.read_csv(io.BytesIO(raw), sep=sep, header=None, dtype=str, encoding="latin-1", engine="python")
    df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
    _log(f"✅ Lookup loaded: {len(df):,} rows, {df.shape[1]} cols")
    return df

def get_seq_startpos_cells(lookup_df: pd.DataFrame, table_id: str):
    tt = lookup_df[lookup_df[1].astype(str).str.upper() == table_id.upper()].copy()
    if tt.empty:
        raise RuntimeError(f"Table {table_id} not found in lookup.")

    header = tt[tt.apply(lambda r: r.astype(str).str.contains("CELLS", case=False, na=False).any(), axis=1)]
    if header.empty:
        raise RuntimeError(f"Found rows for {table_id} but none contained 'CELLS'.")

    hr = header.iloc[0].tolist()
    seq = str(hr[2]).strip().zfill(4)

    cells_idx, cells_val = None, None
    for i, v in enumerate(hr):
        if isinstance(v, str) and "CELLS" in v.upper():
            cells_idx, cells_val = i, v
            break

    m = re.search(r"(\d+)\s*CELLS", str(cells_val), flags=re.I)
    if not m:
        raise RuntimeError(f"Could not parse cells count from '{cells_val}' for {table_id}.")
    cells = int(m.group(1))

    startpos = None
    for j in range(cells_idx - 1, -1, -1):
        v = hr[j]
        if isinstance(v, str) and v.strip().isdigit():
            startpos = int(v.strip())
            break
    if startpos is None:
        raise RuntimeError(f"Could not parse startpos for {table_id}.")

    _log(f"✅ Lookup parse: table={table_id} seq={seq} startpos={startpos} cells={cells}")
    return seq, startpos, cells

def get_table_line_map(lookup_df: pd.DataFrame, table_id: str) -> pd.DataFrame:
    t = lookup_df[lookup_df[1].astype(str).str.upper() == table_id.upper()].copy()
    t["line_num"] = pd.to_numeric(t[3], errors="coerce")

    text_cols = [c for c in [6, 7, 8] if c in t.columns]
    t["label"] = (
        t[text_cols]
        .fillna("")
        .astype(str)
        .agg(" ".join, axis=1)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

    out = t.loc[t["line_num"].notna(), ["line_num", "label"]].copy()
    out["line_num"] = out["line_num"].astype(int)
    out = out[out["label"].ne("")].drop_duplicates(subset=["line_num"]).sort_values("line_num")
    return out

def find_file_in_zip(z: zipfile.ZipFile, predicate, label="file") -> str:
    for name in z.namelist():
        if predicate(name):
            return name
    raise RuntimeError(f"No matching {label} found in zip.")

def read_csv_from_zip(z: zipfile.ZipFile, member: str, usecols=None, header=None, nrows=None) -> pd.DataFrame:
    raw = z.read(member)
    sep = sniff_delimiter(raw[:2000])
    return pd.read_csv(
        io.BytesIO(raw),
        sep=sep,
        header=header,
        dtype=str,
        encoding="latin-1",
        engine="python",
        usecols=usecols,
        nrows=nrows,
    )

def read_geo_from_zip_auto(z: zipfile.ZipFile, member: str) -> pd.DataFrame:
    raw = z.read(member)
    sep = sniff_delimiter(raw[:2000])

    df0 = pd.read_csv(io.BytesIO(raw), sep=sep, header=0, dtype=str, encoding="latin-1", engine="python")
    cols = [str(c).strip().upper() for c in df0.columns]
    useful = any(c in ("GEO_ID", "GEOID", "LOGRECNO") for c in cols) or any("LOGREC" in c for c in cols)
    if useful:
        df0.columns = [str(c).strip() for c in df0.columns]
        return df0

    df = pd.read_csv(io.BytesIO(raw), sep=sep, header=None, dtype=str, encoding="latin-1", engine="python")
    df.columns = list(range(df.shape[1]))
    return df

def build_bg_geoid_from_geofile(geo_df: pd.DataFrame) -> pd.DataFrame:
    g = geo_df.copy()

    # headered
    if not isinstance(g.columns[0], (int, np.integer)):
        cols_upper = {str(c).strip().upper(): c for c in g.columns}

        def pick(*names):
            for n in names:
                if n in cols_upper:
                    return cols_upper[n]
            return None

        logrec = pick("LOGRECNO") or next((cols_upper[k] for k in cols_upper if "LOGREC" in k), None)
        geoid_src = pick("GEO_ID", "GEOID")
        if logrec is None or geoid_src is None:
            raise RuntimeError("Geo: could not find LOGRECNO and GEO_ID/GEOID.")

        s = g[geoid_src].astype(str).str.strip()
        geoid12 = s.str.extract(r"US(36\d{10})", expand=False)
        out = pd.DataFrame({"LOGRECNO": g[logrec].astype(str).str.strip(), "GEOID": geoid12}).dropna(subset=["GEOID"])
        out = out[out["GEOID"].str.fullmatch(r"36\d{10}", na=False)].drop_duplicates()
        if out.empty:
            raise RuntimeError("Geo: headered GEO_ID present, but no US36########## extracted.")
        return out

    # headerless
    g.columns = list(range(g.shape[1]))

    def logrec_score(col):
        s = g[col].astype(str).str.strip()
        return (s.str.fullmatch(r"\d+").mean(), s.nunique())

    scores = {c: logrec_score(c) for c in g.columns}
    logrec_col = sorted(scores, key=lambda c: (scores[c][0], scores[c][1]), reverse=True)[0]

    best_col, best_hits = None, 0
    for c in g.columns:
        s = g[c].astype(str).str.strip()
        hits = s.str.contains(r"US36\d{10}", regex=True, na=False).sum()
        if hits > best_hits:
            best_hits, best_col = hits, c
    if best_col is None or best_hits < 100:
        raise RuntimeError("Geo headerless: couldn't find US36########## column.")

    geoid12 = g[best_col].astype(str).str.strip().str.extract(r"US(36\d{10})", expand=False)
    out = pd.DataFrame({"LOGRECNO": g[logrec_col].astype(str).str.strip(), "GEOID": geoid12}).dropna(subset=["GEOID"])
    out = out[out["GEOID"].str.fullmatch(r"36\d{10}", na=False)].drop_duplicates()
    if out.empty:
        raise RuntimeError("Geo headerless: extracted no NY BG GEOIDs.")
    return out

def detect_offset_on_nyc_bg(est_sample_with_logrec: pd.DataFrame,
                           geo_key: pd.DataFrame,
                           base_total_col: int,
                           base_noveh_col: int,
                           window: int = 120):
    """
    Detect offset using only NYC BG rows.
    """
    s = est_sample_with_logrec.copy()
    s["LOGRECNO"] = s["LOGRECNO"].astype(str).str.strip()

    m = geo_key.merge(s, on="LOGRECNO", how="inner")
    m["county_prefix"] = m["GEOID"].astype(str).str[:5]
    m = m[m["county_prefix"].isin(NYC_COUNTIES)].copy()

    _log(f"🧪 Offset detect subset: merged NYC rows={len(m):,} (from sample)")
    if len(m) < 500:
        _log("⚠️ Very few NYC rows in the estimate sample. Increase sample_n.")

    cols = [c for c in m.columns if isinstance(c, (int, np.integer))]
    if not cols:
        raise RuntimeError("Offset detect: no numeric estimate columns found in merged sample.")

    X = m[cols].apply(pd.to_numeric, errors="coerce").fillna(0)

    candidates = []
    for off in range(-window, window + 1):
        tc = base_total_col + off
        nc = base_noveh_col + off
        if tc not in X.columns or nc not in X.columns:
            continue

        total = X[tc].to_numpy()
        noveh = X[nc].to_numpy()

        total_pos = (total > 0).mean()
        if total_pos < 0.80:
            continue

        bad = ((total > 0) & (noveh > total)).mean()

        share = np.divide(
            noveh.astype(float),
            total.astype(float),
            out=np.zeros_like(total, dtype=float),
            where=total > 0
        )

        near_one = (share > 0.95).mean()
        near_zero = (share < 0.001).mean()

        score = (3.0 * total_pos) - (20.0 * bad) - (2.0 * near_one) - (0.25 * near_zero)
        candidates.append((score, bad, total_pos, near_one, near_zero, tc, nc, off))

    if not candidates:
        raise RuntimeError(
            "Offset detect (NYC-only): no viable candidates met total_pos>=0.80. "
            "Increase sample_n/window or inspect estimate columns."
        )

    candidates.sort(reverse=True, key=lambda x: x[0])
    top = candidates[:10]

    _log("✅ Offset detect (NYC-only): top candidates (score, bad, total_pos, near_one, near_zero, total_col, noveh_col, offset):")
    for i, row in enumerate(top, 1):
        score, bad, total_pos, near_one, near_zero, tc, nc, off = row
        _log(f"  {i}. score={score:.4f}, bad={bad:.4%}, total_pos={total_pos:.2%}, "
             f"near_one={near_one:.2%}, near_zero={near_zero:.2%}, Tcol={tc}, NVcol={nc}, off={off:+d}")

    best = top[0]
    _, _, _, _, _, tc, nc, off = best
    _log(f"🏁 Selected columns (NYC-only): total_col={tc} (offset {off:+d}), no_vehicle_col={nc}")
    return tc, nc, off

# =========================================================
# 0) Load from cache or build BG extract
# =========================================================
download_if_missing(NY_ZIP_URL, NY_ZIP_PATH)

# Hard reset for THIS version if you want:
# if os.path.exists(CACHE_PARQUET):
#     os.remove(CACHE_PARQUET)

if os.path.exists(CACHE_PARQUET):
    _log(f"✅ Loading cached extract: {CACHE_PARQUET}")
    bg = pd.read_parquet(CACHE_PARQUET)

else:
    lookup = read_lookup_from_web(LOOKUP_URL)
    seq4, startpos, cells = get_seq_startpos_cells(lookup, TABLE_ID)

    line_map = get_table_line_map(lookup, TABLE_ID)
    _log("🧾 B08201 line labels (first 60):")
    _log(line_map.head(60).to_string(index=False))

    _log(f"✅ Using fixed lines: total line={LINE_TOTAL}, no-vehicle line={LINE_NO_VEH}")

    ID_COLS_ASSUMED = 6
    cell_total = startpos + (LINE_TOTAL - 1)
    cell_noveh = startpos + (LINE_NO_VEH - 1)

    base_total_col = ID_COLS_ASSUMED + (cell_total - 1)
    base_noveh_col = ID_COLS_ASSUMED + (cell_noveh - 1)

    _log(f"📌 Base columns (pre-offset): line{LINE_TOTAL}->cell{cell_total} base_col{base_total_col}, "
         f"line{LINE_NO_VEH}->cell{cell_noveh} base_col{base_noveh_col}")

    with zipfile.ZipFile(NY_ZIP_PATH, "r") as z:
        geo_member = find_file_in_zip(
            z,
            lambda n: re.search(r"(^|/)(g).*" + STATE_ABBR + r".*\.(csv|txt)$", n, flags=re.I) is not None,
            label="geography file",
        )
        _log(f"🗺️ Geography member: {geo_member}")

        est_member = find_file_in_zip(
            z,
            lambda n: (
                re.search(r"(^|/)(e).*" + STATE_ABBR + r".*\.(csv|txt)$", n, flags=re.I) is not None
                and seq4 in os.path.basename(n)
            ),
            label="estimate sequence file",
        )
        _log(f"📄 Estimate member: {est_member}")

        _log("📥 Reading geography file (can take a bit)...")
        geo_df = read_geo_from_zip_auto(z, geo_member)
        geo_key = build_bg_geoid_from_geofile(geo_df)
        _log(f"✅ Geography BG rows: {len(geo_key):,}")

        # --- dynamic scan width (robust across files) ---
        # read 1 row full-width to learn number of columns
        one = read_csv_from_zip(z, est_member, usecols=None, header=None, nrows=1)
        ncols = one.shape[1]
        scan_max = min(ncols, 260)  # scan first 260 columns or fewer if file smaller

        sample_n = 60000
        LOGREC_COL = 5
        usecols_sample = [LOGREC_COL] + list(range(0, scan_max))

        _log(f"🧪 Reading estimate sample (n={sample_n:,}) with cols [0..{scan_max-1}] plus LOGRECNO for NYC-only offset detection...")
        est_sample = read_csv_from_zip(z, est_member, usecols=usecols_sample, header=None, nrows=sample_n)
        est_sample = est_sample.rename(columns={LOGREC_COL: "LOGRECNO"})

        total_col, noveh_col, offset = detect_offset_on_nyc_bg(
            est_sample_with_logrec=est_sample,
            geo_key=geo_key,
            base_total_col=base_total_col,
            base_noveh_col=base_noveh_col,
            window=120,
        )

        usecols_final = sorted(set([LOGREC_COL, total_col, noveh_col]))
        _log(f"📥 Reading FINAL estimate columns: {usecols_final}")
        est_df = read_csv_from_zip(z, est_member, usecols=usecols_final, header=None)

        est_df = est_df.rename(columns={
            LOGREC_COL: "LOGRECNO",
            total_col: "hh_total_raw",
            noveh_col: "hh_no_vehicle_raw",
        })
        est_df["LOGRECNO"] = est_df["LOGRECNO"].astype(str).str.strip()

        _log("🔬 Raw estimate sample (LOGRECNO, total, no_vehicle):")
        _log(est_df[["LOGRECNO", "hh_total_raw", "hh_no_vehicle_raw"]].head(10).to_string(index=False))

    merged = geo_key.merge(est_df, on="LOGRECNO", how="inner")
    _log(f"✅ Joined rows: {len(merged):,}")

    merged["county_prefix"] = merged["GEOID"].astype(str).str[:5]
    merged = merged[merged["county_prefix"].isin(NYC_COUNTIES)].copy()
    _log(f"✅ NYC BG rows retained: {len(merged):,}")

    merged["hh_total"] = pd.to_numeric(merged["hh_total_raw"], errors="coerce").fillna(0)
    merged["hh_no_vehicle"] = pd.to_numeric(merged["hh_no_vehicle_raw"], errors="coerce").fillna(0)

    total_pos_share = (merged["hh_total"] > 0).mean()
    _log(f"QA: BG total>0 share = {total_pos_share:.3%}")

    bad_share = ((merged["hh_total"] > 0) & (merged["hh_no_vehicle"] > merged["hh_total"])).mean()
    _log(f"QA: BG no_vehicle > total share = {bad_share:.3%}")

    if total_pos_share < 0.80:
        raise RuntimeError("Totals not present for most NYC BGs — still wrong columns. Check top candidates output.")

    # --- extra sanity: BG no-vehicle share distribution ---
    p_bg = np.where(merged["hh_total"] > 0, 100.0 * merged["hh_no_vehicle"] / merged["hh_total"], np.nan)
    med = float(np.nanmedian(p_bg))
    p90 = float(np.nanpercentile(p_bg, 90))
    _log(f"QA: BG pct no-vehicle median={med:.2f}%, p90={p90:.2f}%")
    if not (1.0 <= med <= 70.0):
        _log("⚠️ QA WARNING: median BG no-vehicle share looks unusual. Verify selected columns / inputs.")

    bg = merged[["GEOID", "hh_total", "hh_no_vehicle"]].copy()
    bg["GEOID"] = bg["GEOID"].map(clean_geoid12)

    bg.to_parquet(CACHE_PARQUET, index=False)
    _log(f"💾 Cached extracted table to: {CACHE_PARQUET}")

# =========================================================
# Area-weight into station buffers
# =========================================================
detail = station_bg_detail.copy()
detail["GEOID"] = detail["GEOID"].map(clean_geoid12)
detail["area_ratio"] = parse_area_ratio(detail["area_ratio"])

bg = bg.copy()
bg["GEOID"] = bg["GEOID"].map(clean_geoid12)

detail = detail.merge(bg, on="GEOID", how="left")
_log(f"✅ Merge match rate (rows with hh_total): {detail['hh_total'].notna().mean():.2%}")

detail[["hh_total", "hh_no_vehicle"]] = detail[["hh_total", "hh_no_vehicle"]].fillna(0)

detail["hh_total_within"] = detail["hh_total"] * detail["area_ratio"]
detail["hh_no_vehicle_within"] = detail["hh_no_vehicle"] * detail["area_ratio"]

agg = detail.groupby(["stop_id", "stop_name"], as_index=False)[["hh_total_within", "hh_no_vehicle_within"]].sum()

agg[OUT_COL] = np.where(
    agg["hh_total_within"] > 0,
    100.0 * agg["hh_no_vehicle_within"] / agg["hh_total_within"],
    np.nan,
)

_log(f"🧪 Station-level {OUT_COL} summary:")
print(agg[OUT_COL].describe(percentiles=[.01, .05, .1, .25, .5, .75, .9, .95, .99]))

if OUT_COL in station_buffers.columns:
    station_buffers = station_buffers.drop(columns=[OUT_COL])

station_buffers = station_buffers.merge(
    agg[["stop_id", "stop_name", OUT_COL]],
    on=["stop_id", "stop_name"],
    how="left",
)

_log("✅ Variable 6 complete.")
display(station_buffers[["stop_id", "stop_name", OUT_COL]].head())
print("✅ Variable 6 output column:", OUT_COL)


NYC_COUNTIES (normalized) = ['36005', '36047', '36061', '36081', '36085']
✅ Using cached file: ./acs_sf_cache/NewYork_Tracts_Block_Groups_Only_2020.zip
🌐 Fetching lookup from web: https://www2.census.gov/programs-surveys/acs/summary_file/2020/documentation/user_tools/ACS_5yr_Seq_Table_Number_Lookup.txt
🔎 Lookup delimiter guessed as: ','
✅ Lookup loaded: 30,327 rows, 9 cols
✅ Lookup parse: table=B08201 seq=0027 startpos=75 cells=30
🧾 B08201 line labels (first 60):
 line_num                        label
        1                       Total:
        2         No vehicle available
        3          1 vehicle available
        4         2 vehicles available
        5         3 vehicles available
        6 4 or more vehicles available
        7          1-person household:
        8         No vehicle available
        9          1 vehicle available
       10         2 vehicles available
       11         3 vehicles available
       12 4 or more vehicles available
       13          2-pers

,stop_id,stop_name,pct_hh_no_vehicle_0p5mi
0,101,Van Cortlandt Park-242 St,34.697480
1,103,238 St,30.753513
2,104,231 St,25.197863
3,106,Marble Hill-225 St,20.724292
4,107,215 St,14.853057


✅ Variable 6 output column: pct_hh_no_vehicle_0p5mi


In [20]:
# =========================================================
# Urban Variable 7 (CAPSTONE-READY, DETERMINISTIC): Station-level commute mode shares (0.5 mile)
# Data source: ACS 2019 5-year Summary File (SEQUENCE-BASED), NY Tracts+Block Groups only
#
# Keeps ONLY the capstone-useful, non-redundant features (7 columns):
#   1) pct_transit_0p5mi
#   2) pct_car_truck_van_0p5mi
#   3) pct_walked_0p5mi
#   4) pct_bicycle_0p5mi
#   5) pct_wfh_0p5mi
#   6) pct_taxicab_0p5mi
#   7) pct_other_means_0p5mi
#
# Why this subset:
# - Interpretable and stable
# - Avoids parent/child redundancy (no bus/subway breakdown; no carpool breakdown)
# - Captures broad mode split + WFH sanity anchor
#
# Requires:
# - station_bg_detail: GEOID (12), area_ratio, stop_id, stop_name
# - station_buffers: stop_id, stop_name
# - nyc_countyfps: either ["005","047","061","081","085"] OR ["36005","36047","36061","36081","36085"]
# =========================================================

import os
import io
import re
import zipfile
import hashlib
import requests
import pandas as pd
import numpy as np

# -----------------------------
# Config
# -----------------------------
ACS_ENDYEAR = 2019
STATE_FIPS = "36"
STATE_ABBR = "ny"

TABLE_ID = "B08301"
CELLS_EXPECTED = 21

WORKDIR = "./acs_sf_cache"
os.makedirs(WORKDIR, exist_ok=True)

NY_ZIP_URL = (
    "https://www2.census.gov/programs-surveys/acs/summary_file/2019/data/5_year_by_state/"
    "NewYork_Tracts_Block_Groups_Only.zip"
)
LOOKUP_URL = (
    "https://www2.census.gov/programs-surveys/acs/summary_file/2019/documentation/user_tools/"
    "ACS_5yr_Seq_Table_Number_Lookup.txt"
)

NY_ZIP_PATH = os.path.join(WORKDIR, "NewYork_Tracts_Block_Groups_Only_2019.zip")

DEBUG = True
CACHE_VERSION = "v8_capstone_subset_b08301_block"
def _cache_path():
    key = f"{ACS_ENDYEAR}|{TABLE_ID}|{STATE_FIPS}|{CACHE_VERSION}"
    h = hashlib.md5(key.encode("utf-8")).hexdigest()[:10]
    return os.path.join(WORKDIR, f"acs{ACS_ENDYEAR}_{STATE_ABBR}_bg_{TABLE_ID}_{h}.parquet")

CACHE_PARQUET = _cache_path()

# -----------------------------
# Helpers
# -----------------------------
def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

def normalize_nyc_counties(nyc_countyfps, state_fips="36"):
    out = set()
    for c in nyc_countyfps:
        s = re.sub(r"\D", "", str(c).strip())
        if len(s) == 3:
            out.add(state_fips + s)
        elif len(s) == 5:
            out.add(s)
        else:
            out.add(state_fips + s[-3:].zfill(3))
    return out

NYC_COUNTIES = normalize_nyc_counties(nyc_countyfps, STATE_FIPS)
_log(f"NYC_COUNTIES (normalized) = {sorted(NYC_COUNTIES)}")

def download_if_missing(url: str, path: str, chunk=1024 * 1024):
    if os.path.exists(path) and os.path.getsize(path) > 0:
        _log(f"✅ Using cached file: {path}")
        return
    _log(f"⬇️ Downloading: {url}")
    with requests.get(url, stream=True, timeout=300) as r:
        r.raise_for_status()
        with open(path, "wb") as f:
            for part in r.iter_content(chunk_size=chunk):
                if part:
                    f.write(part)
    _log(f"✅ Saved: {path} ({os.path.getsize(path)/1e6:.1f} MB)")

def sniff_delimiter(sample: bytes) -> str:
    text = sample.decode("latin-1", errors="replace")
    candidates = [",", "\t", "|"]
    best = ","
    best_cols = 0
    first = text.splitlines()[0] if text.splitlines() else ""
    for d in candidates:
        cols = len(first.split(d)) if first else 0
        if cols > best_cols:
            best_cols = cols
            best = d
    return best

def read_lookup_from_web(url: str) -> pd.DataFrame:
    _log(f"🌐 Fetching lookup from web: {url}")
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    raw = r.content
    sep = sniff_delimiter(raw[:2000])
    _log(f"🔎 Lookup delimiter guessed as: {repr(sep)}")
    df = pd.read_csv(io.BytesIO(raw), sep=sep, header=None, dtype=str, encoding="latin-1", engine="python")
    df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
    _log(f"✅ Lookup loaded: {len(df):,} rows, {df.shape[1]} cols")
    return df

def get_seq_startpos_cells(lookup_df: pd.DataFrame, table_id: str):
    tt = lookup_df[lookup_df[1].astype(str).str.upper() == table_id.upper()].copy()
    if tt.empty:
        raise RuntimeError(f"Table {table_id} not found in lookup.")

    header = tt[tt.apply(lambda r: r.astype(str).str.contains("CELLS", case=False, na=False).any(), axis=1)]
    if header.empty:
        preview = tt.head(10).to_string(index=False)
        raise RuntimeError(f"Found rows for {table_id} but none contained 'CELLS'. Preview:\n{preview}")

    hr = header.iloc[0].tolist()
    seq = str(hr[2]).strip().zfill(4)

    cells_idx, cells_val = None, None
    for i, v in enumerate(hr):
        if isinstance(v, str) and "CELLS" in v.upper():
            cells_idx, cells_val = i, v
            break

    m = re.search(r"(\d+)\s*CELLS", str(cells_val), flags=re.I)
    if not m:
        raise RuntimeError(f"Could not parse cells count from '{cells_val}' for {table_id}.")
    cells = int(m.group(1))

    startpos = None
    for j in range(cells_idx - 1, -1, -1):
        v = hr[j]
        if isinstance(v, str) and v.strip().isdigit():
            startpos = int(v.strip())
            break
    if startpos is None:
        raise RuntimeError(f"Could not parse startpos for {table_id} from header row: {hr}")

    _log(f"✅ Lookup parse: table={table_id} seq={seq} startpos={startpos} cells={cells}")
    return seq, startpos, cells

def read_csv_from_zip(z: zipfile.ZipFile, member: str, usecols=None, header=None, nrows=None) -> pd.DataFrame:
    raw = z.read(member)
    sep = sniff_delimiter(raw[:2000])
    return pd.read_csv(
        io.BytesIO(raw),
        sep=sep,
        header=header,
        dtype=str,
        encoding="latin-1",
        engine="python",
        usecols=usecols,
        nrows=nrows
    )

def find_file_in_zip(z: zipfile.ZipFile, predicate, label="file") -> str:
    for name in z.namelist():
        if predicate(name):
            return name
    raise RuntimeError(f"No matching {label} found in zip.")

def read_geo_from_zip_auto(z: zipfile.ZipFile, member: str) -> pd.DataFrame:
    raw = z.read(member)
    sep = sniff_delimiter(raw[:2000])

    df0 = pd.read_csv(io.BytesIO(raw), sep=sep, header=0, dtype=str, encoding="latin-1", engine="python")
    cols = [str(c).strip().upper() for c in df0.columns]
    useful = any(c in ("GEO_ID", "GEOID", "LOGRECNO") for c in cols) or any("LOGREC" in c for c in cols)
    if useful:
        df0.columns = [str(c).strip() for c in df0.columns]
        return df0

    df = pd.read_csv(io.BytesIO(raw), sep=sep, header=None, dtype=str, encoding="latin-1", engine="python")
    df.columns = list(range(df.shape[1]))
    return df

def build_bg_geoid_from_geofile(geo_df: pd.DataFrame) -> pd.DataFrame:
    g = geo_df.copy()

    if not isinstance(g.columns[0], (int, np.integer)):
        cols_upper = {str(c).strip().upper(): c for c in g.columns}

        def pick(*names):
            for n in names:
                if n in cols_upper:
                    return cols_upper[n]
            return None

        logrec = pick("LOGRECNO") or next((cols_upper[k] for k in cols_upper if "LOGREC" in k), None)
        if logrec is None:
            raise RuntimeError("Geo: could not find LOGRECNO-like column.")

        geoid_src = pick("GEO_ID", "GEOID")
        if geoid_src is None:
            raise RuntimeError("Geo: could not find GEO_ID/GEOID column.")

        s = g[geoid_src].astype(str).str.strip()
        geoid12 = s.str.extract(r"US(36\d{10})", expand=False)

        out = pd.DataFrame({"LOGRECNO": g[logrec].astype(str).str.strip(), "GEOID": geoid12}).dropna(subset=["GEOID"])
        out = out[out["GEOID"].str.fullmatch(r"36\d{10}", na=False)].drop_duplicates()
        if out.empty:
            raise RuntimeError("Geo: couldn't extract any US36##########.")
        return out

    g.columns = list(range(g.shape[1]))
    def logrec_score(col):
        s = g[col].astype(str).str.strip()
        return (s.str.fullmatch(r"\d+").mean(), s.nunique())
    scores = {c: logrec_score(c) for c in g.columns}
    logrec_col = sorted(scores, key=lambda c: (scores[c][0], scores[c][1]), reverse=True)[0]

    best_col, best_hits = None, 0
    for c in g.columns:
        s = g[c].astype(str).str.strip()
        hits = s.str.contains(r"US36\d{10}", regex=True, na=False).sum()
        if hits > best_hits:
            best_hits, best_col = hits, c

    if best_col is None or best_hits < 100:
        raise RuntimeError("Geo headerless: couldn't find a US36########## column.")

    geoid12 = g[best_col].astype(str).str.strip().str.extract(r"US(36\d{10})", expand=False)
    out = pd.DataFrame({"LOGRECNO": g[logrec_col].astype(str).str.strip(), "GEOID": geoid12}).dropna(subset=["GEOID"])
    out = out[out["GEOID"].str.fullmatch(r"36\d{10}", na=False)].drop_duplicates()
    if out.empty:
        raise RuntimeError("Geo headerless: extraction produced no NY BG GEOIDs.")
    return out

def detect_b08301_block_start(df_sample: pd.DataFrame, cells: int = 21) -> int:
    X = df_sample.apply(pd.to_numeric, errors="coerce").fillna(0).to_numpy()
    _, ncols = X.shape

    # within-block indices (0-based)
    i_total = 0
    i_car = 1
    i_transit = 9
    i_taxicab = 15
    i_bike = 17
    i_walk = 18
    i_other = 19
    i_wfh = 20
    i_moto = 16  # not used in identity? include it to be safe
    # identity uses motorcycle too
    best_s, best_score = None, -1.0

    for s in range(0, ncols - cells + 1):
        block = X[:, s : s + cells]
        total = block[:, i_total]
        if (total > 0).mean() < 0.30:
            continue

        rhs = (
            block[:, i_car]
            + block[:, i_transit]
            + block[:, i_taxicab]
            + block[:, i_moto]
            + block[:, i_bike]
            + block[:, i_walk]
            + block[:, i_other]
            + block[:, i_wfh]
        )

        match = ((total == rhs) & (total > 0)).mean()
        share_transit = np.divide(block[:, i_transit], total, out=np.zeros_like(total), where=total > 0)
        near_one = (share_transit > 0.90).mean()
        score = match - 0.25 * near_one

        if score > best_score:
            best_score = score
            best_s = s

    if best_s is None or best_score < 0.50:
        raise RuntimeError(f"Could not reliably detect B08301 block. Best score={best_score:.3f}.")
    _log(f"✅ Detected B08301 block start column = {best_s} (score={best_score:.3f})")
    return best_s

# -----------------------------
# Capstone subset: lines to keep
# -----------------------------
# B08301 lines:
#  2 car/truck/van
# 10 public transportation (excluding taxicab)
# 16 taxicab
# 18 bicycle
# 19 walked
# 20 other means
# 21 worked from home
KEEP_LINES = {
    2:  "pct_car_truck_van_0p5mi",
    10: "pct_transit_0p5mi",
    16: "pct_taxicab_0p5mi",
    18: "pct_bicycle_0p5mi",
    19: "pct_walked_0p5mi",
    20: "pct_other_means_0p5mi",
    21: "pct_wfh_0p5mi",
}

# -----------------------------
# Main
# -----------------------------
download_if_missing(NY_ZIP_URL, NY_ZIP_PATH)

if os.path.exists(CACHE_PARQUET):
    _log(f"✅ Loading cached extract: {CACHE_PARQUET}")
    mode_bg = pd.read_parquet(CACHE_PARQUET)

else:
    lookup = read_lookup_from_web(LOOKUP_URL)
    seq4, startpos, cells = get_seq_startpos_cells(lookup, TABLE_ID)
    if cells != CELLS_EXPECTED:
        _log(f"⚠️ Lookup cells={cells} (expected {CELLS_EXPECTED}). Proceeding with lookup value.")

    with zipfile.ZipFile(NY_ZIP_PATH, "r") as z:
        geo_member = find_file_in_zip(
            z,
            lambda n: re.search(r"(^|/)(g).*" + STATE_ABBR + r".*\.(csv|txt)$", n, flags=re.I) is not None,
            label="geography file"
        )
        _log(f"🗺️ Geography member: {geo_member}")

        est_member = find_file_in_zip(
            z,
            lambda n: (
                re.search(r"(^|/)(e).*" + STATE_ABBR + r".*\.(csv|txt)$", n, flags=re.I) is not None
                and seq4 in os.path.basename(n)
            ),
            label="estimate sequence file"
        )
        _log(f"📄 Estimate member: {est_member}")

        sample_n = 12000
        _log(f"🧪 Reading estimate sample (first {sample_n} rows) to detect B08301 block...")
        est_sample = read_csv_from_zip(z, est_member, usecols=None, header=None, nrows=sample_n)
        _log(f"🧪 Sample shape: {est_sample.shape[0]:,} rows x {est_sample.shape[1]} cols")

        block_start = detect_b08301_block_start(est_sample, cells=cells)

        LOGREC_COL = 5
        table_cols = list(range(block_start, block_start + cells))
        usecols = sorted(set([LOGREC_COL] + table_cols))
        _log(f"📥 Reading LOGRECNO + {cells} table columns with usecols (min={min(usecols)}, max={max(usecols)})")

        try:
            est_df = read_csv_from_zip(z, est_member, usecols=usecols, header=None)
        except Exception as e:
            _log(f"⚠️ usecols failed ({type(e).__name__}: {e}). Falling back to full read then slice.")
            full = read_csv_from_zip(z, est_member, usecols=None, header=None)
            est_df = full.iloc[:, usecols].copy()

        # rename columns -> LOGRECNO + line_01..line_21
        col_map = {}
        for c in est_df.columns:
            if c == LOGREC_COL:
                col_map[c] = "LOGRECNO"
            else:
                line_num = (c - block_start) + 1
                col_map[c] = f"line_{line_num:02d}"
        est_df = est_df.rename(columns=col_map)
        est_df["LOGRECNO"] = est_df["LOGRECNO"].astype(str).str.strip()

        _log("🔬 Raw estimate sample (LOGRECNO, Total, Transit, WFH):")
        _log(est_df[["LOGRECNO", "line_01", "line_10", "line_21"]].head(8).to_string(index=False))

        _log("📥 Reading geography file (can take a bit)...")
        geo_df = read_geo_from_zip_auto(z, geo_member)
        geo_key = build_bg_geoid_from_geofile(geo_df)
        _log(f"✅ Geography BG rows: {len(geo_key):,}")

    merged = geo_key.merge(est_df, on="LOGRECNO", how="inner")
    _log(f"✅ Joined rows: {len(merged):,}")

    merged["county_prefix"] = merged["GEOID"].str[:5]
    _log("🔎 Top county_prefix counts (before NYC filter):")
    _log(merged["county_prefix"].value_counts().head(15).to_string())

    merged = merged[merged["county_prefix"].isin(NYC_COUNTIES)].copy()
    _log(f"✅ NYC BG rows retained: {len(merged):,}")

    # Convert needed lines only
    needed_line_cols = ["line_01"] + [f"line_{ln:02d}" for ln in KEEP_LINES.keys()]
    for c in needed_line_cols:
        merged[c] = pd.to_numeric(merged[c], errors="coerce").fillna(0)

    # QA: transit <= total
    bad_share = ((merged["line_01"] > 0) & (merged["line_10"] > merged["line_01"])).mean()
    _log(f"QA: BG transit > total share = {bad_share:.3%}")
    if bad_share > 0.001:
        _log(merged.loc[(merged["line_01"] > 0) & (merged["line_10"] > merged["line_01"]),
                        ["GEOID","line_01","line_10"]].head(10).to_string(index=False))
        raise RuntimeError("Detected block yields transit>total. Check parsing/detection.")

    mode_bg = merged[["GEOID"] + needed_line_cols].copy()
    mode_bg["GEOID"] = mode_bg["GEOID"].astype(str).str.zfill(12)

    mode_bg.to_parquet(CACHE_PARQUET, index=False)
    _log(f"💾 Cached BG table to: {CACHE_PARQUET}")

# =========================================================
# Area-weight BG into station buffers and compute capstone % columns
# =========================================================
detail = station_bg_detail.copy()
detail["GEOID"] = detail["GEOID"].astype(str).str.zfill(12)
detail["area_ratio"] = pd.to_numeric(detail["area_ratio"], errors="coerce").fillna(0)

mode_bg = mode_bg.copy()
mode_bg["GEOID"] = mode_bg["GEOID"].astype(str).str.zfill(12)

detail = detail.merge(mode_bg, on="GEOID", how="left")

needed_line_cols = ["line_01"] + [f"line_{ln:02d}" for ln in KEEP_LINES.keys()]
detail[needed_line_cols] = detail[needed_line_cols].fillna(0)

# Weighted counts
detail["total_within"] = detail["line_01"] * detail["area_ratio"]
for ln in KEEP_LINES.keys():
    c = f"line_{ln:02d}"
    detail[c + "_within"] = detail[c] * detail["area_ratio"]

# Aggregate to stops
sum_cols = ["total_within"] + [f"line_{ln:02d}_within" for ln in KEEP_LINES.keys()]
agg = detail.groupby(["stop_id","stop_name"], as_index=False)[sum_cols].sum()

# Compute % columns
pct_cols = []
for ln, outcol in KEEP_LINES.items():
    within_col = f"line_{ln:02d}_within"
    agg[outcol] = np.where(agg["total_within"] > 0, 100.0 * agg[within_col] / agg["total_within"], np.nan)
    pct_cols.append(outcol)

# Sanity check: WFH should be single digits/teens for 2019
_log("🧪 Station-level WFH share summary (pct_wfh_0p5mi):")
print(agg["pct_wfh_0p5mi"].describe(percentiles=[.01,.05,.1,.25,.5,.75,.9,.95,.99]))

# Merge into station_buffers (drop old if present)
for c in pct_cols:
    if c in station_buffers.columns:
        station_buffers = station_buffers.drop(columns=[c])

station_buffers = station_buffers.merge(
    agg[["stop_id","stop_name"] + pct_cols],
    on=["stop_id","stop_name"],
    how="left"
)

_log("✅ Capstone commute mode shares complete.")
display(station_buffers[["stop_id","stop_name"] + pct_cols].head())
display(station_buffers[pct_cols].describe())
print("✅ Added capstone columns:", pct_cols)


NYC_COUNTIES (normalized) = ['36005', '36047', '36061', '36081', '36085']
⬇️ Downloading: https://www2.census.gov/programs-surveys/acs/summary_file/2019/data/5_year_by_state/NewYork_Tracts_Block_Groups_Only.zip
✅ Saved: ./acs_sf_cache/NewYork_Tracts_Block_Groups_Only_2019.zip (265.6 MB)
🌐 Fetching lookup from web: https://www2.census.gov/programs-surveys/acs/summary_file/2019/documentation/user_tools/ACS_5yr_Seq_Table_Number_Lookup.txt
🔎 Lookup delimiter guessed as: ','
✅ Lookup loaded: 29,502 rows, 9 cols
✅ Lookup parse: table=B08301 seq=0027 startpos=157 cells=21
🗺️ Geography member: g20195ny.csv
📄 Estimate member: e20195ny0027000.txt
🧪 Reading estimate sample (first 12000 rows) to detect B08301 block...
🧪 Sample shape: 12,000 rows x 205 cols
✅ Detected B08301 block start column = 156 (score=0.984)
📥 Reading LOGRECNO + 21 table columns with usecols (min=5, max=176)
🔬 Raw estimate sample (LOGRECNO, Total, Transit, WFH):
LOGRECNO line_01 line_10 line_21
 0003392     888      99      14

,stop_id,stop_name,pct_car_truck_van_0p5mi,pct_transit_0p5mi,pct_taxicab_0p5mi,pct_bicycle_0p5mi,pct_walked_0p5mi,pct_other_means_0p5mi,pct_wfh_0p5mi
0,101,Van Cortlandt Park-242 St,30.543990,53.240565,1.119433,0.506095,9.291717,0.359779,4.938422
1,103,238 St,26.381776,58.646856,1.091549,0.452522,8.468014,0.866780,4.004387
2,104,231 St,24.333560,63.181387,0.823569,0.430627,6.818583,1.224881,3.121919
3,106,Marble Hill-225 St,22.956021,63.200495,0.990886,0.692038,6.682591,1.180028,3.877528
4,107,215 St,14.766364,66.345185,1.123450,0.617780,8.731441,1.075166,6.577047


,pct_car_truck_van_0p5mi,pct_transit_0p5mi,pct_taxicab_0p5mi,pct_bicycle_0p5mi,pct_walked_0p5mi,pct_other_means_0p5mi,pct_wfh_0p5mi
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,19.403600,59.603179,1.166024,1.494372,12.786834,0.628998,4.847267
std,15.087383,12.539803,1.163016,1.244440,10.433874,0.397095,2.453330
min,2.209194,16.578355,0.000000,0.000000,0.763190,0.000000,0.000000
25%,7.361411,51.582249,0.394734,0.506095,5.957587,0.356460,2.947263
50%,16.188789,62.506567,0.738188,1.111548,8.527354,0.583690,4.456673
75%,26.641323,69.422971,1.523774,2.313493,15.160428,0.837740,6.489859
max,79.845722,94.737580,5.965763,5.931691,44.990283,2.647126,12.286622


✅ Added capstone columns: ['pct_car_truck_van_0p5mi', 'pct_transit_0p5mi', 'pct_taxicab_0p5mi', 'pct_bicycle_0p5mi', 'pct_walked_0p5mi', 'pct_other_means_0p5mi', 'pct_wfh_0p5mi']


In [21]:
# =========================================================
# Urban Variable 8: % Workers Working from Home (station catchment)
# Source: ACS 2020 5-year (same commute mode table as above)
# Method: Area-weighted aggregation using station_bg_detail area_ratio
# =========================================================

import pandas as pd
import requests

acs_base = "https://api.census.gov/data/2020/acs/acs5"
meta = requests.get(f"{acs_base}/variables.json", timeout=60).json()["variables"]

# --- USER MUST CONFIRM THESE VARIABLE IDS FOR 2020 ACS5 ---
TOTAL_WORKERS_VAR = "B08301_001E"
WFH_VAR = "B08301_021E"  # verify; "Worked from home" code can vary by table/vintage

if TOTAL_WORKERS_VAR not in meta or WFH_VAR not in meta:
    raise ValueError(
        "WFH variable codes not found in ACS metadata. "
        "Inspect variables.json to choose correct IDs for total workers and worked-from-home."
    )

acs_parts = []
for c in sorted(nyc_countyfps):
    url = (
        f"{acs_base}"
        f"?get=NAME,{TOTAL_WORKERS_VAR},{WFH_VAR}"
        f"&for=block%20group:*"
        f"&in=state:36%20county:{c}%20tract:*"
    )
    resp = requests.get(url, timeout=120)
    resp.raise_for_status()
    data = resp.json()
    acs_parts.append(pd.DataFrame(data[1:], columns=data[0]))

wfh = pd.concat(acs_parts, ignore_index=True)

wfh["GEOID"] = (
    wfh["state"].astype(str).str.zfill(2) +
    wfh["county"].astype(str).str.zfill(3) +
    wfh["tract"].astype(str).str.zfill(6) +
    wfh["block group"].astype(str).str.zfill(1)
)

wfh["workers_total"] = pd.to_numeric(wfh[TOTAL_WORKERS_VAR], errors="coerce").fillna(0)
wfh["workers_wfh"] = pd.to_numeric(wfh[WFH_VAR], errors="coerce").fillna(0)
wfh = wfh[["GEOID", "workers_total", "workers_wfh"]]

detail = station_bg_detail.merge(wfh, on="GEOID", how="left")
detail[["workers_total", "workers_wfh"]] = detail[["workers_total", "workers_wfh"]].fillna(0)

detail["workers_total_within"] = detail["workers_total"] * detail["area_ratio"]
detail["workers_wfh_within"] = detail["workers_wfh"] * detail["area_ratio"]

agg = (
    detail.groupby(["stop_id", "stop_name"])[["workers_total_within", "workers_wfh_within"]]
    .sum()
    .reset_index()
)

agg["pct_workers_wfh_0p5mi"] = (
    100.0 * agg["workers_wfh_within"] / agg["workers_total_within"]
).replace([pd.NA, pd.NaT, float("inf")], 0).fillna(0)

station_buffers = station_buffers.merge(
    agg[["stop_id", "stop_name", "pct_workers_wfh_0p5mi"]],
    on=["stop_id", "stop_name"],
    how="left",
)
station_buffers["pct_workers_wfh_0p5mi"] = station_buffers["pct_workers_wfh_0p5mi"].fillna(0)

display(station_buffers[["stop_id", "stop_name", "pct_workers_wfh_0p5mi"]].head())


,stop_id,stop_name,pct_workers_wfh_0p5mi
0,101,Van Cortlandt Park-242 St,8.004019
1,103,238 St,7.269194
2,104,231 St,5.364991
3,106,Marble Hill-225 St,6.247171
4,107,215 St,8.612343


In [22]:
# =========================================================
# Urban Variable 9: Median Household Income (station catchment) - SAFE VERSION
# Source: ACS 2020 5-year (API), B19013_001E (median HH income) + B11001_001E (total households)
# NOTE: Medians do not aggregate perfectly.
# Practical approximation: household-weighted average of BG medians within station buffer.
# =========================================================

import numpy as np
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

ACS_YEAR = 2020
acs_base = f"https://api.census.gov/data/{ACS_YEAR}/acs/acs5"

MED_INC_VAR = "B19013_001E"   # Median household income (dollars)
HH_VAR      = "B11001_001E"   # Total households (for weighting)

OUT_COL = "median_hh_income_0p5mi_approx"

# -----------------------------
# Small HTTP helper with retries (prevents random API hiccups)
# -----------------------------
def make_session():
    s = requests.Session()
    retry = Retry(
        total=6,
        connect=6,
        read=6,
        status=6,
        backoff_factor=0.8,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        respect_retry_after_header=True,
        raise_on_status=False,
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s

sess = make_session()

def get_json(url: str, timeout=120):
    r = sess.get(url, timeout=timeout)
    r.raise_for_status()
    return r.json()

# -----------------------------
# 1) Pull BG median income + households for NYC counties
# -----------------------------
acs_parts = []
for c in sorted(nyc_countyfps):
    url = (
        f"{acs_base}"
        f"?get=NAME,{MED_INC_VAR},{HH_VAR}"
        f"&for=block%20group:*"
        f"&in=state:36%20county:{c}%20tract:*"
    )
    data = get_json(url)
    acs_parts.append(pd.DataFrame(data[1:], columns=data[0]))

inc = pd.concat(acs_parts, ignore_index=True)

# Build 12-digit BG GEOID
inc["GEOID"] = (
    inc["state"].astype(str).str.zfill(2)
    + inc["county"].astype(str).str.zfill(3)
    + inc["tract"].astype(str).str.zfill(6)
    + inc["block group"].astype(str).str.zfill(1)
)

# -----------------------------
# 2) Parse + CLEAN values (this is the main fix)
# -----------------------------
inc["median_income_bg"] = pd.to_numeric(inc[MED_INC_VAR], errors="coerce")
inc["hh_total"] = pd.to_numeric(inc[HH_VAR], errors="coerce")

# Drop nonsense / sentinel / suppressed values:
# - median should be positive
# - households should be positive
inc = inc[(inc["median_income_bg"].notna()) & (inc["hh_total"].notna())].copy()
inc = inc[(inc["median_income_bg"] > 0) & (inc["hh_total"] > 0)].copy()

# Optional guardrail: clip to a plausible range so one weird BG can't dominate
# (You can remove this if you prefer raw ACS after filtering)
inc["median_income_bg"] = inc["median_income_bg"].clip(lower=1, upper=500000)

inc = inc[["GEOID", "median_income_bg", "hh_total"]].copy()

# -----------------------------
# 3) Join to station_bg_detail and compute household-weighted approximation
# -----------------------------
detail = station_bg_detail.copy()
detail["GEOID"] = detail["GEOID"].astype(str).str.zfill(12)

detail = detail.merge(inc, on="GEOID", how="left")

# If a BG is missing median/households, exclude it from BOTH numerator/denominator
detail["hh_total"] = detail["hh_total"].where(detail["median_income_bg"].notna(), np.nan)

# Area-weight households into buffer
detail["hh_within"] = detail["hh_total"] * detail["area_ratio"]

# Weighted numerator
detail["inc_weighted"] = detail["median_income_bg"] * detail["hh_within"]

agg = (
    detail.groupby(["stop_id", "stop_name"], as_index=False)[["inc_weighted", "hh_within"]]
    .sum(min_count=1)
)

# Safe divide
agg[OUT_COL] = np.where(
    agg["hh_within"] > 0,
    agg["inc_weighted"] / agg["hh_within"],
    np.nan
)

# Format
agg[OUT_COL] = agg[OUT_COL].round(0)

# -----------------------------
# 4) Merge into station_buffers (avoid duplicate columns)
# -----------------------------
station_buffers = station_buffers.drop(columns=[OUT_COL], errors="ignore")

station_buffers = station_buffers.merge(
    agg[["stop_id", "stop_name", OUT_COL]],
    on=["stop_id", "stop_name"],
    how="left",
)

# Optional: keep NaN if you want “missingness” visible; otherwise fill with 0
# station_buffers[OUT_COL] = station_buffers[OUT_COL].fillna(0).astype(int)
station_buffers[OUT_COL] = station_buffers[OUT_COL].astype("float")

display(station_buffers[["stop_id", "stop_name", OUT_COL]].head())

# Quick sanity check
print("Sanity check (station-level):")
print(station_buffers[OUT_COL].describe(percentiles=[.01,.05,.1,.25,.5,.75,.9,.95,.99]))
print("Missing share:", station_buffers[OUT_COL].isna().mean())


,stop_id,stop_name,median_hh_income_0p5mi_approx
0,101,Van Cortlandt Park-242 St,73618.0
1,103,238 St,67034.0
2,104,231 St,56674.0
3,106,Marble Hill-225 St,59890.0
4,107,215 St,67005.0


Sanity check (station-level):
count       518.000000
mean      86477.841699
std       42026.589007
min       26214.000000
1%        28229.990000
5%        36035.300000
10%       41486.300000
25%       56527.500000
50%       70323.500000
75%      117781.250000
90%      148653.100000
95%      165508.200000
99%      191512.160000
max      201221.000000
Name: median_hh_income_0p5mi_approx, dtype: float64
Missing share: 0.0


In [23]:
# =========================================================
# Connectivity Variable 1: Number of Connecting Bus Routes (within 0.5 mile buffer)
# Source: MTA Bus GTFS static feeds (downloaded from URLs, cached locally)
# Output: station_buffers["bus_routes_0p5mi"]
#
# Assumes:
# - station_buffers is a GeoDataFrame with stop_id, stop_name, geometry
# - station_buffers.crs is projected (e.g., EPSG:2263)
# =========================================================

import os
import zipfile
import requests
import pandas as pd
import geopandas as gpd
import numpy as np

# -----------------------------
# Config
# -----------------------------
WORKDIR = "./gtfs_cache"
os.makedirs(WORKDIR, exist_ok=True)

BUS_GTFS_URLS = [
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_bx.zip",
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_b.zip",
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_m.zip",
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_q.zip",
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_si.zip",
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_busco.zip",
]

OUT_COL = "bus_routes_0p5mi"
DEBUG = True

def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

# -----------------------------
# Download (cached)
# -----------------------------
def download_gtfs_zip(url: str, out_dir: str, chunk=1024 * 1024) -> str:
    """
    Download a GTFS zip to out_dir if not already present.
    Returns local filepath.
    """
    base = url.split("?")[0]
    fname = os.path.join(out_dir, os.path.basename(base) or "gtfs_bus.zip")

    if os.path.exists(fname) and os.path.getsize(fname) > 0:
        _log(f"✅ Using cached GTFS zip: {fname}")
        return fname

    _log(f"⬇️ Downloading GTFS: {url}")
    with requests.get(url, stream=True, timeout=300) as r:
        r.raise_for_status()
        with open(fname, "wb") as f:
            for part in r.iter_content(chunk_size=chunk):
                if part:
                    f.write(part)

    _log(f"✅ Saved: {fname} ({os.path.getsize(fname)/1e6:.1f} MB)")
    return fname

gtfs_zip_paths = [download_gtfs_zip(u, WORKDIR) for u in BUS_GTFS_URLS]
_log(f"✅ Ready GTFS zips: {len(gtfs_zip_paths)}")

# -----------------------------
# Helpers
# -----------------------------
def _read_gtfs_csv(z: zipfile.ZipFile, name: str, usecols=None, dtype=None) -> pd.DataFrame:
    with z.open(name) as f:
        return pd.read_csv(f, usecols=usecols, dtype=dtype)

def _iter_gtfs_csv_chunks(z: zipfile.ZipFile, name: str, usecols=None, dtype=None, chunksize=600_000):
    with z.open(name) as f:
        for chunk in pd.read_csv(f, usecols=usecols, dtype=dtype, chunksize=chunksize):
            yield chunk

def _to_station_crs_points(df_stops: pd.DataFrame, station_crs) -> gpd.GeoDataFrame:
    gdf = gpd.GeoDataFrame(
        df_stops,
        geometry=gpd.points_from_xy(df_stops["stop_lon"], df_stops["stop_lat"]),
        crs="EPSG:4326"
    )
    return gdf.to_crs(station_crs)

# -----------------------------
# Main compute
# -----------------------------
def compute_bus_routes_for_station_buffers(station_buffers: gpd.GeoDataFrame, gtfs_zip_paths: list) -> pd.DataFrame:
    for col in ["stop_id", "stop_name", "geometry"]:
        if col not in station_buffers.columns:
            raise ValueError(f"station_buffers missing required column: {col}")

    all_stops = []
    feed_trip_to_route = []

    for zp in gtfs_zip_paths:
        _log(f"📦 Processing GTFS zip: {zp}")
        with zipfile.ZipFile(zp, "r") as z:
            names = set(z.namelist())
            needed = {"stops.txt", "trips.txt", "stop_times.txt"}
            missing = needed - names
            if missing:
                raise RuntimeError(f"{os.path.basename(zp)} missing required files: {missing}")

            # Stops: rename stop_id -> bus_stop_id now to avoid sjoin collision
            stops = _read_gtfs_csv(
                z, "stops.txt",
                usecols=["stop_id", "stop_lat", "stop_lon"],
                dtype={"stop_id": str, "stop_lat": float, "stop_lon": float}
            ).rename(columns={"stop_id": "bus_stop_id"})

            stops["bus_stop_id"] = stops["bus_stop_id"].astype(str)

            stops_gdf = _to_station_crs_points(stops, station_buffers.crs)
            all_stops.append(stops_gdf[["bus_stop_id", "geometry"]])

            trips = _read_gtfs_csv(
                z, "trips.txt",
                usecols=["trip_id", "route_id"],
                dtype={"trip_id": str, "route_id": str}
            ).dropna().drop_duplicates()

            feed_trip_to_route.append(trips)

    stops_all = pd.concat(all_stops, ignore_index=True).drop_duplicates(subset=["bus_stop_id"])
    stops_all = gpd.GeoDataFrame(stops_all, geometry="geometry", crs=station_buffers.crs)
    _log(f"🧾 Unique bus stops loaded: {len(stops_all):,}")

    # Spatial join: bus stops within station buffers
    join = gpd.sjoin(
        stops_all,
        station_buffers[["stop_id", "stop_name", "geometry"]],
        how="inner",
        predicate="within"
    )

    needed_cols = {"stop_id", "stop_name", "bus_stop_id"}
    if not needed_cols.issubset(set(join.columns)):
        _log("⚠️ sjoin columns:")
        _log(str(sorted(join.columns)))
        raise RuntimeError(
            f"sjoin did not produce expected columns {needed_cols}. "
            "Check station_buffers columns or GeoPandas behavior."
        )

    join = join[["stop_id", "stop_name", "bus_stop_id"]].drop_duplicates()
    _log(f"🔗 Bus stops within station buffers: {len(join):,}")

    if join.empty:
        out = station_buffers[["stop_id", "stop_name"]].drop_duplicates().copy()
        out[OUT_COL] = 0
        return out

    target_stop_ids = set(join["bus_stop_id"].unique().tolist())
    _log(f"🎯 Target bus stop_ids to map -> routes: {len(target_stop_ids):,}")

    stop_route_pairs = []

    for zp, trips in zip(gtfs_zip_paths, feed_trip_to_route):
        with zipfile.ZipFile(zp, "r") as z:
            _log(f"🧩 Mapping bus_stop_id->route_id (chunked): {os.path.basename(zp)}")
            trip_to_route = trips.set_index("trip_id")["route_id"]

            for chunk in _iter_gtfs_csv_chunks(
                z, "stop_times.txt",
                usecols=["trip_id", "stop_id"],
                dtype={"trip_id": str, "stop_id": str},
                chunksize=600_000
            ):
                chunk = chunk[chunk["stop_id"].isin(target_stop_ids)]
                if chunk.empty:
                    continue

                chunk["route_id"] = chunk["trip_id"].map(trip_to_route)
                chunk = chunk.dropna(subset=["route_id"])[["stop_id", "route_id"]]
                if chunk.empty:
                    continue

                stop_route_pairs.append(chunk.drop_duplicates())

    if not stop_route_pairs:
        out = station_buffers[["stop_id", "stop_name"]].drop_duplicates().copy()
        out[OUT_COL] = 0
        return out

    stop_routes_all = pd.concat(stop_route_pairs, ignore_index=True).drop_duplicates()
    stop_routes_all = stop_routes_all.rename(columns={"stop_id": "bus_stop_id"})
    _log(f"🧾 Unique (bus_stop_id, route_id) pairs (within buffers only): {len(stop_routes_all):,}")

    join_routes = join.merge(stop_routes_all, on="bus_stop_id", how="left")

    counts = (
        join_routes.dropna(subset=["route_id"])
        .groupby(["stop_id", "stop_name"])["route_id"]
        .nunique()
        .reset_index(name=OUT_COL)
    )

    return counts

# -----------------------------
# Run
# -----------------------------
bus_counts = compute_bus_routes_for_station_buffers(station_buffers, gtfs_zip_paths)

if OUT_COL in station_buffers.columns:
    station_buffers = station_buffers.drop(columns=[OUT_COL])

station_buffers = station_buffers.merge(bus_counts, on=["stop_id", "stop_name"], how="left")
station_buffers[OUT_COL] = station_buffers[OUT_COL].fillna(0).astype(int)

_log("✅ Connectivity Variable 1 complete.")
display(station_buffers[["stop_id", "stop_name", OUT_COL]].head())
print(station_buffers[OUT_COL].describe())


⬇️ Downloading GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfs_bx.zip
✅ Saved: ./gtfs_cache/gtfs_bx.zip (9.7 MB)
⬇️ Downloading GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfs_b.zip
✅ Saved: ./gtfs_cache/gtfs_b.zip (19.8 MB)
⬇️ Downloading GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfs_m.zip
✅ Saved: ./gtfs_cache/gtfs_m.zip (9.6 MB)
⬇️ Downloading GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfs_q.zip
✅ Saved: ./gtfs_cache/gtfs_q.zip (6.8 MB)
⬇️ Downloading GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfs_si.zip
✅ Saved: ./gtfs_cache/gtfs_si.zip (7.9 MB)
⬇️ Downloading GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfs_busco.zip
✅ Saved: ./gtfs_cache/gtfs_busco.zip (9.9 MB)
✅ Ready GTFS zips: 6
📦 Processing GTFS zip: ./gtfs_cache/gtfs_bx.zip
📦 Processing GTFS zip: ./gtfs_cache/gtfs_b.zip
📦 Processing GTFS zip: ./gtfs_cache/gtfs_m.zip
📦 Processing GTFS zip: ./gtfs_cache/gtfs_q.zip
📦 Processing GTFS zip: ./gtfs_cache/gtfs_si.zip
📦 Processing GTFS zip: ./gtfs_cache/gtfs_busco.zip
🧾 Unique bus

/tmp/ipython-input-2530795087.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["route_id"] = chunk["trip_id"].map(trip_to_route)


🧩 Mapping bus_stop_id->route_id (chunked): gtfs_b.zip


/tmp/ipython-input-2530795087.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["route_id"] = chunk["trip_id"].map(trip_to_route)
/tmp/ipython-input-2530795087.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["route_id"] = chunk["trip_id"].map(trip_to_route)
/tmp/ipython-input-2530795087.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

🧩 Mapping bus_stop_id->route_id (chunked): gtfs_m.zip


...

/tmp/ipython-input-2530795087.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["route_id"] = chunk["trip_id"].map(trip_to_route)
/tmp/ipython-input-2530795087.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["route_id"] = chunk["trip_id"].map(trip_to_route)


🧾 Unique (bus_stop_id, route_id) pairs (within buffers only): 13,413
✅ Connectivity Variable 1 complete.


/tmp/ipython-input-2530795087.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["route_id"] = chunk["trip_id"].map(trip_to_route)


,stop_id,stop_name,bus_routes_0p5mi
0,101,Van Cortlandt Park-242 St,11
1,103,238 St,10
2,104,231 St,11
3,106,Marble Hill-225 St,12
4,107,215 St,8


count    518.000000
mean      15.332046
std       16.619585
min        1.000000
25%        6.000000
50%        9.000000
75%       15.000000
max       81.000000
Name: bus_routes_0p5mi, dtype: float64


In [24]:
# =========================================================
# Connectivity Variable 2 (CONSOLIDATED BEST VERSION):
# Number of Connecting Subway Lines (within 0.5 mile buffer)
#
# Source: MTA Subway GTFS static feed
# Feed: https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip
# Output: station_buffers["subway_lines_0p5mi"]
#
# Counts distinct route_short_name (e.g., A, 1, F) that have >=1 PLATFORM stop
# within each station's 0.5-mile buffer.
#
# Requires:
# - station_buffers: GeoDataFrame with ["stop_id","stop_name","geometry"] in projected CRS (e.g., EPSG:2263)
# =========================================================

import os
import zipfile
import requests
import pandas as pd
import geopandas as gpd
import numpy as np

# -----------------------------
# Config
# -----------------------------
WORKDIR = "./gtfs_cache"
os.makedirs(WORKDIR, exist_ok=True)

SUBWAY_GTFS_URL = "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip"
GTFS_ZIP_PATH = os.path.join(WORKDIR, "gtfs_subway.zip")

OUT_COL = "subway_lines_0p5mi"
DEBUG = True

def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

def download_if_missing(url: str, path: str, chunk=1024 * 1024):
    if os.path.exists(path) and os.path.getsize(path) > 0:
        _log(f"✅ Using cached subway GTFS zip: {path}")
        return
    _log(f"⬇️ Downloading subway GTFS: {url}")
    r = requests.get(url, stream=True, timeout=300)
    r.raise_for_status()
    with open(path, "wb") as f:
        for part in r.iter_content(chunk_size=chunk):
            if part:
                f.write(part)
    _log(f"✅ Saved: {path} ({os.path.getsize(path)/1e6:.1f} MB)")

def _read_gtfs_csv(z: zipfile.ZipFile, name: str, usecols=None, dtype=None) -> pd.DataFrame:
    with z.open(name) as f:
        return pd.read_csv(f, usecols=usecols, dtype=dtype)

def _iter_gtfs_csv_chunks(z: zipfile.ZipFile, name: str, usecols=None, dtype=None, chunksize=800_000):
    with z.open(name) as f:
        for chunk in pd.read_csv(f, usecols=usecols, dtype=dtype, chunksize=chunksize):
            yield chunk

def _to_station_crs_points(df_stops: pd.DataFrame, station_crs) -> gpd.GeoDataFrame:
    gdf = gpd.GeoDataFrame(
        df_stops,
        geometry=gpd.points_from_xy(df_stops["stop_lon"], df_stops["stop_lat"]),
        crs="EPSG:4326"
    )
    return gdf.to_crs(station_crs)

def compute_subway_lines_for_station_buffers(station_buffers: gpd.GeoDataFrame, gtfs_zip_path: str) -> pd.DataFrame:
    # --- validate inputs ---
    for col in ["stop_id", "stop_name", "geometry"]:
        if col not in station_buffers.columns:
            raise ValueError(f"station_buffers missing required column: {col}")

    if not os.path.exists(gtfs_zip_path):
        raise FileNotFoundError(f"Subway GTFS zip not found: {gtfs_zip_path}")

    # Ensure station_buffers is projected (not strictly required, but strongly recommended)
    if station_buffers.crs is None:
        raise ValueError("station_buffers.crs is None. Please set/project station_buffers to a projected CRS (e.g., EPSG:2263).")

    with zipfile.ZipFile(gtfs_zip_path, "r") as z:
        names = set(z.namelist())
        needed = {"stops.txt", "stop_times.txt", "trips.txt", "routes.txt"}
        missing = needed - names
        if missing:
            raise RuntimeError(f"GTFS zip missing required files: {missing}")

        # 1) stops: keep PLATFORM stops (location_type blank or 0)
        stops = _read_gtfs_csv(
            z, "stops.txt",
            usecols=["stop_id", "stop_lat", "stop_lon", "location_type", "parent_station"],
            dtype={"stop_id": str, "stop_lat": float, "stop_lon": float, "location_type": str, "parent_station": str}
        )

        lt = stops["location_type"].fillna("").astype(str).str.strip()
        platform_mask = (lt == "") | (lt == "0")
        stops = stops.loc[platform_mask, ["stop_id", "stop_lat", "stop_lon"]].copy()

        # Rename to avoid collision with station stop_id
        stops = stops.rename(columns={"stop_id": "subway_stop_id"})
        stops["subway_stop_id"] = stops["subway_stop_id"].astype(str)

        stops_gdf = _to_station_crs_points(stops.rename(columns={"stop_lat":"stop_lat", "stop_lon":"stop_lon"}), station_buffers.crs)
        stops_gdf = stops_gdf[["subway_stop_id", "geometry"]]
        _log(f"🧾 Subway platform stops loaded: {len(stops_gdf):,}")

        # 2) spatial join: platform stops within station buffers
        sj = gpd.sjoin(
            stops_gdf,
            station_buffers[["stop_id", "stop_name", "geometry"]],
            how="inner",
            predicate="within"
        )

        needed_cols = {"stop_id", "stop_name", "subway_stop_id"}
        if not needed_cols.issubset(set(sj.columns)):
            _log("⚠️ sjoin columns:")
            _log(str(sorted(sj.columns)))
            raise RuntimeError(f"sjoin did not produce expected columns {needed_cols}")

        sj = sj[["stop_id", "stop_name", "subway_stop_id"]].drop_duplicates()
        _log(f"🔗 Subway platform stops within station buffers: {len(sj):,}")

        if sj.empty:
            out = station_buffers[["stop_id", "stop_name"]].drop_duplicates().copy()
            out[OUT_COL] = 0
            return out

        target_stop_ids = set(sj["subway_stop_id"].unique())
        _log(f"🎯 Target subway_stop_ids to map -> lines: {len(target_stop_ids):,}")

        # 3) routes: route_id -> route_short_name (A/1/F)
        routes = _read_gtfs_csv(
            z, "routes.txt",
            usecols=["route_id", "route_short_name"],
            dtype={"route_id": str, "route_short_name": str}
        ).dropna()

        routes["route_short_name"] = routes["route_short_name"].astype(str).str.strip()
        route_id_to_name = routes.set_index("route_id")["route_short_name"]

        # 4) trips: trip_id -> route_id
        trips = _read_gtfs_csv(
            z, "trips.txt",
            usecols=["trip_id", "route_id"],
            dtype={"trip_id": str, "route_id": str}
        ).dropna().drop_duplicates()
        trip_to_route = trips.set_index("trip_id")["route_id"]

        # 5) stop_times (chunked): stop_id -> trip_id -> route_id -> route_short_name
        stop_line_pairs = []
        _log("🧩 Mapping subway_stop_id -> line (chunked stop_times)...")

        for chunk in _iter_gtfs_csv_chunks(
            z, "stop_times.txt",
            usecols=["trip_id", "stop_id"],
            dtype={"trip_id": str, "stop_id": str},
            chunksize=800_000
        ):
            chunk = chunk.rename(columns={"stop_id": "subway_stop_id"})
            chunk = chunk[chunk["subway_stop_id"].isin(target_stop_ids)]
            if chunk.empty:
                continue

            chunk["route_id"] = chunk["trip_id"].map(trip_to_route)
            chunk = chunk.dropna(subset=["route_id"])

            chunk["line"] = chunk["route_id"].map(route_id_to_name)
            chunk = chunk.dropna(subset=["line"])

            stop_line_pairs.append(chunk[["subway_stop_id", "line"]].drop_duplicates())

        if not stop_line_pairs:
            out = station_buffers[["stop_id", "stop_name"]].drop_duplicates().copy()
            out[OUT_COL] = 0
            return out

        stop_lines = pd.concat(stop_line_pairs, ignore_index=True).drop_duplicates()
        _log(f"🧾 Unique (subway_stop_id, line) pairs (within buffers only): {len(stop_lines):,}")

    # 6) attach lines to stations and count distinct lines per station
    joined = sj.merge(stop_lines, on="subway_stop_id", how="left")

    counts = (
        joined.dropna(subset=["line"])
        .groupby(["stop_id", "stop_name"])["line"]
        .nunique()
        .reset_index(name=OUT_COL)
    )

    return counts

# -----------------------------
# Run
# -----------------------------
download_if_missing(SUBWAY_GTFS_URL, GTFS_ZIP_PATH)

subway_counts = compute_subway_lines_for_station_buffers(station_buffers, GTFS_ZIP_PATH)

# Attach overwrite-safe
if OUT_COL in station_buffers.columns:
    station_buffers = station_buffers.drop(columns=[OUT_COL])

station_buffers = station_buffers.merge(subway_counts, on=["stop_id", "stop_name"], how="left")
station_buffers[OUT_COL] = station_buffers[OUT_COL].fillna(0).astype(int)

_log("✅ Connectivity Variable 2 complete.")
display(station_buffers[["stop_id", "stop_name", OUT_COL]].tail())
print(station_buffers[OUT_COL].describe())


⬇️ Downloading subway GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip
✅ Saved: ./gtfs_cache/gtfs_subway.zip (5.6 MB)
🧾 Subway platform stops loaded: 992
🔗 Subway platform stops within station buffers: 5,450
🎯 Target subway_stop_ids to map -> lines: 992
🧩 Mapping subway_stop_id -> line (chunked stop_times)...
🧾 Unique (subway_stop_id, line) pairs (within buffers only): 2,021
✅ Connectivity Variable 2 complete.


,stop_id,stop_name,subway_lines_0p5mi
513,IBX_15,McDonald Avenue,2
514,IBX_16,New Utrecht Avenue,3
515,IBX_17,8 Avenue,3
516,IBX_18,4 Avenue,3
517,IBX_19,Brooklyn Army Terminal,0


count    518.000000
mean       5.735521
std        5.238742
min        0.000000
25%        2.000000
50%        4.000000
75%        7.000000
max       22.000000
Name: subway_lines_0p5mi, dtype: float64


In [25]:
# =========================================================
# Connectivity Variable 3: Number of Connecting Commuter Rail Lines (within 0.5 mile buffer)
# Sources: GTFS static feeds (LIRR, Metro-North, optional PATH if feed works)
# Output: station_buffers["commuter_lines_0p5mi"]
#
# UPDATED:
# - Returns BOTH (counts_df, stops_all_gdf) so you can reuse GTFS stop points for other vars (e.g., transfer flag)
# - Reads GTFS headers once per file (faster/safer than re-reading inside list comp)
# - Cleans blank line labels (drops empty strings)
#
# Requires:
# - station_buffers: GeoDataFrame with ["stop_id","stop_name","geometry"] in projected CRS (e.g., EPSG:2263)
# =========================================================

import os
import zipfile
import requests
import pandas as pd
import geopandas as gpd
import numpy as np

# -----------------------------
# Config
# -----------------------------
WORKDIR = "./gtfs_cache"
os.makedirs(WORKDIR, exist_ok=True)

# GTFS_URLS = [
#     "https://rrgtfsfeeds.s3.amazonaws.com/gtfslirr.zip",
#     "https://rrgtfsfeeds.s3.amazonaws.com/gtfsmnr.zip",
#     # Optional PATH feed (may fail depending on source availability):
#     "http://gtfs-source-feeds.transit.land/path-nj-us.zip",
# ]

GTFS_URLS = [
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfslirr.zip",
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfsmnr.zip",
    "http://gtfs-source-feeds.transit.land/path-nj-us.zip",
    "https://www.njtransit.com/sites/default/files/gtfs/rail/google_transit.zip"
]


OUT_COL = "commuter_lines_0p5mi"
DEBUG = True

def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

def download_if_missing(url: str, out_dir: str, chunk=1024*1024) -> str:
    fname = os.path.join(out_dir, os.path.basename(url.split("?")[0]) or "gtfs.zip")
    if os.path.exists(fname) and os.path.getsize(fname) > 0:
        _log(f"✅ Using cached GTFS zip: {fname}")
        return fname
    _log(f"⬇️ Downloading GTFS: {url}")
    r = requests.get(url, stream=True, timeout=300, allow_redirects=True)
    r.raise_for_status()
    with open(fname, "wb") as f:
        for part in r.iter_content(chunk_size=chunk):
            if part:
                f.write(part)
    _log(f"✅ Saved: {fname} ({os.path.getsize(fname)/1e6:.1f} MB)")
    return fname

def _read_header_cols(z: zipfile.ZipFile, name: str):
    return pd.read_csv(z.open(name), nrows=0).columns

def _read_gtfs_csv(z: zipfile.ZipFile, name: str, usecols=None, dtype=None) -> pd.DataFrame:
    with z.open(name) as f:
        return pd.read_csv(f, usecols=usecols, dtype=dtype)

def _iter_gtfs_csv_chunks(z: zipfile.ZipFile, name: str, usecols=None, dtype=None, chunksize=700_000):
    with z.open(name) as f:
        for chunk in pd.read_csv(f, usecols=usecols, dtype=dtype, chunksize=chunksize):
            yield chunk

def _to_station_crs_points(df_stops: pd.DataFrame, station_crs) -> gpd.GeoDataFrame:
    gdf = gpd.GeoDataFrame(
        df_stops,
        geometry=gpd.points_from_xy(df_stops["stop_lon"], df_stops["stop_lat"]),
        crs="EPSG:4326"
    )
    return gdf.to_crs(station_crs)

def _platform_stop_filter(stops: pd.DataFrame) -> pd.DataFrame:
    """
    Keep platform-level stops (location_type blank/0) when present.
    Some commuter rail feeds may omit location_type; in that case keep all.
    """
    if "location_type" not in stops.columns:
        return stops
    lt = stops["location_type"].fillna("").astype(str).str.strip()
    return stops[(lt == "") | (lt == "0")].copy()

def compute_commuter_lines_for_station_buffers(
    station_buffers: gpd.GeoDataFrame,
    gtfs_zip_paths: list
):
    """
    Returns:
      counts_df: DataFrame with ["stop_id","stop_name", OUT_COL]
      stops_all: GeoDataFrame with ["gtfs_stop_id","feed","geometry"] for all feeds (projected to station CRS)
    """
    for col in ["stop_id", "stop_name", "geometry"]:
        if col not in station_buffers.columns:
            raise ValueError(f"station_buffers missing required column: {col}")
    if station_buffers.crs is None:
        raise ValueError("station_buffers.crs is None (need a projected CRS).")

    if not gtfs_zip_paths:
        raise ValueError("No GTFS zips provided.")

    empty_counts = station_buffers[["stop_id","stop_name"]].drop_duplicates().copy()
    empty_counts[OUT_COL] = 0
    empty_stops = gpd.GeoDataFrame(
        {"gtfs_stop_id": [], "feed": []},
        geometry=[],
        crs=station_buffers.crs
    )

    # Collect all GTFS stops across feeds for ONE spatial join
    stops_all_list = []
    feed_meta = []  # (zip_path, route_id_to_line_series, trip_to_route_series)

    for zp in gtfs_zip_paths:
        _log(f"📦 Processing: {zp}")
        try:
            with zipfile.ZipFile(zp, "r") as z:
                names = set(z.namelist())
                needed = {"stops.txt", "trips.txt", "stop_times.txt", "routes.txt"}
                missing = needed - names
                if missing:
                    raise RuntimeError(f"Missing files: {missing}")

                # -------- stops --------
                stops_cols = _read_header_cols(z, "stops.txt")
                stop_usecols = [c for c in ["stop_id","stop_lat","stop_lon","location_type","parent_station"] if c in stops_cols]

                stops = _read_gtfs_csv(z, "stops.txt", usecols=stop_usecols, dtype=str)

                stops["stop_lat"] = pd.to_numeric(stops["stop_lat"], errors="coerce")
                stops["stop_lon"] = pd.to_numeric(stops["stop_lon"], errors="coerce")
                stops = stops.dropna(subset=["stop_lat","stop_lon"])

                stops = _platform_stop_filter(stops)
                stops = stops.rename(columns={"stop_id": "gtfs_stop_id"}).copy()
                stops["gtfs_stop_id"] = stops["gtfs_stop_id"].astype(str)

                stops_gdf = _to_station_crs_points(
                    stops[["gtfs_stop_id","stop_lat","stop_lon"]],
                    station_buffers.crs
                )
                feed_name = os.path.basename(zp)
                stops_gdf["feed"] = feed_name
                stops_all_list.append(stops_gdf[["gtfs_stop_id","feed","geometry"]])

                # -------- routes --------
                routes_cols = _read_header_cols(z, "routes.txt")
                route_usecols = [c for c in ["route_id","route_short_name","route_long_name"] if c in routes_cols]
                routes = _read_gtfs_csv(z, "routes.txt", usecols=route_usecols, dtype=str).dropna(subset=["route_id"]).drop_duplicates()

                if "route_short_name" in routes.columns:
                    routes["line"] = routes["route_short_name"].astype(str).str.strip()
                elif "route_long_name" in routes.columns:
                    routes["line"] = routes["route_long_name"].astype(str).str.strip()
                else:
                    routes["line"] = routes["route_id"].astype(str)

                # treat blanks as missing
                routes["line"] = routes["line"].replace(r"^\s*$", np.nan, regex=True)

                route_id_to_line = routes.set_index("route_id")["line"]

                # -------- trips --------
                trips = _read_gtfs_csv(
                    z, "trips.txt",
                    usecols=["trip_id","route_id"],
                    dtype=str
                ).dropna().drop_duplicates()

                trip_to_route = trips.set_index("trip_id")["route_id"]

                feed_meta.append((zp, route_id_to_line, trip_to_route))

        except Exception as e:
            _log(f"⚠️ Skipping feed (could not read): {zp} :: {type(e).__name__}: {e}")

    if not stops_all_list:
        return empty_counts, empty_stops

    stops_all = pd.concat(stops_all_list, ignore_index=True).drop_duplicates(subset=["gtfs_stop_id","feed"])
    stops_all = gpd.GeoDataFrame(stops_all, geometry="geometry", crs=station_buffers.crs)
    _log(f"🧾 GTFS platform stops loaded (all feeds): {len(stops_all):,}")

    # Spatial join: GTFS stops within station buffers
    sj = gpd.sjoin(
        stops_all,
        station_buffers[["stop_id","stop_name","geometry"]],
        how="inner",
        predicate="within"
    )

    need = {"stop_id","stop_name","gtfs_stop_id","feed"}
    if not need.issubset(set(sj.columns)):
        _log("⚠️ sjoin columns:")
        _log(str(sorted(sj.columns)))
        raise RuntimeError(f"sjoin missing expected columns: {need}")

    sj = sj[["stop_id","stop_name","gtfs_stop_id","feed"]].drop_duplicates()
    _log(f"🔗 GTFS stops within station buffers: {len(sj):,}")

    if sj.empty:
        return empty_counts, stops_all

    # Map gtfs_stop_id -> line, chunking stop_times per feed, filtered to only stops in buffers for that feed
    stop_line_pairs_all = []

    for (zp, route_id_to_line, trip_to_route) in feed_meta:
        feed_name = os.path.basename(zp)
        target_stop_ids = set(sj.loc[sj["feed"] == feed_name, "gtfs_stop_id"].unique().tolist())
        if not target_stop_ids:
            continue

        _log(f"🧩 Mapping stop_id -> line (chunked): {feed_name} | target stops: {len(target_stop_ids):,}")

        with zipfile.ZipFile(zp, "r") as z:
            for chunk in _iter_gtfs_csv_chunks(
                z, "stop_times.txt",
                usecols=["trip_id","stop_id"],
                dtype=str,
                chunksize=700_000
            ):
                chunk = chunk.rename(columns={"stop_id":"gtfs_stop_id"})
                chunk = chunk[chunk["gtfs_stop_id"].isin(target_stop_ids)]
                if chunk.empty:
                    continue

                chunk["route_id"] = chunk["trip_id"].map(trip_to_route)
                chunk = chunk.dropna(subset=["route_id"])

                chunk["line"] = chunk["route_id"].map(route_id_to_line)
                chunk = chunk.dropna(subset=["line"])

                out = chunk[["gtfs_stop_id","line"]].drop_duplicates()
                out["feed"] = feed_name
                stop_line_pairs_all.append(out)

    if not stop_line_pairs_all:
        return empty_counts, stops_all

    stop_lines = pd.concat(stop_line_pairs_all, ignore_index=True).drop_duplicates()
    _log(f"🧾 Unique (gtfs_stop_id, line, feed) pairs (within buffers only): {len(stop_lines):,}")

    # Attach lines to stations and count distinct line labels per station
    joined = sj.merge(stop_lines, on=["gtfs_stop_id","feed"], how="left")

    counts = (
        joined.dropna(subset=["line"])
        .groupby(["stop_id","stop_name"])["line"]
        .nunique()
        .reset_index(name=OUT_COL)
    )

    # Ensure every station appears
    base = station_buffers[["stop_id","stop_name"]].drop_duplicates()
    counts = base.merge(counts, on=["stop_id","stop_name"], how="left")
    counts[OUT_COL] = counts[OUT_COL].fillna(0).astype(int)

    return counts, stops_all

# -----------------------------
# Run
# -----------------------------
gtfs_zip_paths = []
for u in GTFS_URLS:
    try:
        gtfs_zip_paths.append(download_if_missing(u, WORKDIR))
    except Exception as e:
        _log(f"⚠️ Could not download {u} :: {type(e).__name__}: {e}")

commuter_counts, commuter_stops_all = compute_commuter_lines_for_station_buffers(station_buffers, gtfs_zip_paths)

# Attach overwrite-safe (counts)
if OUT_COL in station_buffers.columns:
    station_buffers = station_buffers.drop(columns=[OUT_COL])

station_buffers = station_buffers.merge(commuter_counts, on=["stop_id","stop_name"], how="left")
station_buffers[OUT_COL] = station_buffers[OUT_COL].fillna(0).astype(int)

_log("✅ Connectivity Variable 3 complete (updated).")
display(station_buffers[["stop_id","stop_name",OUT_COL]].head())
print(station_buffers[OUT_COL].describe())

# commuter_stops_all is now available for Connectivity Var 4 (transfer flag)
print("Commuter stops available for reuse:", len(commuter_stops_all))


⬇️ Downloading GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfslirr.zip
✅ Saved: ./gtfs_cache/gtfslirr.zip (1.9 MB)
⬇️ Downloading GTFS: https://rrgtfsfeeds.s3.amazonaws.com/gtfsmnr.zip
✅ Saved: ./gtfs_cache/gtfsmnr.zip (1.9 MB)
⬇️ Downloading GTFS: http://gtfs-source-feeds.transit.land/path-nj-us.zip
⚠️ Could not download http://gtfs-source-feeds.transit.land/path-nj-us.zip :: HTTPError: 401 Client Error: Unauthorized for url: https://gtfs-source-feeds.transit.land/path-nj-us.zip
⬇️ Downloading GTFS: https://www.njtransit.com/sites/default/files/gtfs/rail/google_transit.zip
⚠️ Could not download https://www.njtransit.com/sites/default/files/gtfs/rail/google_transit.zip :: HTTPError: 404 Client Error: Not Found for url: https://www.njtransit.com/sites/default/files/gtfs/rail/google_transit.zip
📦 Processing: ./gtfs_cache/gtfslirr.zip
📦 Processing: ./gtfs_cache/gtfsmnr.zip
🧾 GTFS platform stops loaded (all feeds): 241
🔗 GTFS stops within station buffers: 91
🧩 Mapping stop_id -> line (chun

,stop_id,stop_name,commuter_lines_0p5mi
0,101,Van Cortlandt Park-242 St,0
1,103,238 St,0
2,104,231 St,1
3,106,Marble Hill-225 St,1
4,107,215 St,1


count    518.000000
mean       0.953668
std        2.693530
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       11.000000
Name: commuter_lines_0p5mi, dtype: float64
Commuter stops available for reuse: 241


In [27]:
# =========================================================
# Connectivity Variable 4 (BEST): Distance to Nearest Commuter Rail / PATH Stop
# Sources: GTFS static feeds (LIRR, Metro-North, optional PATH)
# Uses: commuter_stops_all produced by Connectivity Var 3
#
# Output:
# - station_buffers["dist_to_commuter_stop_ft"]   (feet)
# - station_buffers["dist_to_commuter_stop_mi"]   (miles)
#
# Requires:
# - station_buffers: GeoDataFrame with ["stop_id","stop_name","geometry"] in projected CRS (e.g., EPSG:2263 feet)
# - commuter_stops_all: GeoDataFrame with point geometries in SAME CRS (from Var 3)
# =========================================================

import geopandas as gpd
import pandas as pd
import numpy as np

DEBUG = True
OUT_FT = "dist_to_commuter_stop_ft"
OUT_MI = "dist_to_commuter_stop_mi"
FEET_PER_MILE = 5280.0

def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

def compute_nearest_commuter_distance(
    station_buffers: gpd.GeoDataFrame,
    commuter_stops_all: gpd.GeoDataFrame
) -> pd.DataFrame:
    # --- checks
    for col in ["stop_id","stop_name","geometry"]:
        if col not in station_buffers.columns:
            raise ValueError(f"station_buffers missing required column: {col}")
    if station_buffers.crs is None:
        raise ValueError("station_buffers.crs is None (must be projected CRS).")
    if commuter_stops_all is None or len(commuter_stops_all) == 0:
        raise ValueError("commuter_stops_all is empty. Re-run Connectivity Variable 3 (updated) first.")
    if commuter_stops_all.crs is None:
        raise ValueError("commuter_stops_all.crs is None.")
    if str(commuter_stops_all.crs) != str(station_buffers.crs):
        _log(f"⚠️ CRS mismatch, projecting commuter stops to {station_buffers.crs}")
        commuter_stops_all = commuter_stops_all.to_crs(station_buffers.crs)

    # --- station centroids
    station_pts = station_buffers[["stop_id","stop_name","geometry"]].drop_duplicates().copy()
    station_pts["geometry"] = station_pts.geometry.centroid
    station_pts = gpd.GeoDataFrame(station_pts, geometry="geometry", crs=station_buffers.crs)

    # --- commuter stop geometries (dedupe)
    commuter_pts = commuter_stops_all[["geometry"]].drop_duplicates().copy()
    commuter_pts = gpd.GeoDataFrame(commuter_pts, geometry="geometry", crs=station_buffers.crs)

    _log(f"Stations: {len(station_pts):,} | Commuter/PATH stops: {len(commuter_pts):,}")

    # --- nearest distance (feet, since EPSG:2263 uses feet)
    nearest = gpd.sjoin_nearest(
        station_pts,
        commuter_pts,
        how="left",
        distance_col=OUT_FT
    )

    # Keep one row per station
    dist_df = nearest[["stop_id","stop_name",OUT_FT]].drop_duplicates()

    # Miles version
    dist_df[OUT_MI] = dist_df[OUT_FT] / FEET_PER_MILE

    return dist_df

# -----------------------------
# Run
# -----------------------------
dist_df = compute_nearest_commuter_distance(station_buffers, commuter_stops_all)

# Overwrite-safe attach
for col in [OUT_FT, OUT_MI]:
    if col in station_buffers.columns:
        station_buffers = station_buffers.drop(columns=[col])

station_buffers = station_buffers.merge(dist_df, on=["stop_id","stop_name"], how="left")

_log("✅ Connectivity Variable 4 complete (nearest commuter/PATH distance).")
display(station_buffers[["stop_id","stop_name",OUT_FT,OUT_MI]].head())
display(station_buffers[[OUT_FT,OUT_MI]].describe())


Stations: 518 | Commuter/PATH stops: 241
✅ Connectivity Variable 4 complete (nearest commuter/PATH distance).


,stop_id,stop_name,dist_to_commuter_stop_ft,dist_to_commuter_stop_mi
0,101,Van Cortlandt Park-242 St,6419.320827,1.215780
1,103,238 St,4683.159876,0.886962
2,104,231 St,2359.635421,0.446901
3,106,Marble Hill-225 St,318.017306,0.060231
4,107,215 St,2147.615832,0.406745


,dist_to_commuter_stop_ft,dist_to_commuter_stop_mi
count,518.000000,518.000000
mean,10770.464844,2.039861
std,13674.686224,2.589903
min,80.434447,0.015234
25%,3310.588116,0.627005
50%,6668.305195,1.262937
75%,12174.045239,2.305690
max,98795.892863,18.711343


In [28]:
station_buffers[["stop_id","stop_name","dist_to_commuter_stop_ft"]].sort_values("dist_to_commuter_stop_ft").head(20)

,stop_id,stop_name,dist_to_commuter_stop_ft
164,712,61 St-Woodside,80.434447
398,L24,Atlantic Av,187.954082
3,106,Marble Hill-225 St,318.017306
321,G06,Sutphin Blvd-Archer Av-JFK Airport,355.626955
145,631,Grand Central-42 St,448.928418
268,D24,Atlantic Av-Barclays Ctr,461.926007
135,621,125 St,568.469920
171,720,Hunters Point Av,592.552174
177,901,Grand Central-42 St,596.831185
70,235,Atlantic Av-Barclays Ctr,619.189632


In [30]:
import geopandas as gpd
import numpy as np

# Use your own existing subway stations subset (non-IBX)
is_ibx = station_buffers["stop_id"].astype(str).str.startswith("IBX_")
subway_stations = station_buffers.loc[~is_ibx, ["stop_id","stop_name","geometry","subway_lines_at_station"]].copy()

# If geometry in station_buffers is a 0.5-mile polygon buffer, use centroid for proximity
subway_pts = subway_stations.copy()
subway_pts["pt"] = subway_pts.geometry.centroid
subway_pts = subway_pts.set_geometry("pt")

ibx_pts = station_buffers.loc[is_ibx, ["stop_id","stop_name","geometry"]].copy()
ibx_pts["pt"] = ibx_pts.geometry.centroid
ibx_pts = ibx_pts.set_geometry("pt")

# Distance thresholds in feet if EPSG:2263 (NY State Plane feet)
r_0p25mi_ft = 0.25 * 5280
r_0p10mi_ft = 0.10 * 5280

def transfer_count_by_radius(ibx_pts, subway_pts, radius_ft, out_col):
    buffers = ibx_pts[["stop_id","stop_name","pt"]].copy()
    buffers["geometry"] = buffers["pt"].buffer(radius_ft)
    buffers = buffers.drop(columns=["pt"])
    buffers = gpd.GeoDataFrame(buffers, geometry="geometry", crs=ibx_pts.crs)

    sj = gpd.sjoin(subway_pts[["stop_id","subway_lines_at_station","pt"]].set_geometry("pt"),
                  buffers[["stop_id","geometry"]],
                  predicate="within", how="inner")
    # Here: simplest transfer measure = count of distinct subway station stop_ids within radius
    counts = sj.groupby("stop_id_right")["stop_id_left"].nunique().rename(out_col).reset_index()
    counts = counts.rename(columns={"stop_id_right":"stop_id"})
    return counts

t25 = transfer_count_by_radius(ibx_pts, subway_pts, r_0p25mi_ft, "transfer_subway_stations_0p25mi")
t10 = transfer_count_by_radius(ibx_pts, subway_pts, r_0p10mi_ft, "transfer_subway_stations_0p10mi")

station_buffers = station_buffers.merge(t25, on="stop_id", how="left").merge(t10, on="stop_id", how="left")
station_buffers[["transfer_subway_stations_0p25mi","transfer_subway_stations_0p10mi"]] = (
    station_buffers[["transfer_subway_stations_0p25mi","transfer_subway_stations_0p10mi"]].fillna(0).astype(int)
)


In [31]:
import pandas as pd
import numpy as np

# 1) Create the station-level assumption table (from the report table)
ibx_assumptions = pd.DataFrame([
    ("IBX_1","Roosevelt Avenue",1,12.0,6.0,5),
    ("IBX_2","Grand Avenue",0,12.0,6.0,0),
    ("IBX_3","Eliot Avenue",0,12.0,6.0,0),
    ("IBX_4","Metropolitan Avenue",0,12.0,6.0,1),
    ("IBX_5","Myrtle Avenue",0,12.0,6.0,0),
    ("IBX_6","Wilson Avenue",0,12.0,6.0,1),
    ("IBX_7","Atlantic Avenue",0,12.0,6.0,5),
    ("IBX_8","Sutter Avenue",0,12.0,6.0,1),
    ("IBX_9","Livonia Avenue",0,12.0,6.0,2),
    ("IBX_10","Linden Blvd",0,12.0,6.0,0),
    ("IBX_11","Remsen Avenue",0,12.0,6.0,0),
    ("IBX_12","Utica Avenue",0,12.0,6.0,0),
    ("IBX_13","Flatbush–Nostrand Av",0,12.0,6.0,4),
    ("IBX_14","East 16 Street",0,12.0,6.0,2),
    ("IBX_15","McDonald Avenue",0,12.0,6.0,1),
    ("IBX_16","New Utrecht Avenue",0,12.0,6.0,2),
    ("IBX_17","8 Avenue",0,12.0,6.0,1),
    ("IBX_18","4 Avenue",0,12.0,6.0,1),
    ("IBX_19","Brooklyn Army Terminal",1,12.0,6.0,0),
], columns=[
    "stop_id","stop_name","is_terminal",
    "peak_trains_per_hour","midday_trains_per_hour",
    "planned_transfer_count"
])

# 2) Optional: define a single-column "subway_lines_at_station" for IBX as 1 + transfers
ibx_assumptions["subway_lines_at_station"] = 1 + ibx_assumptions["planned_transfer_count"]

# 3) Merge into station_buffers (preserve any already-computed subway station values)
svc_cols = ["is_terminal","peak_trains_per_hour","midday_trains_per_hour",
            "planned_transfer_count","subway_lines_at_station"]

station_buffers = station_buffers.merge(
    ibx_assumptions[["stop_id"] + svc_cols],
    on="stop_id",
    how="left",
    suffixes=("","_ibx")
)

# 4) If you want these fields to apply ONLY to IBX and not overwrite subways:
is_ibx = station_buffers["stop_id"].astype(str).str.startswith("IBX_")
for c in svc_cols:
    # keep existing subway values; only fill for IBX
    station_buffers.loc[is_ibx, c] = station_buffers.loc[is_ibx, c].fillna(station_buffers.loc[is_ibx, f"{c}_ibx"])

# 5) Cleanup helper columns
drop_cols = [f"{c}_ibx" for c in svc_cols if f"{c}_ibx" in station_buffers.columns]
station_buffers = station_buffers.drop(columns=drop_cols)


In [51]:
# =========================================================
# COMBINED TRANSIT SERVICE VARIABLES (Subway GTFS + IBX)
#
# Computes, in ONE pass:
# 1) peak_trains_per_hour   (unique trip_ids stopping at station complex during 08:00–08:59)
# 2) midday_trains_per_hour (unique trip_ids stopping at station complex during 13:00–13:59)
# 3) subway_lines_serving_station (distinct route_short_name serving station complex on a weekday service day)
# 4) is_terminal_subway (station complex is a primary terminal for at least one route; inferred from trip endpoints)
#
# Then merges into station_buffers, and fills IBX assumptions:
# - peak_trains_per_hour: default 12.0 if missing
# - midday_trains_per_hour: default 6.0 if missing
# - subway_lines_serving_station: 1 + planned_transfer_count (if available) else 1
# - is_terminal_subway: from station_buffers["is_terminal"] if present, else 0
#
# Requires:
# - station_buffers: GeoDataFrame/DataFrame with at least ["stop_id","stop_name", ...]
# - A Subway GTFS zip (downloaded below with fallbacks)
# =========================================================

import os
import zipfile
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# -----------------------------
# Config
# -----------------------------
WORKDIR = "./gtfs_cache"
os.makedirs(WORKDIR, exist_ok=True)

GTFS_SUBWAY_URLS = [
    "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip",
    "http://web.mta.info/developers/data/nyct/subway/google_transit.zip",
]
GTFS_ZIP_PATH = os.path.join(WORKDIR, "gtfs_subway.zip")

PEAK_HOUR = 8       # 08:00–08:59
MIDDAY_HOUR = 13    # 13:00–13:59 (1–2 PM)

DEFAULT_IBX_PEAK_TPH = 12.0
DEFAULT_IBX_MIDDAY_TPH = 6.0

DEBUG = True
def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

# -----------------------------
# Helpers
# -----------------------------
def download_first_working(urls, out_path, chunk=1024*1024):
    if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
        _log(f"✅ Using cached GTFS zip: {out_path}")
        return out_path

    last_err = None
    for url in urls:
        try:
            _log(f"⬇️ Downloading subway GTFS: {url}")
            r = requests.get(url, stream=True, timeout=300, allow_redirects=True)
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for part in r.iter_content(chunk_size=chunk):
                    if part:
                        f.write(part)
            _log(f"✅ Saved: {out_path} ({os.path.getsize(out_path)/1e6:.1f} MB)")
            return out_path
        except Exception as e:
            last_err = e
            _log(f"⚠️ Failed: {url} :: {type(e).__name__}: {e}")

    raise RuntimeError(f"Could not download any subway GTFS feed. Last error: {last_err}")

def _iter_gtfs_csv_chunks(z: zipfile.ZipFile, name: str, usecols=None, dtype=None, chunksize=900_000):
    with z.open(name) as f:
        for chunk in pd.read_csv(f, usecols=usecols, dtype=dtype, chunksize=chunksize):
            yield chunk

def _yyyymmdd(dt: datetime) -> int:
    return int(dt.strftime("%Y%m%d"))

def pick_weekday_date(calendar_df: pd.DataFrame) -> int:
    """
    Pick a reasonable weekday date within the calendar range.
    We choose the midpoint of the overlapping weekday-valid range, snapped to Monday.
    """
    c = calendar_df.copy()
    c["start_date"] = c["start_date"].astype(int)
    c["end_date"] = c["end_date"].astype(int)

    wk = c[
        (c["monday"]=="1") & (c["tuesday"]=="1") & (c["wednesday"]=="1") &
        (c["thursday"]=="1") & (c["friday"]=="1")
    ]
    if wk.empty:
        wk = c

    start = wk["start_date"].max()
    end   = wk["end_date"].min()
    if start > end:
        start = c["start_date"].min()
        end   = c["end_date"].max()

    start_dt = datetime.strptime(str(start), "%Y%m%d")
    end_dt   = datetime.strptime(str(end), "%Y%m%d")
    mid_dt   = start_dt + (end_dt - start_dt)/2

    while mid_dt.weekday() != 0:  # Monday
        mid_dt += timedelta(days=1)

    return _yyyymmdd(mid_dt)

def service_ids_active_on_date(calendar_df, calendar_dates_df, target_date_yyyymmdd: int) -> set:
    d = str(target_date_yyyymmdd)

    base = calendar_df[
        (calendar_df["start_date"].astype(int) <= target_date_yyyymmdd) &
        (calendar_df["end_date"].astype(int) >= target_date_yyyymmdd)
    ].copy()

    target_dt = datetime.strptime(d, "%Y%m%d")
    dow = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"][target_dt.weekday()]
    if dow in base.columns:
        base = base[base[dow].astype(str) == "1"]

    active = set(base["service_id"].astype(str).unique().tolist())

    if calendar_dates_df is not None and not calendar_dates_df.empty:
        cd = calendar_dates_df[calendar_dates_df["date"].astype(int) == target_date_yyyymmdd].copy()
        if not cd.empty:
            adds = set(cd.loc[cd["exception_type"].astype(str)=="1", "service_id"].astype(str))
            rems = set(cd.loc[cd["exception_type"].astype(str)=="2", "service_id"].astype(str))
            active |= adds
            active -= rems

    return active

def hour_from_time(t: str):
    # GTFS may have >24 (e.g., 25:10:00); that's fine.
    try:
        return int(str(t).split(":")[0])
    except Exception:
        return None

# -----------------------------
# Core combined computation
# -----------------------------
def compute_subway_service_variables(
    gtfs_zip_path: str,
    peak_hour: int = 8,
    midday_hour: int = 13,
    terminal_mode_top_k: int = 1
) -> pd.DataFrame:
    """
    Returns DataFrame keyed by parent_station with:
      - peak_trains_per_hour
      - midday_trains_per_hour
      - subway_lines_serving_station
      - is_terminal_subway
    """

    with zipfile.ZipFile(gtfs_zip_path, "r") as z:
        needed = {"stops.txt","stop_times.txt","trips.txt","routes.txt","calendar.txt"}
        missing = needed - set(z.namelist())
        if missing:
            raise RuntimeError(f"GTFS zip missing required files: {missing}")

        # calendar
        calendar = pd.read_csv(z.open("calendar.txt"), dtype=str)
        cal_dates = None
        if "calendar_dates.txt" in z.namelist():
            cal_dates = pd.read_csv(z.open("calendar_dates.txt"), dtype=str)

        target_date = pick_weekday_date(calendar)
        _log(f"📅 Using representative weekday date: {target_date}")

        active_services = service_ids_active_on_date(calendar, cal_dates, target_date)
        _log(f"✅ Active service_ids on that date: {len(active_services):,}")

        # trips: trip_id -> route_id (filtered to active services)
        trips = pd.read_csv(z.open("trips.txt"), dtype=str, usecols=["trip_id","service_id","route_id"])
        trips = trips[trips["service_id"].isin(active_services)].dropna(subset=["trip_id","route_id"])
        _log(f"✅ Trips active on that date: {len(trips):,}")

        trip_to_route = trips.set_index("trip_id")["route_id"]
        active_trip_ids = set(trips["trip_id"].astype(str).unique())

        # routes: route_id -> route_short_name (line label)
        routes = pd.read_csv(z.open("routes.txt"), dtype=str, usecols=lambda c: c in {"route_id","route_short_name","route_long_name"})
        if "route_short_name" in routes.columns:
            routes["line"] = routes["route_short_name"].astype(str).str.strip()
        elif "route_long_name" in routes.columns:
            routes["line"] = routes["route_long_name"].astype(str).str.strip()
        else:
            routes["line"] = routes["route_id"].astype(str)

        routes["line"] = routes["line"].replace(r"^\s*$", np.nan, regex=True)
        route_to_line = routes.set_index("route_id")["line"]

        # stops: stop_id -> parent_station
        stops = pd.read_csv(z.open("stops.txt"), dtype=str, usecols=lambda c: c in {"stop_id","parent_station"})
        stops["parent_station"] = stops["parent_station"].fillna(stops["stop_id"])
        stop_to_parent = stops.set_index("stop_id")["parent_station"]

        # accumulators
        peak_tripsets = {}     # parent_station -> set(trip_id)
        midday_tripsets = {}   # parent_station -> set(trip_id)
        line_sets = {}         # parent_station -> set(line)
        trip_endpoints = {}    # trip_id -> (min_seq, min_stop_id, max_seq, max_stop_id)

        hours_needed = {peak_hour, midday_hour}

        # stream stop_times
        for chunk in _iter_gtfs_csv_chunks(
            z, "stop_times.txt",
            usecols=["trip_id","stop_id","arrival_time","stop_sequence"],
            dtype={"trip_id": str, "stop_id": str, "arrival_time": str, "stop_sequence": str},
            chunksize=900_000
        ):
            # keep only active trips
            chunk = chunk[chunk["trip_id"].isin(active_trip_ids)]
            if chunk.empty:
                continue

            # terminal endpoints tracking (needs stop_sequence numeric)
            seq = pd.to_numeric(chunk["stop_sequence"], errors="coerce")
            chunk = chunk.assign(stop_sequence_num=seq)
            chunk = chunk.dropna(subset=["stop_sequence_num"])

            # update trip endpoints
            # (loop is OK here because active trips are ~few thousands; chunk is large but filtered)
            for t_id, grp in chunk.groupby("trip_id"):
                mn = grp["stop_sequence_num"].min()
                mx = grp["stop_sequence_num"].max()
                # stop_id at min/max
                mn_stop = grp.loc[grp["stop_sequence_num"].idxmin(), "stop_id"]
                mx_stop = grp.loc[grp["stop_sequence_num"].idxmax(), "stop_id"]

                if t_id not in trip_endpoints:
                    trip_endpoints[t_id] = (mn, mn_stop, mx, mx_stop)
                else:
                    prev_mn, prev_mn_stop, prev_mx, prev_mx_stop = trip_endpoints[t_id]
                    if mn < prev_mn:
                        prev_mn, prev_mn_stop = mn, mn_stop
                    if mx > prev_mx:
                        prev_mx, prev_mx_stop = mx, mx_stop
                    trip_endpoints[t_id] = (prev_mn, prev_mn_stop, prev_mx, prev_mx_stop)

            # map to parent_station + line
            chunk["parent_station"] = chunk["stop_id"].map(stop_to_parent)
            chunk["route_id"] = chunk["trip_id"].map(trip_to_route)
            chunk["line"] = chunk["route_id"].map(route_to_line)

            # update line sets (weekday service day)
            sub = chunk.dropna(subset=["parent_station","line"])
            for ps, grp in sub.groupby("parent_station")["line"]:
                if ps not in line_sets:
                    line_sets[ps] = set()
                line_sets[ps].update(grp.dropna().unique().tolist())

            # compute hour for peak/midday
            hrs = chunk["arrival_time"].map(hour_from_time)
            chunk = chunk.assign(hour=hrs)
            chunk = chunk[chunk["hour"].isin(hours_needed)]
            if chunk.empty:
                continue

            chunk = chunk.dropna(subset=["parent_station","trip_id","hour"])

            # update tripsets per target hour
            peak = chunk[chunk["hour"] == peak_hour]
            if not peak.empty:
                for ps, grp in peak.groupby("parent_station")["trip_id"]:
                    peak_tripsets.setdefault(ps, set()).update(grp.unique().tolist())

            mid = chunk[chunk["hour"] == midday_hour]
            if not mid.empty:
                for ps, grp in mid.groupby("parent_station")["trip_id"]:
                    midday_tripsets.setdefault(ps, set()).update(grp.unique().tolist())

        # Build endpoints DF: infer primary terminals per route (mode endpoints)
        # Create per-trip endpoints with route_id
        ep_rows = []
        for t_id, (mn, mn_stop, mx, mx_stop) in trip_endpoints.items():
            r_id = trip_to_route.get(t_id)
            if pd.isna(r_id):
                continue
            start_ps = stop_to_parent.get(mn_stop, mn_stop)
            end_ps = stop_to_parent.get(mx_stop, mx_stop)
            ep_rows.append((r_id, start_ps, end_ps))

        ep = pd.DataFrame(ep_rows, columns=["route_id","start_parent","end_parent"])
        # If endpoints empty, no terminals can be inferred
        terminal_parents = set()

        if not ep.empty:
            # For each route, pick the most common start and end parent station as "primary terminals"
            for r_id, g in ep.groupby("route_id"):
                start_mode = g["start_parent"].value_counts().head(terminal_mode_top_k).index.tolist()
                end_mode = g["end_parent"].value_counts().head(terminal_mode_top_k).index.tolist()
                terminal_parents.update(start_mode)
                terminal_parents.update(end_mode)

        # Assemble output keyed by parent_station
        parents = set()
        parents |= set(line_sets.keys())
        parents |= set(peak_tripsets.keys())
        parents |= set(midday_tripsets.keys())
        parents |= terminal_parents

        out = pd.DataFrame({"parent_station": sorted(parents)})

        out["peak_trains_per_hour"] = out["parent_station"].map(lambda ps: len(peak_tripsets.get(ps, set()))).astype(float)
        out["midday_trains_per_hour"] = out["parent_station"].map(lambda ps: len(midday_tripsets.get(ps, set()))).astype(float)
        out["subway_lines_serving_station"] = out["parent_station"].map(lambda ps: len(line_sets.get(ps, set()))).fillna(0).astype(int)
        out["is_terminal_subway"] = out["parent_station"].isin(terminal_parents).astype(int)

        return out

# -----------------------------
# RUN + MERGE INTO station_buffers
# -----------------------------
gtfs_zip_path = download_first_working(GTFS_SUBWAY_URLS, GTFS_ZIP_PATH)

svc = compute_subway_service_variables(
    gtfs_zip_path,
    peak_hour=PEAK_HOUR,
    midday_hour=MIDDAY_HOUR,
    terminal_mode_top_k=1  # 1 = most common endpoints only (reduces short-turn noise)
)

# Merge once
cols_to_drop = ["peak_trains_per_hour","midday_trains_per_hour","subway_lines_serving_station","is_terminal_subway"]
station_buffers = station_buffers.drop(columns=cols_to_drop, errors="ignore")

station_buffers = station_buffers.merge(
    svc,
    left_on="stop_id",
    right_on="parent_station",
    how="left"
).drop(columns=["parent_station"], errors="ignore")

# Fill subway (non-IBX) missing values
station_buffers["peak_trains_per_hour"] = station_buffers["peak_trains_per_hour"].fillna(0).astype(float)
station_buffers["midday_trains_per_hour"] = station_buffers["midday_trains_per_hour"].fillna(0).astype(float)
station_buffers["subway_lines_serving_station"] = station_buffers["subway_lines_serving_station"].fillna(0).astype(int)
station_buffers["is_terminal_subway"] = station_buffers["is_terminal_subway"].fillna(0).astype(int)

# -----------------------------
# IBX FILL
# -----------------------------
# --- IBX defaults should match "both directions combined" convention ---
DEFAULT_IBX_PEAK_TPH = 24.0    # 12 per direction * 2
DEFAULT_IBX_MIDDAY_TPH = 12.0  # 6 per direction * 2

is_ibx = station_buffers["stop_id"].astype(str).str.startswith("IBX_")

# overwrite IBX values (don’t depend on GTFS for IBX)
station_buffers.loc[is_ibx, "peak_trains_per_hour"] = DEFAULT_IBX_PEAK_TPH
station_buffers.loc[is_ibx, "midday_trains_per_hour"] = DEFAULT_IBX_MIDDAY_TPH

# if you want to keep terminals from your planning field (already correct)
if "is_terminal" in station_buffers.columns:
    station_buffers.loc[is_ibx, "is_terminal_subway"] = (
        station_buffers.loc[is_ibx, "is_terminal"].fillna(0).astype(int)
    )

display(
    station_buffers.loc[is_ibx, ["stop_id","stop_name","peak_trains_per_hour","midday_trains_per_hour",
                                 "subway_lines_serving_station","is_terminal_subway"]]
)


_log("✅ Combined service variables merged into station_buffers.")
display(station_buffers[["stop_id","stop_name","peak_trains_per_hour","midday_trains_per_hour",
                        "subway_lines_serving_station","is_terminal_subway"]].tail(19))
print(station_buffers[["peak_trains_per_hour","midday_trains_per_hour",
                       "subway_lines_serving_station","is_terminal_subway"]].describe())


✅ Using cached GTFS zip: ./gtfs_cache/gtfs_subway.zip
📅 Using representative weekday date: 20260302
✅ Active service_ids on that date: 1
✅ Trips active on that date: 8,492


,stop_id,stop_name,peak_trains_per_hour,midday_trains_per_hour,subway_lines_serving_station,is_terminal_subway
499,IBX_1,Roosevelt Avenue,24.0,12.0,0,1
500,IBX_2,Grand Avenue,24.0,12.0,0,0
501,IBX_3,Eliot Avenue,24.0,12.0,0,0
502,IBX_4,Metropolitan Avenue,24.0,12.0,0,0
503,IBX_5,Myrtle Avenue,24.0,12.0,0,0
504,IBX_6,Wilson Avenue,24.0,12.0,0,0
505,IBX_7,Atlantic Avenue,24.0,12.0,0,0
506,IBX_8,Sutter Avenue,24.0,12.0,0,0
507,IBX_9,Livonia Avenue,24.0,12.0,0,0
508,IBX_10,Linden Blvd,24.0,12.0,0,0


✅ Combined service variables merged into station_buffers.


,stop_id,stop_name,peak_trains_per_hour,midday_trains_per_hour,subway_lines_serving_station,is_terminal_subway
499,IBX_1,Roosevelt Avenue,24.0,12.0,0,1
500,IBX_2,Grand Avenue,24.0,12.0,0,0
501,IBX_3,Eliot Avenue,24.0,12.0,0,0
502,IBX_4,Metropolitan Avenue,24.0,12.0,0,0
503,IBX_5,Myrtle Avenue,24.0,12.0,0,0
504,IBX_6,Wilson Avenue,24.0,12.0,0,0
505,IBX_7,Atlantic Avenue,24.0,12.0,0,0
506,IBX_8,Sutter Avenue,24.0,12.0,0,0
507,IBX_9,Livonia Avenue,24.0,12.0,0,0
508,IBX_10,Linden Blvd,24.0,12.0,0,0


       peak_trains_per_hour  midday_trains_per_hour  \
count            518.000000              518.000000   
mean              29.075290               22.862934   
std               16.896091               12.278464   
min                0.000000                0.000000   
25%               17.000000               15.000000   
50%               25.000000               20.000000   
75%               37.000000               30.000000   
max               85.000000               65.000000   

       subway_lines_serving_station  is_terminal_subway  
count                    518.000000          518.000000  
mean                       1.976834            0.067568  
std                        1.105396            0.251245  
min                        0.000000            0.000000  
25%                        1.000000            0.000000  
50%                        2.000000            0.000000  
75%                        3.000000            0.000000  
max                        6.000000     

In [61]:
# =========================================================
# ANCHOR VARIABLES (PARAMETERIZED RADII + BEST-PRACTICE)
#
# Adds RAW + CAPPED + BINARY for Cultural at both radii:
#   - cultural_count_0p25mi, cultural_count_0p5mi                (raw)
#   - cultural_count_0p25mi_capped10, cultural_count_0p5mi_capped10
#   - has_cultural_0p25mi, has_cultural_0p5mi                    (binary)
#
# Existing outputs retained:
#   Universities:
#     has_university_0p25mi, has_university_0p5mi
#   Hospitals (tight definition):
#     has_hospital_0p25mi, has_hospital_0p5mi
#   Stadiums/Arenas (tight):
#     has_stadium_0p1mi
#   Parks (big parks adjacency flags):
#     adjacent_major_park_0p25mi, adjacent_major_park_0p5mi
#
# Notes:
# - Uses station centroids for point-based buffers.
# - Facilities dataset sometimes lacks geometry; we build points from lat/lon if needed.
#
# Requires:
# - station_buffers: GeoDataFrame with ["stop_id","stop_name","geometry"] in projected CRS (EPSG:2263 recommended)
# =========================================================

import geopandas as gpd
import pandas as pd
import numpy as np
import requests
from shapely.geometry import shape

DEBUG = True
def _log(msg: str):
    if DEBUG:
        print(msg, flush=True)

# -----------------------------
# Validate station_buffers
# -----------------------------
req = {"stop_id", "stop_name", "geometry"}
missing = req - set(station_buffers.columns)
if missing:
    raise ValueError(f"station_buffers missing required columns: {missing}")
if station_buffers.crs is None:
    raise ValueError("station_buffers.crs is None. Please project station_buffers to a projected CRS (e.g., EPSG:2263).")

# -----------------------------
# Config: radii
# -----------------------------
FEET_PER_MILE = 5280.0
R_UNIV = [0.25, 0.50]
R_HOSP = [0.25, 0.50]
R_CULT = [0.25, 0.50]
R_PARK = [0.25, 0.50]
R_STAD = 0.10

# Cultural groups to include (tight)
CULT_GROUPS = {
    "CULTURAL INSTITUTIONS",
    "HISTORICAL SITES",
    # "LIBRARIES",  # uncomment if you want libraries included
}

# Cultural refinements
CULT_CAP = 10  # cap counts at 10 (Winsorize)

# -----------------------------
# Socrata paging helpers
# -----------------------------
def socrata_paged(url, limit=50000, where=None, select=None, timeout=120):
    offset = 0
    rows = []
    while True:
        params = {"$limit": limit, "$offset": offset}
        if where:  params["$where"] = where
        if select: params["$select"] = select

        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        js = r.json()

        if isinstance(js, dict) and "features" in js:
            feats = js.get("features", [])
            if not feats:
                break
            rows.extend(feats)
            _log(f"⬇️ Loaded {len(feats):,} features (offset={offset:,})")
            if len(feats) < limit:
                break
        elif isinstance(js, list):
            if not js:
                break
            rows.extend(js)
            _log(f"⬇️ Loaded {len(js):,} rows (offset={offset:,})")
            if len(js) < limit:
                break
        else:
            raise ValueError("Unexpected Socrata response format.")

        offset += limit

    return rows

def facilities_to_gdf(features):
    props = []
    geoms = []
    for f in features:
        p = f.get("properties", {}) if isinstance(f, dict) else {}
        g = f.get("geometry", None) if isinstance(f, dict) else None
        props.append(p)
        geoms.append(shape(g) if g else None)

    df = pd.DataFrame(props)
    gdf = gpd.GeoDataFrame(df, geometry=geoms, crs="EPSG:4326")

    # If geometry is missing, build points from lon/lat
    if gdf.geometry.isna().all():
        lon_col = next((c for c in gdf.columns if c.lower() == "longitude"), None)
        lat_col = next((c for c in gdf.columns if c.lower() == "latitude"), None)
        if lon_col and lat_col:
            gdf[lon_col] = pd.to_numeric(gdf[lon_col], errors="coerce")
            gdf[lat_col] = pd.to_numeric(gdf[lat_col], errors="coerce")
            gdf = gdf.dropna(subset=[lon_col, lat_col]).copy()
            gdf = gpd.GeoDataFrame(
                gdf.drop(columns=["geometry"], errors="ignore"),
                geometry=gpd.points_from_xy(gdf[lon_col], gdf[lat_col]),
                crs="EPSG:4326"
            )
        else:
            raise ValueError("Facilities has no geometry and no longitude/latitude to build points.")

    gdf = gdf.dropna(subset=["geometry"]).copy()
    return gdf

def parks_to_gdf(features):
    props = []
    geoms = []
    for f in features:
        p = f.get("properties", {}) if isinstance(f, dict) else {}
        g = f.get("geometry", None) if isinstance(f, dict) else None
        props.append(p)
        geoms.append(shape(g) if g else None)
    parks = gpd.GeoDataFrame(pd.DataFrame(props), geometry=geoms, crs="EPSG:4326")
    parks = parks.dropna(subset=["geometry"]).copy()
    return parks

def norm_upper(s):
    return s.astype(str).str.strip().str.upper()

# -----------------------------
# Station centroids (points)
# -----------------------------
station_pts = station_buffers[["stop_id","stop_name","geometry"]].drop_duplicates().copy()
station_pts["geometry"] = station_pts.geometry.centroid
station_pts = gpd.GeoDataFrame(station_pts, geometry="geometry", crs=station_buffers.crs)

# -----------------------------
# Load Facilities (NYC Open Data)
# -----------------------------
FAC_URL = "https://data.cityofnewyork.us/resource/ji82-xba5.geojson"
_log("Loading DCP Facilities Database (paged)…")
fac_feats = socrata_paged(FAC_URL, limit=50000)
fac = facilities_to_gdf(fac_feats)
_log(f"✅ Facilities loaded with geometry: {len(fac):,}")

# Project to station CRS
fac = fac.to_crs(station_buffers.crs)

# Standardize key fields
for c in ["facgroup", "facsubgrp", "facname"]:
    if c not in fac.columns:
        raise ValueError(f"Expected column '{c}' not found in facilities. Columns: {list(fac.columns)[:50]}")
fac["FACGROUP_U"]  = norm_upper(fac["facgroup"])
fac["FACSUBGRP_U"] = norm_upper(fac["facsubgrp"])
fac["FACNAME_L"]   = fac["facname"].astype(str).str.strip().str.lower()

# -----------------------------
# Helper: compute 0/1 flag within radius using station point buffers
# -----------------------------
def flag_within_radius(points_gdf, station_points, radii_miles, prefix):
    out = station_buffers
    for r in radii_miles:
        out_col = f"{prefix}_{str(r).replace('.','p')}mi"
        radius_ft = r * FEET_PER_MILE

        buf = station_points[["stop_id","geometry"]].copy()
        buf["geometry"] = buf.geometry.buffer(radius_ft)
        buf = gpd.GeoDataFrame(buf, geometry="geometry", crs=station_points.crs)

        sj = gpd.sjoin(
            points_gdf[["geometry"]],
            buf[["stop_id","geometry"]],
            how="inner",
            predicate="within"
        )
        hit_ids = set(sj["stop_id"].astype(str).unique())
        out = out.drop(columns=[out_col], errors="ignore")
        out[out_col] = out["stop_id"].astype(str).isin(hit_ids).astype(int)

        _log(f"✅ {out_col}: flagged={out[out_col].sum()} (radius={r}mi)")
    return out

# -----------------------------
# Helper: compute COUNT within radius using station point buffers
# -----------------------------
def count_within_radius(points_gdf, station_points, radii_miles, prefix):
    out = station_buffers
    for r in radii_miles:
        out_col = f"{prefix}_{str(r).replace('.','p')}mi"
        radius_ft = r * FEET_PER_MILE

        buf = station_points[["stop_id","geometry"]].copy()
        buf["geometry"] = buf.geometry.buffer(radius_ft)
        buf = gpd.GeoDataFrame(buf, geometry="geometry", crs=station_points.crs)

        sj = gpd.sjoin(
            points_gdf[["geometry"]],
            buf[["stop_id","geometry"]],
            how="inner",
            predicate="within"
        )
        counts = sj.groupby("stop_id").size()
        out = out.drop(columns=[out_col], errors="ignore")
        out[out_col] = out["stop_id"].map(counts).fillna(0).astype(int)

        _log(f"✅ {out_col}: mean={out[out_col].mean():.2f}, max={out[out_col].max()} (radius={r}mi)")
    return out

# =========================================================
# 1) UNIVERSITY / COLLEGE — structured filter
# =========================================================
univ = fac[
    (fac["FACGROUP_U"] == "HIGHER EDUCATION") &
    (fac["FACSUBGRP_U"] == "COLLEGES OR UNIVERSITIES")
].copy()

univ_pts = univ.copy()
univ_pts["geometry"] = univ_pts.geometry.centroid

station_buffers = flag_within_radius(univ_pts, station_pts, R_UNIV, "has_university")

# =========================================================
# 2) HOSPITAL — tighter: structured + name contains 'hospital'
# =========================================================
hosp = fac[
    (fac["FACGROUP_U"] == "HEALTH CARE") &
    (fac["FACSUBGRP_U"] == "HOSPITALS AND CLINICS") &
    (fac["FACNAME_L"].str.contains("hospital", na=False))
].copy()

hosp_pts = hosp.copy()
hosp_pts["geometry"] = hosp_pts.geometry.centroid

station_buffers = flag_within_radius(hosp_pts, station_pts, R_HOSP, "has_hospital")

# =========================================================
# 3) STADIUM / ARENA — manual list + 0.10mi radius (tight)
# =========================================================
major_venues = pd.DataFrame([
    ("Yankee Stadium", 40.829643, -73.926175),
    ("Citi Field", 40.757088, -73.845821),
    ("Madison Square Garden", 40.750504, -73.993439),
    ("Barclays Center", 40.682650, -73.975370),
    ("USTA Billie Jean King / Arthur Ashe", 40.749824, -73.847147),
], columns=["venue", "lat", "lon"])

venues_gdf = gpd.GeoDataFrame(
    major_venues,
    geometry=gpd.points_from_xy(major_venues["lon"], major_venues["lat"]),
    crs="EPSG:4326"
).to_crs(station_buffers.crs)

# Create has_stadium_0p1mi
station_buffers = flag_within_radius(venues_gdf, station_pts, [R_STAD], "has_stadium")

# =========================================================
# 4) CULTURAL — structured groups, COUNT at two radii (RAW)
#     Then add:
#       - BINARY presence
#       - CAPPED count (upper=CULT_CAP)
# =========================================================
cult = fac[fac["FACGROUP_U"].isin(CULT_GROUPS)].copy()
cult_pts = cult.copy()
cult_pts["geometry"] = cult_pts.geometry.centroid

# Raw counts
station_buffers = count_within_radius(cult_pts, station_pts, R_CULT, "cultural_count")

# Add binary + capped for each radius
for r in R_CULT:
    base = f"{str(r).replace('.','p')}mi"
    raw_col   = f"cultural_count_{base}"
    bin_col   = f"has_cultural_{base}"
    cap_col   = f"cultural_count_{base}_capped{CULT_CAP}"

    station_buffers = station_buffers.drop(columns=[bin_col, cap_col], errors="ignore")
    station_buffers[bin_col] = (station_buffers[raw_col] > 0).astype(int)
    station_buffers[cap_col] = station_buffers[raw_col].clip(upper=CULT_CAP).astype(int)

    _log(f"✅ {bin_col} + {cap_col} created (raw={raw_col})")

# =========================================================
# 5) MAJOR PARK adjacency (>=50 acres) — parks polygons
# =========================================================
PARKS_URL = "https://data.cityofnewyork.us/resource/enfh-gkve.geojson"
_log("Loading NYC Parks Properties (paged)…")
park_feats = socrata_paged(PARKS_URL, limit=50000)
parks = parks_to_gdf(park_feats)
_log(f"✅ Parks loaded with geometry: {len(parks):,}")

parks = parks.to_crs(station_buffers.crs)

# Big parks
ACRE_TO_SQFT = 43560.0
parks["area_sqft"] = parks.geometry.area
parks_big = parks[parks["area_sqft"] >= 50 * ACRE_TO_SQFT].copy()
_log(f"✅ Major parks (>=50 acres): {len(parks_big):,}")

# adjacency flag for each radius: station point buffer intersects park polygons
for r in R_PARK:
    out_col = f"adjacent_major_park_{str(r).replace('.','p')}mi"
    radius_ft = r * FEET_PER_MILE

    buf = station_pts[["stop_id","geometry"]].copy()
    buf["geometry"] = buf.geometry.buffer(radius_ft)
    buf = gpd.GeoDataFrame(buf, geometry="geometry", crs=station_pts.crs)

    sj = gpd.sjoin(
        buf[["stop_id","geometry"]],
        parks_big[["geometry"]],
        how="inner",
        predicate="intersects"
    )
    hit_ids = set(sj["stop_id"].astype(str).unique())
    station_buffers = station_buffers.drop(columns=[out_col], errors="ignore")
    station_buffers[out_col] = station_buffers["stop_id"].astype(str).isin(hit_ids).astype(int)

    _log(f"✅ {out_col}: flagged={station_buffers[out_col].sum()} (radius={r}mi)")

# -----------------------------
# Cleanup helper cols (optional)
# -----------------------------
station_buffers = station_buffers.drop(columns=["FACGROUP_U","FACSUBGRP_U","FACNAME_L"], errors="ignore")

# -----------------------------
# Preview (choose which columns you want in the final model)
# -----------------------------
cols_preview = [
    "stop_id","stop_name",
    "has_university_0p25mi","has_university_0p5mi",
    "has_hospital_0p25mi","has_hospital_0p5mi",
    "has_stadium_0p1mi",
    "cultural_count_0p25mi","cultural_count_0p25mi_capped10","has_cultural_0p25mi",
    "cultural_count_0p5mi","cultural_count_0p5mi_capped10","has_cultural_0p5mi",
    "adjacent_major_park_0p25mi","adjacent_major_park_0p5mi",
]
existing = [c for c in cols_preview if c in station_buffers.columns]

_log("✅ Anchor variables (parameterized radii + cultural raw/cap/binary) complete.")
display(station_buffers[existing].tail(40))

print("\nCultural raw (0.25mi) summary:\n", station_buffers["cultural_count_0p25mi"].describe())
print("\nCultural capped (0.25mi) summary:\n", station_buffers["cultural_count_0p25mi_capped10"].describe())
print("\nCultural binary (0.25mi):\n", station_buffers["has_cultural_0p25mi"].value_counts())

print("\nCultural raw (0.5mi) summary:\n", station_buffers["cultural_count_0p5mi"].describe())
print("\nCultural capped (0.5mi) summary:\n", station_buffers["cultural_count_0p5mi_capped10"].describe())
print("\nCultural binary (0.5mi):\n", station_buffers["has_cultural_0p5mi"].value_counts())


Loading DCP Facilities Database (paged)…
⬇️ Loaded 34,708 features (offset=0)
✅ Facilities loaded with geometry: 34,708
✅ has_university_0p25mi: flagged=113 (radius=0.25mi)
✅ has_university_0p5mi: flagged=217 (radius=0.5mi)
✅ has_hospital_0p25mi: flagged=39 (radius=0.25mi)
✅ has_hospital_0p5mi: flagged=122 (radius=0.5mi)
✅ has_stadium_0p1mi: flagged=0 (radius=0.1mi)
✅ cultural_count_0p25mi: mean=9.97, max=82 (radius=0.25mi)
✅ cultural_count_0p5mi: mean=34.11, max=207 (radius=0.5mi)
✅ has_cultural_0p25mi + cultural_count_0p25mi_capped10 created (raw=cultural_count_0p25mi)
✅ has_cultural_0p5mi + cultural_count_0p5mi_capped10 created (raw=cultural_count_0p5mi)
Loading NYC Parks Properties (paged)…
⬇️ Loaded 2,056 features (offset=0)
✅ Parks loaded with geometry: 2,056
✅ Major parks (>=50 acres): 95
✅ adjacent_major_park_0p25mi: flagged=87 (radius=0.25mi)
✅ adjacent_major_park_0p5mi: flagged=163 (radius=0.5mi)
✅ Anchor variables (parameterized radii + cultural raw/cap/binary) complete.


,stop_id,stop_name,has_university_0p25mi,has_university_0p5mi,has_hospital_0p25mi,has_hospital_0p5mi,has_stadium_0p1mi,cultural_count_0p25mi,cultural_count_0p25mi_capped10,has_cultural_0p25mi,cultural_count_0p5mi,cultural_count_0p5mi_capped10,has_cultural_0p5mi,adjacent_major_park_0p25mi,adjacent_major_park_0p5mi
478,S09,Tottenville,0,0,0,0,0,0,0,0,0,0,0,0,1
479,S11,Arthur Kill,0,0,0,0,0,0,0,0,0,0,0,0,1
480,S13,Richmond Valley,0,0,0,0,0,0,0,0,0,0,0,1,1
481,S14,Pleasant Plains,0,0,0,0,0,0,0,0,0,0,0,0,1
482,S15,Prince's Bay,0,0,0,0,0,1,1,1,1,1,1,1,1
483,S16,Huguenot,0,0,0,0,0,0,0,0,0,0,0,1,1
484,S17,Annadale,0,0,0,0,0,0,0,0,0,0,0,0,1
485,S18,Eltingville,0,0,0,0,0,2,2,1,2,2,1,0,1
486,S19,Great Kills,0,0,0,0,0,0,0,0,0,0,0,0,0
487,S20,Bay Terrace,0,0,0,0,0,0,0,0,0,0,0,0,1



Cultural raw (0.25mi) summary:
 count    518.000000
mean       9.965251
std       15.142864
min        0.000000
25%        1.000000
50%        3.000000
75%       12.000000
max       82.000000
Name: cultural_count_0p25mi, dtype: float64

Cultural capped (0.25mi) summary:
 count    518.000000
mean       4.289575
std        4.082932
min        0.000000
25%        1.000000
50%        3.000000
75%       10.000000
max       10.000000
Name: cultural_count_0p25mi_capped10, dtype: float64

Cultural binary (0.25mi):
 has_cultural_0p25mi
1    391
0    127
Name: count, dtype: int64

Cultural raw (0.5mi) summary:
 count    518.000000
mean      34.110039
std       49.872269
min        0.000000
25%        3.000000
50%        9.000000
75%       43.000000
max      207.000000
Name: cultural_count_0p5mi, dtype: float64

Cultural capped (0.5mi) summary:
 count    518.000000
mean       6.677606
std        3.807038
min        0.000000
25%        3.000000
50%        9.000000
75%       10.000000
max       10